<a href="https://colab.research.google.com/github/henryonomakpo/The-Impact-of-ESG-Ratings-on-EV-Manufacturing-Industry/blob/main/Panel_Regression_ESG_data_for_Transportation%2C_3PL_providers%2C_and_Courier_service_sector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Required libraries, yfinance, statsmodels, pandas, numpy, scikit-learn, xlsxwriter, linearmodels
!pip install yesg
!pip install yfinance
!pip install statsmodels
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install xlsxwriter
!pip install linearmodels

  Preparing metadata (setup.py) ... done
  Created wheel for yesg: filename=yesg-2.1.1-py3-none-any.whl size=6105 sha256=b174c59a864eca96b8dd0a568c66b5e0c87a4641ed7d61b7a48cac2585b52579
  Stored in directory: /root/.cache/pip/wheels/78/8d/48/f5e8ff0315a46301e15c68371e297b460b33e1c846117725bc
Successfully built yesg
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.8 MB/s eta 0:00:00


### Fetch ESG data for Transportation, 3PL providers, and Courier service sector

In [ ]:
# Required libraries: yesg, pandas
# Optional for Google Drive: google.colab
# If running locally, you may need to install these:
# !pip install yesg pandas

import yesg
import pandas as pd
import time # To add delays between API calls
import warnings # To potentially suppress warnings

# Attempt to import and use Google Drive specific libraries only if needed
try:
    from google.colab import drive
    google_colab_available = True
except ImportError:
    google_colab_available = False
    print("Google Colab environment not detected. Will save CSV locally.")

print("--- ESG Data Fetching Script for Global Transportation Firms ---")

# --- Configuration ---

# List of tickers for Global Transportation firms
# Note: Data availability may vary significantly by ticker and source.
TICKERS_GLOBAL_TRANSPORTATION = [
    'UPS',    # United Parcel Service (NYSE)
    'FDX',    # FedEx Corporation (NYSE)
    'DPSGY',  # Deutsche Post DHL Group (OTC)
    'AMKBY',  # A.P. Møller - Mærsk (OTC)
    'KHNGY',  # Kuehne + Nagel International (OTC)
    'DSDVY',  # DSV A/S (OTC)
    'UNP',    # Union Pacific Corp. (NYSE)
    'CNI',    # Canadian National Railway (NYSE)
    'CP',     # Canadian Pacific Kansas City (NYSE)
    'CSX',    # CSX Corporation (NASDAQ)
    'XPO',    # XPO, Inc. (NYSE)
    'ODFL',   # Old Dominion Freight Line (NASDAQ)
    'JBHT',   # J.B. Hunt Transport (NASDAQ)
    'ZIM',    # ZIM Integrated Shipping (NYSE)
    'JYD',    # Jayud Global Logistics (NASDAQ) - Note: Low Market Cap, data might be sparse
]

# Optional mapping for clearer output
TICKER_NAMES = {
    'UPS': 'United Parcel Service', 'FDX': 'FedEx Corporation',
    'DPSGY': 'Deutsche Post DHL Group', 'AMKBY': 'A.P. Møller - Mærsk',
    'KHNGY': 'Kuehne + Nagel International', 'DSDVY': 'DSV A/S',
    'UNP': 'Union Pacific Corp.', 'CNI': 'Canadian National Railway',
    'CP': 'Canadian Pacific Kansas City', 'CSX': 'CSX Corporation',
    'XPO': 'XPO, Inc.', 'ODFL': 'Old Dominion Freight Line',
    'JBHT': 'J.B. Hunt Transport', 'ZIM': 'ZIM Integrated Shipping',
    'JYD': 'Jayud Global Logistics'
}

# Define where to save the output file
DRIVE_MOUNT_PATH = '/content/drive'
OUTPUT_FILENAME = 'historic_esg_scores_global_transportation.csv'
OUTPUT_PATH_DRIVE = f'{DRIVE_MOUNT_PATH}/My Drive/{OUTPUT_FILENAME}' # Standard Google Drive path
OUTPUT_PATH_LOCAL = OUTPUT_FILENAME # Save in current directory if Drive fails

# Delay between API calls (in seconds) to avoid potential blocking
API_DELAY = 0.8 # Slightly increased delay as a precaution

# --- Mount Google Drive (if in Colab) ---
drive_mounted = False
save_path = OUTPUT_PATH_LOCAL # Default save path

if google_colab_available:
    try:
        print(f"\nAttempting to mount Google Drive at {DRIVE_MOUNT_PATH}...")
        # Suppress specific warnings that might appear during mounting
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            drive.mount(DRIVE_MOUNT_PATH, force_remount=True) # force_remount=True can help with issues

        drive_mounted = True
        save_path = OUTPUT_PATH_DRIVE
        print("Google Drive mounted successfully.")
        print(f"Output CSV will be saved to Google Drive at '{save_path}'.")

    except Exception as e:
        print(f"Failed to mount Google Drive: {e}")
        print(f"Output CSV will be saved locally as '{OUTPUT_PATH_LOCAL}'.")
else:
    # Not in Colab, saving locally
    print(f"\nOutput CSV will be saved locally as '{OUTPUT_PATH_LOCAL}'.")


# --- Data Fetching Loop ---
print(f"\nTickers to fetch ESG data for ({len(TICKERS_GLOBAL_TRANSPORTATION)} total): {TICKERS_GLOBAL_TRANSPORTATION}")
print("Starting ESG data download loop...")
print("WARNING: 'yesg' library relies on Yahoo Finance and may be outdated or have limited data coverage.")
print("Note: ESG data is typically published annually, so expect one data point per year if available.")


# Initialize lists to store results and track progress
all_esg_data_list = []
successful_tickers = []
failed_tickers = []

for ticker in TICKERS_GLOBAL_TRANSPORTATION:
    ticker_name = TICKER_NAMES.get(ticker, ticker) # Use mapped name if available
    print(f"  -> Processing: {ticker} ({ticker_name})")
    try:
        # Add the delay BEFORE the API call to space them out
        time.sleep(API_DELAY)
        # Fetch all available historic ESG ratings for the ticker
        esg_scores_df = yesg.get_historic_esg(ticker)

        # Check if the result is a non-empty DataFrame
        if isinstance(esg_scores_df, pd.DataFrame) and not esg_scores_df.empty:
            # Add a column for the ticker symbol
            esg_scores_df['Ticker'] = ticker
            # Reset the index to make the date a column before appending
            # The date column is typically the index after fetching
            esg_scores_df = esg_scores_df.reset_index()
            # Rename the date column if needed (common names are 'Date' or 'index')
            if 'index' in esg_scores_df.columns:
                esg_scores_df = esg_scores_df.rename(columns={'index': 'Date'})

            all_esg_data_list.append(esg_scores_df)
            successful_tickers.append(ticker)
            print(f"    -> Success: Found {len(esg_scores_df)} historic ESG data points for {ticker} ({ticker_name}).")
        else:
            # Handle cases where yesg returns None or an empty DataFrame
            print(f"    -> No valid historic ESG data found/returned for {ticker} ({ticker_name}).")
            failed_tickers.append(ticker)

    except Exception as e:
        # Catch any other exceptions during fetching or processing
        print(f"    -> ERROR fetching/processing historic ESG data for {ticker} ({ticker_name}): {e}")
        failed_tickers.append(ticker)

# --- Combine and Save Data ---
if all_esg_data_list:
    print("\nCombining collected historic ESG data...")
    # Concatenate all the collected DataFrames into a single one
    final_esg_data = pd.concat(all_esg_data_list, ignore_index=True)

    # Ensure the Date column is correctly named and formatted if possible
    if 'Date' in final_esg_data.columns:
        try:
            # Attempt to convert Date column to datetime objects for consistency
            final_esg_data['Date'] = pd.to_datetime(final_esg_data['Date'])
            # Sort by Ticker and Date for clarity
            final_esg_data = final_esg_data.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)
            print("  -> Date column converted to datetime and data sorted.")
        except Exception as e:
            print(f"Warning: Could not convert 'Date' column to datetime format or sort data: {e}")
            print("Please inspect the Date column format manually.")
    else:
        print("Warning: 'Date' column not found in combined data. Please inspect the output DataFrame structure.")


    # Display first few rows and info of the final DataFrame
    print("\nPreview of combined historic ESG data:")
    print(final_esg_data.head())
    print("\nData Info:")
    final_esg_data.info()
    print(f"\nTotal rows collected: {len(final_esg_data)}")


    # Save the combined data to the chosen CSV file path
    # Check if the save_path variable was set correctly (especially in Colab failure scenario)
    if save_path:
        try:
            print(f"\nSaving historic ESG data to: {save_path} ...")
            final_esg_data.to_csv(save_path, index=False)
            print(f"Historic ESG data saved successfully.")
        except Exception as e:
            print(f"\nERROR saving historic ESG data to CSV at '{save_path}': {e}")
            if google_colab_available and save_path == OUTPUT_PATH_DRIVE:
                print("Check if your Google Drive is correctly mounted and you have write permissions.")
    else:
         print("\nERROR: Save path was not determined. Cannot save CSV.")


else:
    # Message if no data was collected at all
    print("\nNo historic ESG data was successfully collected for any ticker. No CSV file created.")

# --- Final Summary ---
print("\n--- Historic ESG Fetching Summary ---")
print(f"Successfully fetched historic ESG data for ({len(successful_tickers)} tickers): {successful_tickers}")
print(f"Failed or no historic ESG data for ({len(failed_tickers)} tickers): {failed_tickers}")
print("--- Script Finished ---")

--- ESG Data Fetching Script for Global Transportation Firms ---

Attempting to mount Google Drive at /content/drive...
Failed to mount Google Drive: mount failed
Output CSV will be saved locally as 'historic_esg_scores_global_transportation.csv'.

Tickers to fetch ESG data for (15 total): ['UPS', 'FDX', 'DPSGY', 'AMKBY', 'KHNGY', 'DSDVY', 'UNP', 'CNI', 'CP', 'CSX', 'XPO', 'ODFL', 'JBHT', 'ZIM', 'JYD']
Starting ESG data download loop...
Note: ESG data is typically published annually, so expect one data point per year if available.
  -> Processing: UPS (United Parcel Service)
    -> Success: Found 128 historic ESG data points for UPS (United Parcel Service).
  -> Processing: FDX (FedEx Corporation)
    -> Success: Found 128 historic ESG data points for FDX (FedEx Corporation).
  -> Processing: DPSGY (Deutsche Post DHL Group)
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
    -> No valid historic ESG data found/returned for DPSGY (Deutsche 

### 2 *Fetch ESG Data for Pharma Companies

In [ ]:
# Required libraries: yesg, pandas
# Optional for Google Drive: google.colab
# If running locally, you may need to install these:
# !pip install yesg pandas

import yesg
import pandas as pd
import time # To add delays between API calls
import warnings # To potentially suppress warnings

# Attempt to import and use Google Drive specific libraries only if needed
try:
    from google.colab import drive
    google_colab_available = True
except ImportError:
    google_colab_available = False
    print("Google Colab environment not detected. Will save CSV locally.")

print("--- ESG Data Fetching Script for Pharmaceutical & Healthcare Firms ---")

# --- Configuration ---

# List of tickers for Pharmaceutical & Healthcare firms
# Note: Data availability may vary significantly by ticker and source.
TICKERS_PHARMA_HEALTHCARE = [
    'LLY',    # Eli Lilly and Company (NYSE)
    'JNJ',    # Johnson & Johnson (NYSE)
    'MRK',    # Merck & Co. (NYSE)
    'NVO',    # Novo Nordisk A/S (NYSE)
    'RHHBY',  # Roche Holding AG (OTC)
    'PFE',    # Pfizer Inc. (NYSE)
    'ABBV',   # AbbVie Inc. (NYSE)
    'NVS',    # Novartis AG (NYSE)
    'AZN',    # AstraZeneca PLC (NASDAQ/LSE)
    'SNY',    # Sanofi (NASDAQ/EPA)
    'BMY',    # Bristol Myers Squibb (NYSE)
    'GSK',    # GSK plc (NYSE/LSE)
    'TAK',    # Takeda Pharmaceutical (NYSE)
]

# Optional mapping for clearer output
TICKER_NAMES = {
    'LLY': 'Eli Lilly and Company', 'JNJ': 'Johnson & Johnson',
    'MRK': 'Merck & Co.', 'NVO': 'Novo Nordisk A/S',
    'RHHBY': 'Roche Holding AG', 'PFE': 'Pfizer Inc.',
    'ABBV': 'AbbVie Inc.', 'NVS': 'Novartis AG',
    'AZN': 'AstraZeneca PLC', 'SNY': 'Sanofi',
    'BMY': 'Bristol Myers Squibb', 'GSK': 'GSK plc',
    'TAK': 'Takeda Pharmaceutical',
}

# Define where to save the output file
DRIVE_MOUNT_PATH = '/content/drive'
OUTPUT_FILENAME = 'historic_esg_scores_pharma_healthcare.csv'
OUTPUT_PATH_DRIVE = f'{DRIVE_MOUNT_PATH}/My Drive/{OUTPUT_FILENAME}' # Standard Google Drive path
OUTPUT_PATH_LOCAL = OUTPUT_FILENAME # Save in current directory if Drive fails

# Delay between API calls (in seconds) to avoid potential blocking
API_DELAY = 0.8 # Slightly increased delay as a precaution

# --- Mount Google Drive (if in Colab) ---
drive_mounted = False
save_path = OUTPUT_PATH_LOCAL # Default save path

if google_colab_available:
    try:
        print(f"\nAttempting to mount Google Drive at {DRIVE_MOUNT_PATH}...")
        # Suppress specific warnings that might appear during mounting
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            drive.mount(DRIVE_MOUNT_PATH, force_remount=True) # force_remount=True can help with issues

        drive_mounted = True
        save_path = OUTPUT_PATH_DRIVE
        print("Google Drive mounted successfully.")
        print(f"Output CSV will be saved to Google Drive at '{save_path}'.")

    except Exception as e:
        print(f"Failed to mount Google Drive: {e}")
        print(f"Output CSV will be saved locally as '{OUTPUT_PATH_LOCAL}'.")
else:
    # Not in Colab, saving locally
    print(f"\nOutput CSV will be saved locally as '{OUTPUT_PATH_LOCAL}'.")


# --- Data Fetching Loop ---
print(f"\nTickers to fetch ESG data for ({len(TICKERS_PHARMA_HEALTHCARE)} total): {TICKERS_PHARMA_HEALTHCARE}")
print("Starting ESG data download loop...")
print("WARNING: 'yesg' library relies on Yahoo Finance and may be outdated or have limited data coverage.")
print("Note: ESG data is typically published annually, so expect one data point per year if available.")


# Initialize lists to store results and track progress
all_esg_data_list = []
successful_tickers = []
failed_tickers = []

for ticker in TICKERS_PHARMA_HEALTHCARE:
    ticker_name = TICKER_NAMES.get(ticker, ticker) # Use mapped name if available
    print(f"  -> Processing: {ticker} ({ticker_name})")
    try:
        # Add the delay BEFORE the API call to space them out
        time.sleep(API_DELAY)
        # Fetch all available historic ESG ratings for the ticker
        esg_scores_df = yesg.get_historic_esg(ticker)

        # Check if the result is a non-empty DataFrame
        if isinstance(esg_scores_df, pd.DataFrame) and not esg_scores_df.empty:
            # Add a column for the ticker symbol
            esg_scores_df['Ticker'] = ticker
            # Reset the index to make the date a column before appending
            # The date column is typically the index after fetching
            esg_scores_df = esg_scores_df.reset_index()
            # Rename the date column if needed (common names are 'Date' or 'index')
            if 'index' in esg_scores_df.columns:
                esg_scores_df = esg_scores_df.rename(columns={'index': 'Date'})

            all_esg_data_list.append(esg_scores_df)
            successful_tickers.append(ticker)
            print(f"    -> Success: Found {len(esg_scores_df)} historic ESG data points for {ticker} ({ticker_name}).")
        else:
            # Handle cases where yesg returns None or an empty DataFrame
            print(f"    -> No valid historic ESG data found/returned for {ticker} ({ticker_name}).")
            failed_tickers.append(ticker)

    except Exception as e:
        # Catch any other exceptions during fetching or processing
        print(f"    -> ERROR fetching/processing historic ESG data for {ticker} ({ticker_name}): {e}")
        failed_tickers.append(ticker)

# --- Combine and Save Data ---
if all_esg_data_list:
    print("\nCombining collected historic ESG data...")
    # Concatenate all the collected DataFrames into a single one
    final_esg_data = pd.concat(all_esg_data_list, ignore_index=True)

    # Ensure the Date column is correctly named and formatted if possible
    if 'Date' in final_esg_data.columns:
        try:
            # Attempt to convert Date column to datetime objects for consistency
            final_esg_data['Date'] = pd.to_datetime(final_esg_data['Date'])
            # Sort by Ticker and Date for clarity
            final_esg_data = final_esg_data.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)
            print("  -> Date column converted to datetime and data sorted.")
        except Exception as e:
            print(f"Warning: Could not convert 'Date' column to datetime format or sort data: {e}")
            print("Please inspect the Date column format manually.")
    else:
        print("Warning: 'Date' column not found in combined data. Please inspect the output DataFrame structure.")


    # Display first few rows and info of the final DataFrame
    print("\nPreview of combined historic ESG data:")
    print(final_esg_data.head())
    print("\nData Info:")
    final_esg_data.info()
    print(f"\nTotal rows collected: {len(final_esg_data)}")


    # Save the combined data to the chosen CSV file path
    # Check if the save_path variable was set correctly (especially in Colab failure scenario)
    if save_path:
        try:
            print(f"\nSaving historic ESG data to: {save_path} ...")
            final_esg_data.to_csv(save_path, index=False)
            print(f"Historic ESG data saved successfully.")
        except Exception as e:
            print(f"\nERROR saving historic ESG data to CSV at '{save_path}': {e}")
            if google_colab_available and save_path == OUTPUT_PATH_DRIVE:
                print("Check if your Google Drive is correctly mounted and you have write permissions.")
    else:
         print("\nERROR: Save path was not determined. Cannot save CSV.")


else:
    # Message if no data was collected at all
    print("\nNo historic ESG data was successfully collected for any ticker. No CSV file created.")

# --- Final Summary ---
print("\n--- Historic ESG Fetching Summary ---")
print(f"Successfully fetched historic ESG data for ({len(successful_tickers)} tickers): {successful_tickers}")
print(f"Failed or no historic ESG data for ({len(failed_tickers)} tickers): {failed_tickers}")
print("--- Script Finished ---")

--- ESG Data Fetching Script for Pharmaceutical & Healthcare Firms ---

Attempting to mount Google Drive at /content/drive...
Failed to mount Google Drive: mount failed
Output CSV will be saved locally as 'historic_esg_scores_pharma_healthcare.csv'.

Tickers to fetch ESG data for (13 total): ['LLY', 'JNJ', 'MRK', 'NVO', 'RHHBY', 'PFE', 'ABBV', 'NVS', 'AZN', 'SNY', 'BMY', 'GSK', 'TAK']
Starting ESG data download loop...
Note: ESG data is typically published annually, so expect one data point per year if available.
  -> Processing: LLY (Eli Lilly and Company)
    -> Success: Found 128 historic ESG data points for LLY (Eli Lilly and Company).
  -> Processing: JNJ (Johnson & Johnson)
    -> Success: Found 128 historic ESG data points for JNJ (Johnson & Johnson).
  -> Processing: MRK (Merck & Co.)
    -> Success: Found 128 historic ESG data points for MRK (Merck & Co.).
  -> Processing: NVO (Novo Nordisk A/S)
An error has occurred. The ticker symbol might be wrong or you might need to wait 

### Fetch Construction Firms ESG Data

In [ ]:
# Required libraries: yesg, pandas
# If running locally, you need to install these:
# !pip install yesg pandas

import yesg
import pandas as pd
import time # To add delays between API calls
import warnings # To potentially suppress warnings

print("--- ESG Data Fetching Script for Civil Construction, Engineering & Materials Firms ---")

# --- Configuration ---

# List of tickers for Civil Construction, Engineering & Materials firms
# Note: Data availability may vary significantly by ticker and source.
TICKERS_CONSTRUCTION = [
    'DG.PA',  # VINCI SA (Euronext Paris)
    'ACS.MC', # ACS Actividades Const. Ser. (Bolsa de Madrid)
    'PWR',    # Quanta Services, Inc. (NYSE)
    'HCMLY',  # Holcim Ltd. (OTC)
    'VMC',    # Vulcan Materials Company (NYSE)
    'MLM',    # Martin Marietta Materials (NYSE)
    'CRH',    # CRH plc (NYSE/LSE)
    'ACM',    # AECOM (NYSE)
    'J',      # Jacobs Solutions Inc. (NYSE)
    'FLR',    # Fluor Corporation (NYSE)
    'MTZ',    # MasTec, Inc. (NYSE)
    'HEI.DE', # Heidelberg Materials AG (Frankfurt)
    'LT.NS',  # Larsen & Toubro Ltd. (NSE India)
]

# Optional mapping for clearer output
TICKER_NAMES = {
    'DG.PA': 'VINCI SA', 'ACS.MC': 'ACS Actividades Const. Ser.',
    'PWR': 'Quanta Services, Inc.', 'HCMLY': 'Holcim Ltd.',
    'VMC': 'Vulcan Materials Company', 'MLM': 'Martin Marietta Materials',
    'CRH': 'CRH plc', 'ACM': 'AECOM', 'J': 'Jacobs Solutions Inc.',
    'FLR': 'Fluor Corporation', 'MTZ': 'MasTec, Inc.',
    'HEI.DE': 'Heidelberg Materials AG', 'LT.NS': 'Larsen & Toubro Ltd.',
}

# Define where to save the output file (locally)
OUTPUT_FILENAME = 'historic_esg_scores_construction.csv'
OUTPUT_PATH_LOCAL = OUTPUT_FILENAME # Save in current directory

# Delay between API calls (in seconds) to avoid potential blocking
API_DELAY = 0.8 # Slightly increased delay as a precaution

# --- Data Fetching Loop ---
print(f"\nTickers to fetch ESG data for ({len(TICKERS_CONSTRUCTION)} total): {TICKERS_CONSTRUCTION}")
print("Starting ESG data download loop...")
print("WARNING: 'yesg' library relies on Yahoo Finance and may be outdated or have limited data coverage.")
print("Note: ESG data is typically published annually, so expect one data point per year if available.")


# Initialize lists to store results and track progress
all_esg_data_list = []
successful_tickers = []
failed_tickers = []

for ticker in TICKERS_CONSTRUCTION:
    ticker_name = TICKER_NAMES.get(ticker, ticker) # Use mapped name if available
    print(f"  -> Processing: {ticker} ({ticker_name})")
    try:
        # Add the delay BEFORE the API call to space them out
        time.sleep(API_DELAY)
        # Fetch all available historic ESG ratings for the ticker
        esg_scores_df = yesg.get_historic_esg(ticker)

        # Check if the result is a non-empty DataFrame
        if isinstance(esg_scores_df, pd.DataFrame) and not esg_scores_df.empty:
            # Add a column for the ticker symbol
            esg_scores_df['Ticker'] = ticker
            # Reset the index to make the date a column before appending
            # The date column is typically the index after fetching
            esg_scores_df = esg_scores_df.reset_index()
            # Rename the date column if needed (common names are 'Date' or 'index')
            if 'index' in esg_scores_df.columns:
                esg_scores_df = esg_scores_df.rename(columns={'index': 'Date'})

            all_esg_data_list.append(esg_scores_df)
            successful_tickers.append(ticker)
            print(f"    -> Success: Found {len(esg_scores_df)} historic ESG data points for {ticker} ({ticker_name}).")
        else:
            # Handle cases where yesg returns None or an empty DataFrame
            print(f"    -> No valid historic ESG data found/returned for {ticker} ({ticker_name}).")
            failed_tickers.append(ticker)

    except Exception as e:
        # Catch any other exceptions during fetching or processing
        print(f"    -> ERROR fetching/processing historic ESG data for {ticker} ({ticker_name}): {e}")
        failed_tickers.append(ticker)

# --- Combine and Save Data ---
if all_esg_data_list:
    print("\nCombining collected historic ESG data...")
    # Concatenate all the collected DataFrames into a single one
    final_esg_data = pd.concat(all_esg_data_list, ignore_index=True)

    # Ensure the Date column is correctly named and formatted if possible
    if 'Date' in final_esg_data.columns:
        try:
            # Attempt to convert Date column to datetime objects for consistency
            final_esg_data['Date'] = pd.to_datetime(final_esg_data['Date'])
            # Sort by Ticker and Date for clarity
            final_esg_data = final_esg_data.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)
            print("  -> Date column converted to datetime and data sorted.")
        except Exception as e:
            print(f"Warning: Could not convert 'Date' column to datetime format or sort data: {e}")
            print("Please inspect the Date column format manually.")
    else:
        print("Warning: 'Date' column not found in combined data. Please inspect the output DataFrame structure.")


    # Display first few rows and info of the final DataFrame
    print("\nPreview of combined historic ESG data:")
    print(final_esg_data.head())
    print("\nData Info:")
    final_esg_data.info()
    print(f"\nTotal rows collected: {len(final_esg_data)}")


    # Save the combined data to the chosen CSV file path (local)
    try:
        print(f"\nSaving historic ESG data to: {OUTPUT_PATH_LOCAL} ...")
        final_esg_data.to_csv(OUTPUT_PATH_LOCAL, index=False)
        print(f"Historic ESG data saved successfully.")
    except Exception as e:
        print(f"\nERROR saving historic ESG data to CSV at '{OUTPUT_PATH_LOCAL}': {e}")


else:
    # Message if no data was collected at all
    print("\nNo historic ESG data was successfully collected for any ticker. No CSV file created.")

# --- Final Summary ---
print("\n--- Historic ESG Fetching Summary ---")
print(f"Successfully fetched historic ESG data for ({len(successful_tickers)} tickers): {successful_tickers}")
print(f"Failed or no historic ESG data for ({len(failed_tickers)} tickers): {failed_tickers}")
print("--- Script Finished ---")

--- ESG Data Fetching Script for Civil Construction, Engineering & Materials Firms ---

Tickers to fetch ESG data for (13 total): ['DG.PA', 'ACS.MC', 'PWR', 'HCMLY', 'VMC', 'MLM', 'CRH', 'ACM', 'J', 'FLR', 'MTZ', 'HEI.DE', 'LT.NS']
Starting ESG data download loop...
Note: ESG data is typically published annually, so expect one data point per year if available.
  -> Processing: DG.PA (VINCI SA)
    -> Success: Found 128 historic ESG data points for DG.PA (VINCI SA).
  -> Processing: ACS.MC (ACS Actividades Const. Ser.)
    -> Success: Found 128 historic ESG data points for ACS.MC (ACS Actividades Const. Ser.).
  -> Processing: PWR (Quanta Services, Inc.)
    -> Success: Found 128 historic ESG data points for PWR (Quanta Services, Inc.).
  -> Processing: HCMLY (Holcim Ltd.)
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
    -> No valid historic ESG data found/returned for HCMLY (Holcim Ltd.).
  -> Processing: VMC (Vulcan Materials Company)


### Fetch Crtical Earth Material - Mining ESG Data

In [ ]:
# Required libraries: yesg, pandas
# If running locally, you need to install these:
# !pip install yesg pandas

import yesg
import pandas as pd
import time # To add delays between API calls
import warnings # To potentially suppress warnings

print("--- ESG Data Fetching Script for Critical Materials, Mining & Rare Earths Firms ---")

# --- Configuration ---

# List of tickers for Critical Materials, Mining & Rare Earths firms
# Note: Data availability may vary significantly by ticker and source.
TICKERS_MINING_RE = [
    'BHP',    # BHP Group Ltd. (NYSE/ASX/LSE) - Using NYSE
    'RIO',    # Rio Tinto Group (NYSE/ASX/LSE) - Using NYSE
    'VALE',   # Vale S.A. (NYSE)
    'GLNCY',  # Glencore plc (OTC) - Using OTC for simplicity
    'FCX',    # Freeport-McMoRan Inc. (NYSE)
    'ALB',    # Albemarle Corporation (NYSE)
    'SQM',    # SQM (Soc. Química Minera) (NYSE)
    'MP',     # MP Materials Corp. (NYSE)
    'LYSCF',  # Lynas Rare Earths Ltd. (OTC) - Using OTC for simplicity
    'LAC',    # Lithium Americas Corp. (NYSE/TSX) - Using NYSE
    'ALTM',   # Arcadium Lithium plc (NYSE)
    'SGML',   # Sigma Lithium Corp. (NASDAQ/TSXV) - Using NASDAQ
    'UUUU',   # Energy Fuels Inc. (NYSEAM/TSX) - Using NYSEAM
    'ILKAF',  # Iluka Resources Ltd. (OTC) - Using OTC for simplicity
    'PILBF',  # Pilbara Minerals Ltd. (OTC) - Using OTC for simplicity
]

# Optional mapping for clearer output
TICKER_NAMES = {
    'BHP': 'BHP Group Ltd.', 'RIO': 'Rio Tinto Group', 'VALE': 'Vale S.A.',
    'GLNCY': 'Glencore plc', 'FCX': 'Freeport-McMoRan Inc.', 'ALB': 'Albemarle Corporation',
    'SQM': 'SQM (Soc. Química Minera)', 'MP': 'MP Materials Corp.',
    'LYSCF': 'Lynas Rare Earths Ltd.', 'LAC': 'Lithium Americas Corp.',
    'ALTM': 'Arcadium Lithium plc', 'SGML': 'Sigma Lithium Corp.',
    'UUUU': 'Energy Fuels Inc.', 'ILKAF': 'Iluka Resources Ltd.',
    'PILBF': 'Pilbara Minerals Ltd.',
}

# Define where to save the output file (locally)
OUTPUT_FILENAME = 'historic_esg_scores_mining_re.csv'
OUTPUT_PATH_LOCAL = OUTPUT_FILENAME # Save in current directory

# Delay between API calls (in seconds) to avoid potential blocking
API_DELAY = 0.8 # Slightly increased delay as a precaution

# --- Data Fetching Loop ---
print(f"\nTickers to fetch ESG data for ({len(TICKERS_MINING_RE)} total): {TICKERS_MINING_RE}")
print("Starting ESG data download loop...")
print("WARNING: 'yesg' library relies on Yahoo Finance and may be outdated or have limited data coverage.")
print("Note: ESG data is typically published annually, so expect one data point per year if available.")


# Initialize lists to store results and track progress
all_esg_data_list = []
successful_tickers = []
failed_tickers = []

for ticker in TICKERS_MINING_RE:
    ticker_name = TICKER_NAMES.get(ticker, ticker) # Use mapped name if available
    print(f"  -> Processing: {ticker} ({ticker_name})")
    try:
        # Add the delay BEFORE the API call to space them out
        time.sleep(API_DELAY)
        # Fetch all available historic ESG ratings for the ticker
        esg_scores_df = yesg.get_historic_esg(ticker)

        # Check if the result is a non-empty DataFrame
        if isinstance(esg_scores_df, pd.DataFrame) and not esg_scores_df.empty:
            # Add a column for the ticker symbol
            esg_scores_df['Ticker'] = ticker
            # Reset the index to make the date a column before appending
            # The date column is typically the index after fetching
            esg_scores_df = esg_scores_df.reset_index()
            # Rename the date column if needed (common names are 'Date' or 'index')
            if 'index' in esg_scores_df.columns:
                esg_scores_df = esg_scores_df.rename(columns={'index': 'Date'})

            all_esg_data_list.append(esg_scores_df)
            successful_tickers.append(ticker)
            print(f"    -> Success: Found {len(esg_scores_df)} historic ESG data points for {ticker} ({ticker_name}).")
        else:
            # Handle cases where yesg returns None or an empty DataFrame
            print(f"    -> No valid historic ESG data found/returned for {ticker} ({ticker_name}).")
            failed_tickers.append(ticker)

    except Exception as e:
        # Catch any other exceptions during fetching or processing
        print(f"    -> ERROR fetching/processing historic ESG data for {ticker} ({ticker_name}): {e}")
        failed_tickers.append(ticker)

# --- Combine and Save Data ---
if all_esg_data_list:
    print("\nCombining collected historic ESG data...")
    # Concatenate all the collected DataFrames into a single one
    final_esg_data = pd.concat(all_esg_data_list, ignore_index=True)

    # Ensure the Date column is correctly named and formatted if possible
    if 'Date' in final_esg_data.columns:
        try:
            # Attempt to convert Date column to datetime objects for consistency
            final_esg_data['Date'] = pd.to_datetime(final_esg_data['Date'])
            # Sort by Ticker and Date for clarity
            final_esg_data = final_esg_data.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)
            print("  -> Date column converted to datetime and data sorted.")
        except Exception as e:
            print(f"Warning: Could not convert 'Date' column to datetime format or sort data: {e}")
            print("Please inspect the Date column format manually.")
    else:
        print("Warning: 'Date' column not found in combined data. Please inspect the output DataFrame structure.")


    # Display first few rows and info of the final DataFrame
    print("\nPreview of combined historic ESG data:")
    print(final_esg_data.head())
    print("\nData Info:")
    final_esg_data.info()
    print(f"\nTotal rows collected: {len(final_esg_data)}")


    # Save the combined data to the chosen CSV file path (local)
    try:
        print(f"\nSaving historic ESG data to: {OUTPUT_PATH_LOCAL} ...")
        final_esg_data.to_csv(OUTPUT_PATH_LOCAL, index=False)
        print(f"Historic ESG data saved successfully.")
    except Exception as e:
        print(f"\nERROR saving historic ESG data to CSV at '{OUTPUT_PATH_LOCAL}': {e}")


else:
    # Message if no data was collected at all
    print("\nNo historic ESG data was successfully collected for any ticker. No CSV file created.")

# --- Final Summary ---
print("\n--- Historic ESG Fetching Summary ---")
print(f"Successfully fetched historic ESG data for ({len(successful_tickers)} tickers): {successful_tickers}")
print(f"Failed or no historic ESG data for ({len(failed_tickers)} tickers): {failed_tickers}")
print("--- Script Finished ---")

--- ESG Data Fetching Script for Critical Materials, Mining & Rare Earths Firms ---

Tickers to fetch ESG data for (15 total): ['BHP', 'RIO', 'VALE', 'GLNCY', 'FCX', 'ALB', 'SQM', 'MP', 'LYSCF', 'LAC', 'ALTM', 'SGML', 'UUUU', 'ILKAF', 'PILBF']
Starting ESG data download loop...
Note: ESG data is typically published annually, so expect one data point per year if available.
  -> Processing: BHP (BHP Group Ltd.)
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
    -> No valid historic ESG data found/returned for BHP (BHP Group Ltd.).
  -> Processing: RIO (Rio Tinto Group)
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
    -> No valid historic ESG data found/returned for RIO (Rio Tinto Group).
  -> Processing: VALE (Vale S.A.)
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
    -> No valid historic ESG data found/returned for VALE (Vale S.A.).
  -> Pr

#### Regression and Panel Regression Python code

### 3* Fetch Electronics Firm's ESG Data

In [ ]:
# Required libraries: yesg, pandas
# If running locally, you need to install these:
# !pip install yesg pandas

import yesg
import pandas as pd
import time # To add delays between API calls
import warnings # To potentially suppress warnings

print("--- ESG Data Fetching Script for Global Electronic Manufacturers ---")

# --- Configuration ---

# List of tickers for Global Electronic Manufacturers
# Prioritizing major US exchanges (NASDAQ, NYSE) or primary local ones if available via Yahoo Finance
# Note: Data availability may vary significantly by ticker and source.
TICKERS_ELECTRONICS = [
    'AAPL',   # Apple Inc. (NASDAQ)
    '005930.KS', # Samsung Electronics (KRX) - Yahoo Finance often uses .KS
    'SONY',   # Sony Group Corporation (NYSE) - Using NYSE ticker
    '6752.T', # Panasonic Holdings (TSE) - Using TSE ticker
    '066570.KS',# LG Electronics (KRX) - Yahoo Finance often uses .KS
    '2317.TW',# Foxconn (Hon Hai) (TPE) - Yahoo Finance often uses .TW
    'PHG',    # Philips (NYSE) - Using NYSE ticker
    'SIEGY',  # Siemens AG (OTC) - Using OTC for simplicity
    'CAJ',    # Canon Inc. (NYSE) - Using NYSE ticker
    '6753.T', # Sharp Corporation (TSE) - Using TSE ticker
    'TXN',    # Texas Instruments (NASDAQ)
    'INTC',   # Intel Corporation (NASDAQ)
    'NXPI',   # NXP Semiconductors (NASDAQ)
    'STM',    # STMicroelectronics (NYSE) - Using NYSE ticker
    'IFX.DE', # Infineon Technologies (FWB) - Using FWB ticker
    'ASML',   # ASML Holding (NASDAQ) - Using NASDAQ ticker
    'MU',     # Micron Technology (NASDAQ)
    'ADI',    # Analog Devices (NASDAQ)
    'AVGO',   # Broadcom Inc. (NASDAQ)
    '6502.T', # Toshiba Corporation (TSE) - Using TSE ticker
]

# Optional mapping for clearer output
TICKER_NAMES = {
    'AAPL': 'Apple Inc.', '005930.KS': 'Samsung Electronics', 'SONY': 'Sony Group Corporation',
    '6752.T': 'Panasonic Holdings', '066570.KS': 'LG Electronics', '2317.TW': 'Foxconn (Hon Hai)',
    'PHG': 'Philips', 'SIEGY': 'Siemens AG', 'CAJ': 'Canon Inc.', '6753.T': 'Sharp Corporation',
    'TXN': 'Texas Instruments', 'INTC': 'Intel Corporation', 'NXPI': 'NXP Semiconductors',
    'STM': 'STMicroelectronics', 'IFX.DE': 'Infineon Technologies', 'ASML': 'ASML Holding',
    'MU': 'Micron Technology', 'ADI': 'Analog Devices', 'AVGO': 'Broadcom Inc.',
    '6502.T': 'Toshiba Corporation',
}

# Define where to save the output file (locally)
OUTPUT_FILENAME = 'historic_esg_scores_electronics.csv'
OUTPUT_PATH_LOCAL = OUTPUT_FILENAME # Save in current directory

# Delay between API calls (in seconds) to avoid potential blocking
API_DELAY = 0.8 # Slightly increased delay as a precaution

# --- Data Fetching Loop ---
print(f"\nTickers to fetch ESG data for ({len(TICKERS_ELECTRONICS)} total): {TICKERS_ELECTRONICS}")
print("Starting ESG data download loop...")
print("WARNING: 'yesg' library relies on Yahoo Finance and may be outdated or have limited data coverage.")
print("Note: ESG data is typically published annually, so expect one data point per year if available.")


# Initialize lists to store results and track progress
all_esg_data_list = []
successful_tickers = []
failed_tickers = []

for ticker in TICKERS_ELECTRONICS:
    ticker_name = TICKER_NAMES.get(ticker, ticker) # Use mapped name if available
    print(f"  -> Processing: {ticker} ({ticker_name})")
    try:
        # Add the delay BEFORE the API call to space them out
        time.sleep(API_DELAY)
        # Fetch all available historic ESG ratings for the ticker
        esg_scores_df = yesg.get_historic_esg(ticker)

        # Check if the result is a non-empty DataFrame
        if isinstance(esg_scores_df, pd.DataFrame) and not esg_scores_df.empty:
            # Add a column for the ticker symbol
            esg_scores_df['Ticker'] = ticker
            # Reset the index to make the date a column before appending
            # The date column is typically the index after fetching
            esg_scores_df = esg_scores_df.reset_index()
            # Rename the date column if needed (common names are 'Date' or 'index')
            if 'index' in esg_scores_df.columns:
                esg_scores_df = esg_scores_df.rename(columns={'index': 'Date'})

            all_esg_data_list.append(esg_scores_df)
            successful_tickers.append(ticker)
            print(f"    -> Success: Found {len(esg_scores_df)} historic ESG data points for {ticker} ({ticker_name}).")
        else:
            # Handle cases where yesg returns None or an empty DataFrame
            print(f"    -> No valid historic ESG data found/returned for {ticker} ({ticker_name}).")
            failed_tickers.append(ticker)

    except Exception as e:
        # Catch any other exceptions during fetching or processing
        print(f"    -> ERROR fetching/processing historic ESG data for {ticker} ({ticker_name}): {e}")
        failed_tickers.append(ticker)

# --- Combine and Save Data ---
if all_esg_data_list:
    print("\nCombining collected historic ESG data...")
    # Concatenate all the collected DataFrames into a single one
    final_esg_data = pd.concat(all_esg_data_list, ignore_index=True)

    # Ensure the Date column is correctly named and formatted if possible
    if 'Date' in final_esg_data.columns:
        try:
            # Attempt to convert Date column to datetime objects for consistency
            final_esg_data['Date'] = pd.to_datetime(final_esg_data['Date'])
            # Sort by Ticker and Date for clarity
            final_esg_data = final_esg_data.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)
            print("  -> Date column converted to datetime and data sorted.")
        except Exception as e:
            print(f"Warning: Could not convert 'Date' column to datetime format or sort data: {e}")
            print("Please inspect the Date column format manually.")
    else:
        print("Warning: 'Date' column not found in combined data. Please inspect the output DataFrame structure.")


    # Display first few rows and info of the final DataFrame
    print("\nPreview of combined historic ESG data:")
    print(final_esg_data.head())
    print("\nData Info:")
    final_esg_data.info()
    print(f"\nTotal rows collected: {len(final_esg_data)}")


    # Save the combined data to the chosen CSV file path (local)
    try:
        print(f"\nSaving historic ESG data to: {OUTPUT_PATH_LOCAL} ...")
        final_esg_data.to_csv(OUTPUT_PATH_LOCAL, index=False)
        print(f"Historic ESG data saved successfully.")
    except Exception as e:
        print(f"\nERROR saving historic ESG data to CSV at '{OUTPUT_PATH_LOCAL}': {e}")


else:
    # Message if no data was collected at all
    print("\nNo historic ESG data was successfully collected for any ticker. No CSV file created.")

# --- Final Summary ---
print("\n--- Historic ESG Fetching Summary ---")
print(f"Successfully fetched historic ESG data for ({len(successful_tickers)} tickers): {successful_tickers}")
print(f"Failed or no historic ESG data for ({len(failed_tickers)} tickers): {failed_tickers}")
print("--- Script Finished ---")

--- ESG Data Fetching Script for Global Electronic Manufacturers ---

Tickers to fetch ESG data for (20 total): ['AAPL', '005930.KS', 'SONY', '6752.T', '066570.KS', '2317.TW', 'PHG', 'SIEGY', 'CAJ', '6753.T', 'TXN', 'INTC', 'NXPI', 'STM', 'IFX.DE', 'ASML', 'MU', 'ADI', 'AVGO', '6502.T']
Starting ESG data download loop...
Note: ESG data is typically published annually, so expect one data point per year if available.
  -> Processing: AAPL (Apple Inc.)
    -> Success: Found 128 historic ESG data points for AAPL (Apple Inc.).
  -> Processing: 005930.KS (Samsung Electronics)
    -> Success: Found 109 historic ESG data points for 005930.KS (Samsung Electronics).
  -> Processing: SONY (Sony Group Corporation)
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
    -> No valid historic ESG data found/returned for SONY (Sony Group Corporation).
  -> Processing: 6752.T (Panasonic Holdings)
    -> Success: Found 128 historic ESG data points for 6752.T (P

#### Transportation Regression and Panel Regression

In [ ]:
!pip install famafrench

In [ ]:
# !pip install yfinance statsmodels pandas numpy linearmodels pandas-datareader requests beautifulsoup4 lxml

# --- Core Libraries ---
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import PanelOLS, RandomEffects
from linearmodels.panel import compare as model_compare
from linearmodels.panel.results import PanelEffectsResults, RandomEffectsResults
import pandas_datareader.data as web # For Fama-French data
import requests                          # For downloading FF files (fallback, keep for now)
from io import BytesIO                   # *** Use BytesIO instead of StringIO ***
from zipfile import ZipFile              # For handling zipped FF files (fallback, keep for now)
import warnings
import sys
import re
import time
import traceback # For detailed error logging if needed
from datetime import datetime
from dateutil.relativedelta import relativedelta

# --- MICE Imputation Libraries (Kept) ---
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge # Needed for IterativeImputer estimator

# --- Removed Plotting Libraries ---

# --- Settings and Configuration ---
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
from statsmodels.tools.sm_exceptions import (ValueWarning, ConvergenceWarning,
                                             HessianInversionWarning, PerfectSeparationWarning,
                                             CollinearityWarning, PerfectSeparationError)
warnings.simplefilter('ignore', ValueWarning)
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', HessianInversionWarning)
warnings.simplefilter('ignore', PerfectSeparationWarning)
warnings.simplefilter('ignore', CollinearityWarning)
warnings.filterwarnings("ignore", message="Variables are collinear")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="divide by zero encountered in scalar divide")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in scalar divide")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in divide")
from linearmodels.panel.utility import AbsorbingEffectWarning
warnings.filterwarnings("ignore", category=AbsorbingEffectWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

pd.set_option('display.width', 140)
pd.set_option('display.max_columns', 18)
pd.set_option('display.float_format', '{:.4f}'.format)

# --- Define Tickers for Global Transportation Firms ---
TICKERS_TRANSPORT = [
    'UPS', 'FDX', 'XPO', 'JYD',
    #'DPSGY', # Often fails download
    'AMKBY',
    'CNI', 'CP', 'UNP', 'CSX',
    'ODFL', 'JBHT',
    'ZIM',
]
print(f"Attempting analysis for tickers: {TICKERS_TRANSPORT}")


TICKER_NAMES = { # Optional mapping
    'UPS': 'United Parcel Service', 'FDX': 'FedEx', 'XPO': 'XPO Inc.', 'JYD': 'Jayud Global Logistics',
    #'DPSGY': 'Deutsche Post DHL',
    'AMKBY': 'A.P. Møller - Mærsk',
    'CNI': 'Canadian National Railway', 'CP': 'Canadian Pacific Kansas City', 'UNP': 'Union Pacific', 'CSX': 'CSX Corp.',
    'ODFL': 'Old Dominion Freight Line', 'JBHT': 'J.B. Hunt Transport',
    'ZIM': 'ZIM Integrated Shipping',
}

# --- Define ESG Risk Categories (Time-Invariant) based on user info ---
esg_risk_categories = {
    # Low Risk
    #'DPSGY': 'Low',
    'AMKBY': 'Low', 'CNI': 'Low', 'CP': 'Low',
    # Medium Risk
    'UPS': 'Medium', 'FDX': 'Medium', 'UNP': 'Medium', 'CSX': 'Medium',
    'XPO': 'Medium', 'ODFL': 'Medium', 'JBHT': 'Medium', 'ZIM': 'Medium',
    # High Risk
    'JYD': 'High',
}

# --- Define Date Range ---
START_DATE_PRICES = "2014-01-01" # Start earlier for prices/lags
END_DATE_PRICES = "2024-12-31"
START_DATE_ANALYSIS = "2015-01-01" # Analysis start date
END_DATE_ANALYSIS = "2023-12-31" # Analysis end date

# --- File Paths ---
ESG_DATA_PATH = "/content/historic_esg_scores_global_transportation.csv" # *** USE THE CORRECT FILENAME ***

# --- Parameters ---
ESG_LAG_MONTHS = 1
VIF_THRESHOLD = 10
IMPUTE_DATA = True
RUN_WITHOUT_IMPUTATION_SENSITIVITY = True

# --- Version Control & Script Info ---
SCRIPT_VERSION = "Panel Only v14.1 - Transportation & FF DataReader" # Updated version
print(f"--- Global Transportation ESG Impact Analysis Script Started ({SCRIPT_VERSION}) ---")
# print(f"Tickers: {TICKERS_TRANSPORT}") # Already printed above
print(f"Analysis Period: {START_DATE_ANALYSIS} to {END_DATE_ANALYSIS}")
print(f"ESG Lag: {ESG_LAG_MONTHS} months")
print(f"Imputation Enabled (Main Run - MICE): {IMPUTE_DATA}")
print(f"Run Sensitivity without Imputation: {RUN_WITHOUT_IMPUTATION_SENSITIVITY}")
print(f"Factors Path: Data fetched from K. French Library") # Modified print
print(f"ESG Data Path: {ESG_DATA_PATH} (Source/Quality Not Verified by Script)")


# --- Advanced Imputation Function ---
def advanced_imputation(df_input):
    """
    Performs Iterative Imputation (MICE-like) on numeric columns of a DataFrame.
    Uses BayesianRidge as the estimator by default. Handles all-NaN columns.
    """
    df = df_input.copy()
    original_index = df.index; original_cols = df.columns
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    non_numeric_cols = df.select_dtypes(exclude=np.number).columns.tolist()

    if not numeric_cols: print("  -> Imputation: No numeric columns found."); return df_input
    df_numeric = df[numeric_cols].copy(); df_non_numeric = df[non_numeric_cols].copy()
    if df_numeric.isnull().sum().sum() == 0: print("  -> Imputation: No missing values detected."); return df_input

    print(f"  -> Imputation: Attempting Iterative Imputation (MICE) for {len(numeric_cols)} numeric columns.")
    n_features = len(numeric_cols); n_neighbors = min(5, n_features - 1) if n_features > 1 else 1
    all_nan_cols = df_numeric.columns[df_numeric.isnull().all()].tolist()
    if all_nan_cols:
        print(f"    -> Warning: All-NaN columns cannot be imputed: {all_nan_cols}")
        df_numeric_imputable = df_numeric.drop(columns=all_nan_cols); numeric_cols_imputable = df_numeric_imputable.columns.tolist()
        if not numeric_cols_imputable: print("    -> Error: No imputable numeric columns remain."); return df_input
        n_features = len(numeric_cols_imputable); n_neighbors = min(5, n_features - 1) if n_features > 1 else 1
    else: df_numeric_imputable = df_numeric; numeric_cols_imputable = numeric_cols
    if df_numeric_imputable.empty:
         print("    -> Error: No numeric columns available for imputation.")
         if all_nan_cols: df_all_nan = df_numeric[all_nan_cols]; df_out = pd.concat([df_all_nan, df_non_numeric], axis=1); return df_out[original_cols]
         else: return df_input

    imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=42, tol=1e-3, n_nearest_features=n_neighbors, verbose=0, imputation_order='ascending')
    try:
        imputed_values = imputer.fit_transform(df_numeric_imputable)
        df_imputed_numeric = pd.DataFrame(imputed_values, columns=numeric_cols_imputable, index=df_numeric_imputable.index)
        if all_nan_cols:
             for col in all_nan_cols: df_imputed_numeric[col] = np.nan
        df_out = pd.concat([df_imputed_numeric, df_non_numeric], axis=1); df_out = df_out[original_cols]
        for col in non_numeric_cols:
             if col in df_out.columns:
                 try: df_out[col] = df_out[col].astype(df_input[col].dtype)
                 except Exception as type_err: print(f"    -> Warning: Restore dtype failed '{col}': {type_err}")
        print("  -> Imputation: MICE imputation completed.")
        remaining_nan_count = df_out[numeric_cols].isnull().sum().sum()
        if remaining_nan_count > 0: print(f"  -> !!! WARNING: {remaining_nan_count} NaNs remain post-imputation. !!!")
        return df_out
    except ValueError as ve: print(f"  -> Imputation ERROR (ValueError): {ve}. Check sparse data."); return df_input
    except Exception as e: print(f"  -> Imputation ERROR (General): {e}."); traceback.print_exc(limit=2); return df_input

# ==============================================================================
# --- Step 1: Download Stock Returns ---
# ==============================================================================
print("\n--- 1. Downloading and Preparing Stock Returns ---")
stock_monthly_returns = pd.DataFrame(); tickers_available_yf = []
try:
    tickers_to_download = TICKERS_TRANSPORT # Use the correct list
    all_stock_data = yf.download( tickers_to_download, start=START_DATE_PRICES, end=END_DATE_PRICES, progress=False, auto_adjust=False, actions=False, ignore_tz=True, group_by='ticker')
    if all_stock_data.empty: raise ValueError("No stock price data downloaded.")
    price_data_list = []; available_tickers_in_download = []
    if len(tickers_to_download) == 1: # Handle single ticker case
        ticker = tickers_to_download[0]
        if not all_stock_data.empty:
            df_ticker = all_stock_data[['Adj Close']].copy()
            if df_ticker.empty or df_ticker['Adj Close'].isnull().all(): df_ticker = all_stock_data[['Close']].copy();
            if not (df_ticker.empty or df_ticker['Close'].isnull().all()): print(f"  -> Warning: Using 'Close' for {ticker}.")
            else: print(f"  -> Warning: No valid price for {ticker}. Skipping.")
            if not df_ticker.empty and not df_ticker.isnull().all().all(): df_ticker.columns = [ticker]; price_data_list.append(df_ticker); available_tickers_in_download.append(ticker)
    else: # Handle multiple tickers
        if isinstance(all_stock_data.columns, pd.MultiIndex):
             valid_tickers = all_stock_data.columns.get_level_values(0).unique().tolist()
             for ticker in tickers_to_download:
                  if ticker in valid_tickers:
                    try:
                        df_ticker = all_stock_data[ticker][['Adj Close']].copy()
                        if df_ticker.empty or df_ticker['Adj Close'].isnull().all():
                            df_ticker = all_stock_data[ticker][['Close']].copy()
                            if not (df_ticker.empty or df_ticker['Close'].isnull().all()): print(f"  -> Warning: Using 'Close' for {ticker}.")
                            else: print(f"  -> Warning: No valid price for {ticker}. Skipping."); continue
                        if not df_ticker.empty and not df_ticker.isnull().all().all(): df_ticker.columns = [ticker]; price_data_list.append(df_ticker); available_tickers_in_download.append(ticker)
                    except KeyError: print(f"  -> Warning: Data for {ticker} not in MultiIndex.")
                  else: print(f"  -> Warning: Ticker {ticker} requested but not in yfinance result for this run.") # Modified message
        else: raise TypeError(f"Unexpected yfinance structure: {type(all_stock_data)}") # Should be MultiIndex for multiple tickers

    if not price_data_list: raise ValueError("No valid price data collected.")
    price_data = pd.concat(price_data_list, axis=1); price_data = price_data.ffill().bfill().dropna(axis=1, how='all')
    if price_data.empty: raise ValueError("Price data empty after cleaning.")
    tickers_available_yf = sorted(list(price_data.columns)); print(f"  -> Stock price data processed for {len(tickers_available_yf)} tickers: {tickers_available_yf}")
    price_data.index = pd.to_datetime(price_data.index); monthly_prices = price_data.resample('ME').last()
    stock_monthly_returns = monthly_prices.pct_change()
    buffer_start_date = (pd.to_datetime(START_DATE_ANALYSIS) - pd.DateOffset(months=ESG_LAG_MONTHS + 2))
    stock_monthly_returns = stock_monthly_returns.loc[buffer_start_date:END_DATE_PRICES] # Filter includes buffer
    if stock_monthly_returns.empty or stock_monthly_returns.isnull().all().all(): raise ValueError("Monthly returns empty/all NaN after date filtering.")
    print(f"  -> Stock monthly returns prepared: {stock_monthly_returns.index.min().date()} to {stock_monthly_returns.index.max().date()}")
except Exception as e: print(f" FATAL ERROR processing stock returns: {e}"); traceback.print_exc(); sys.exit()

# ==============================================================================
# --- Step 2: Load and Prepare Fama-French Factors ---
# ==============================================================================
print("\n--- 2. Loading and Preparing Fama-French Factors ---")
ff_factors_monthly = pd.DataFrame(); rf_col = None; available_factors_list = []

try:
    print(f"  -> Downloading Fama-French factors using pandas-datareader...")
    ff_start = START_DATE_PRICES
    ff_end = END_DATE_PRICES

    try:
        # Fetch FF5 factors (monthly is index 0)
        ff_data_5f = web.DataReader('Developed_5_Factors', 'famafrench', start=ff_start, end=ff_end)
        ff_monthly_5f = ff_data_5f[0].copy()

        # Fetch Momentum factor (monthly is index 0)
        ff_mom_data = web.DataReader('Developed_Mom_Factor', 'famafrench', start=ff_start, end=ff_end)
        ff_monthly_mom = ff_mom_data[0].copy()

        # Merge
        ff_factors_raw = pd.merge(ff_monthly_5f, ff_monthly_mom, left_index=True, right_index=True, how='inner')

        # Rename columns
        ff_factors_raw = ff_factors_raw.rename(columns={
            'Mkt-RF': 'mkt_rf', 'SMB': 'smb', 'HML': 'hml',
            'RMW': 'rmw', 'CMA': 'cma', 'RF': 'rf', 'Mom': 'mom'
        })

        # Data from pandas-datareader is already in decimals
        print("  -> Factors downloaded and merged successfully.")
        print(f"  -> Raw factors shape: {ff_factors_raw.shape}")

    except Exception as ff_err:
        print(f"    -> ERROR downloading/processing Fama-French factors via pandas-datareader: {ff_err}")
        print("    -> Check dataset names ('Developed_5_Factors', 'Developed_Mom_Factor') and internet connection.")
        traceback.print_exc(limit=1)
        raise ValueError("Failed to obtain Fama-French factors.") from ff_err

    # Convert PeriodIndex to DatetimeIndex and set to month end
    ff_factors_raw.index = ff_factors_raw.index.to_timestamp() + pd.offsets.MonthEnd(0)

    # Filter by date range needed for analysis + buffer
    buffer_start_date = (pd.to_datetime(START_DATE_ANALYSIS) - pd.DateOffset(months=ESG_LAG_MONTHS + 2))
    ff_factors_monthly_filtered = ff_factors_raw.loc[buffer_start_date:END_DATE_PRICES].copy()

    if ff_factors_monthly_filtered.empty:
        raise ValueError(f"No factor data found within the required date range ({buffer_start_date.date()} to {END_DATE_PRICES}).")

    # --- Imputation for Factors ---
    if IMPUTE_DATA and ff_factors_monthly_filtered.isnull().any().any():
        print("  -> Imputing missing values in factors data (if any)...")
        factor_numeric_cols = ff_factors_monthly_filtered.select_dtypes(include=np.number).columns
        if not factor_numeric_cols.empty:
             ff_factors_monthly = advanced_imputation(ff_factors_monthly_filtered)
             if ff_factors_monthly is ff_factors_monthly_filtered: print("    -> Warning: Factor imputation skipped or failed."); ff_factors_monthly = ff_factors_monthly_filtered.copy()
             elif ff_factors_monthly[factor_numeric_cols].isnull().any().any(): print("    -> Warning: NaNs may remain post-imputation.")
        else: print("    -> No numeric factors found for imputation."); ff_factors_monthly = ff_factors_monthly_filtered.copy()
    else:
        ff_factors_monthly = ff_factors_monthly_filtered.copy()
        if not IMPUTE_DATA: print("  -> Imputation disabled for factors.")
        elif not ff_factors_monthly_filtered.isnull().any().any(): print("  -> No missing values detected in fetched factors.")

    # --- Identify Risk-Free Rate and Available Factors ---
    rf_col_options = ['rf', 'risk_free_rate']
    rf_col = next((col for col in rf_col_options if col in ff_factors_monthly.columns), None)
    if rf_col: print(f"  -> Using '{rf_col}' as RF.");
    if rf_col is None: raise ValueError(f"Critical Error: RF column ({rf_col_options}) not found in downloaded factors.")
    elif ff_factors_monthly[rf_col].isnull().any(): print(f"  -> !!! WARNING: RF column ('{rf_col}') contains NaNs after processing!!!")

    factor_cols_check_for_factors = ["mkt_rf", "smb", "hml", "rmw", "cma", "mom"]
    available_factors_list = sorted([f for f in factor_cols_check_for_factors if f in ff_factors_monthly.columns and pd.api.types.is_numeric_dtype(ff_factors_monthly[f])])
    if not available_factors_list: print("  -> !!! WARNING: No standard factors found. !!!")
    else: print(f"  -> Available factors identified: {available_factors_list}")

except ValueError as ve: print(f" FATAL ERROR processing factors: {ve}"); sys.exit()
except Exception as e: print(f" FATAL ERROR processing factors: {e}"); traceback.print_exc(); sys.exit()


# ==============================================================================
# --- Step 3: Load and Prepare ESG Data from CSV ---
# ==============================================================================
print("\n--- 3. Loading and Preparing ESG Data from CSV ---")
esg_panel_raw = pd.DataFrame()
try:
    try: esg_data_loaded = pd.read_csv(ESG_DATA_PATH)
    except FileNotFoundError: raise ValueError(f"ESG file not found: '{ESG_DATA_PATH}'.")
    except Exception as e: raise ValueError(f"Could not read ESG file '{ESG_DATA_PATH}': {e}")
    print(f"  -> Raw ESG data loaded. Shape: {esg_data_loaded.shape}")
    original_cols = list(esg_data_loaded.columns)
    # *** ADJUST standardization based on actual CSV columns ***
    standardized_names = {
        'total-score': 'total_score',
        'e-score': 'e_score',
        's-score': 's_score',
        'g-score': 'g_score'
    }
    esg_data_loaded.columns = [standardized_names.get(re.sub(r'\s+', '_', col).replace('.', '').lower(),
                                                    re.sub(r'\s+', '_', col).replace('.', '').lower())
                               for col in esg_data_loaded.columns]
    standardized_cols = list(esg_data_loaded.columns); print(f"  -> Standardized ESG columns: {standardized_cols}")
    required_esg_cols = ['date', 'ticker', 'total_score', 'e_score', 's_score', 'g_score'] # Use standardized names
    missing_cols = [col for col in required_esg_cols if col not in esg_data_loaded.columns];
    if missing_cols: raise ValueError(f"ESG CSV missing required columns: {missing_cols}.")
    esg_data_loaded['date'] = pd.to_datetime(esg_data_loaded['date'], errors='coerce')
    initial_rows = len(esg_data_loaded); esg_data_loaded = esg_data_loaded.dropna(subset=['date'])
    if len(esg_data_loaded) < initial_rows: print(f"  -> Warning: Dropped {initial_rows - len(esg_data_loaded)} rows due to invalid ESG dates.")
    if esg_data_loaded.empty: raise ValueError("ESG data empty after removing invalid dates.")
    score_cols_std = ['total_score', 'e_score', 's_score', 'g_score']; # Use standardized names
    print(f"  -> Converting score columns to numeric: {score_cols_std}")
    non_numeric_issues = False
    for col in score_cols_std:
        initial_nan_count = esg_data_loaded[col].isnull().sum(); esg_data_loaded[col] = pd.to_numeric(esg_data_loaded[col], errors='coerce'); final_nan_count = esg_data_loaded[col].isnull().sum()
        if final_nan_count > initial_nan_count: num_coerced = final_nan_count - initial_nan_count; print(f"    -> CRITICAL WARNING: Column '{col}' had {num_coerced} non-numeric values converted to NaN."); non_numeric_issues = True
    if non_numeric_issues and not IMPUTE_DATA: raise ValueError(f"Non-numeric ESG scores found and imputation disabled. Clean source CSV.")
    elif non_numeric_issues: print("    -> Imputation will attempt to handle NaNs from non-numeric scores.")
    esg_data_loaded['ticker'] = esg_data_loaded['ticker'].astype(str).str.upper().str.strip()
    stock_tickers_upper = [t.upper().strip() for t in tickers_available_yf]; esg_tickers = esg_data_loaded['ticker'].unique()
    common_tickers = sorted(list(set(stock_tickers_upper) & set(esg_tickers)));
    if not common_tickers: raise ValueError("No common tickers found between stock and ESG data.")
    print(f"  -> Common tickers identified: {common_tickers} ({len(common_tickers)} firms)")
    esg_only = sorted(list(set(esg_tickers) - set(stock_tickers_upper))); stock_only = sorted(list(set(stock_tickers_upper) - set(esg_tickers)))
    if esg_only: print(f"    -> Tickers in ESG only: {esg_only}")
    if stock_only: print(f"    -> Tickers in Stock only: {stock_only}")
    esg_data_filtered = esg_data_loaded[esg_data_loaded['ticker'].isin(common_tickers)].copy()
    buffer_start_date = (pd.to_datetime(START_DATE_ANALYSIS) - pd.DateOffset(months=ESG_LAG_MONTHS + 2))
    esg_filter_end_date = pd.to_datetime(END_DATE_ANALYSIS) + pd.offsets.MonthEnd(0)
    esg_data_filtered = esg_data_filtered[(esg_data_filtered['date'] >= buffer_start_date) & (esg_data_filtered['date'] <= esg_filter_end_date)]
    if esg_data_filtered.empty: raise ValueError("No ESG data remains after filtering.")
    esg_data_filtered['date'] = esg_data_filtered['date'] + pd.offsets.MonthEnd(0); esg_data_filtered = esg_data_filtered.sort_values(by=['ticker', 'date']).drop_duplicates(subset=['ticker', 'date'], keep='last')
    panel_start_date = esg_data_filtered['date'].min(); panel_end_date = esg_data_filtered['date'].max(); print(f"  -> Creating ESG panel from {panel_start_date.date()} to {panel_end_date.date()}")
    full_date_range = pd.date_range(start=panel_start_date, end=panel_end_date, freq='ME'); multi_index = pd.MultiIndex.from_product([common_tickers, full_date_range], names=['Ticker', 'Date'])
    esg_panel_raw = esg_data_filtered.set_index(['ticker', 'date'])[score_cols_std].reindex(multi_index); print(f"  -> Forward-filling ESG scores...")
    esg_panel_raw[score_cols_std] = esg_panel_raw.groupby(level='Ticker')[score_cols_std].ffill()
    if esg_panel_raw[score_cols_std].isnull().values.any(): nan_counts = esg_panel_raw[score_cols_std].isnull().sum(); print(f"  -> !!! WARNING: NaNs remain after ffill. Imputation will attempt. Counts:\n{nan_counts[nan_counts > 0]}")
    else: print("  -> No NaNs detected post-ffill.")
    esg_panel_raw = esg_panel_raw.reset_index(); print(f"  -> ESG panel structure created. Shape: {esg_panel_raw.shape}")
except Exception as e: print(f" FATAL ERROR processing ESG data: {e}"); traceback.print_exc(); sys.exit()




# ==============================================================================
# --- Step 5: Check for Multicollinearity (VIF) ---
# ==============================================================================
print("\n--- 5. Checking for Multicollinearity (VIF) ---")
vif_results_total = None; vif_results_components = None
def calculate_vif(data, predictors, model_name="VIF Check"):
    print(f"\n  Calculating VIF for: {model_name}")
    if not predictors: print("    -> No predictors."); return None, []
    predictors_in_data = [p for p in predictors if p in data.columns]; missing = [p for p in predictors if p not in data.columns]
    if missing: print(f"    -> Warning: Predictors missing for VIF: {missing}")
    if not predictors_in_data: print("    -> No valid predictors."); return None, []
    X = data[predictors_in_data].copy(); initial_rows = len(X); X = X.dropna(); dropped_rows = initial_rows - len(X)
    if dropped_rows > 0: print(f"    -> Dropped {dropped_rows} rows with NaNs for VIF.")
    if X.empty or len(X) < 2 or X.shape[1] < 1: print(f"    -> Not enough data for VIF."); return None, predictors_in_data
    constant_cols = X.columns[X.nunique() <= 1].tolist()
    if constant_cols: print(f"    -> Warning: Constant columns removed for VIF: {constant_cols}"); X = X.drop(columns=constant_cols); predictors_in_data = X.columns.tolist();
    if X.empty or X.shape[1] < 1: print(f"    -> No variables left after removing constant cols."); return None, predictors_in_data
    try: X_vif = sm.add_constant(X, prepend=True, has_constant='skip')
    except Exception as e: print(f"    -> Error adding constant: {e}."); return None, predictors_in_data
    if not np.all(np.isfinite(X_vif.values)): print("    -> Warning: Non-finite values detected. Attempting removal..."); X_vif = X_vif.replace([np.inf, -np.inf], np.nan).dropna();
    if X_vif.empty: print("    -> Error: Data empty after removing non-finite."); return None, predictors_in_data
    try:
        vif_data = pd.DataFrame(); vif_data["Variable"] = [col for col in X_vif.columns if col.lower() != 'const']
        vif_values = [variance_inflation_factor(X_vif.values.astype(float), i) for i, col in enumerate(X_vif.columns) if col.lower() != 'const']
        vif_data["VIF"] = vif_values; print(vif_data.sort_values('VIF', ascending=False).to_string(index=False))
        high_vif_vars = vif_data[vif_data["VIF"] > VIF_THRESHOLD]["Variable"].tolist()
        if high_vif_vars: print(f"    -> !!! WARNING: High VIF (> {VIF_THRESHOLD}): {high_vif_vars}. !!!")
        else: print(f"    -> VIF check passed (threshold={VIF_THRESHOLD}).")
        return vif_data, predictors_in_data
    except (np.linalg.LinAlgError, ValueError) as vif_calc_err: print(f"    -> VIF calc failed: {vif_calc_err} (Perfect multicollinearity likely)."); return None, predictors_in_data
    except Exception as e: print(f"    -> VIF error: {e}"); traceback.print_exc(); return None, predictors_in_data

# *** Use standardized variable name from Step 4 ***
primary_esg_var = f'total_score_lag{ESG_LAG_MONTHS}' if f'total_score_lag{ESG_LAG_MONTHS}' in final_panel_data.columns else None
component_esg_vars = [col for col in [f'e_score_lag{ESG_LAG_MONTHS}', f's_score_lag{ESG_LAG_MONTHS}', f'g_score_lag{ESG_LAG_MONTHS}'] if col in final_panel_data.columns]
if primary_esg_var:
    predictors_total_vif = available_factors + [primary_esg_var]; predictors_total_vif = [p for p in predictors_total_vif if p in final_panel_data.columns]
    if predictors_total_vif: vif_results_total, _ = calculate_vif(final_panel_data.reset_index(), predictors_total_vif, "Factors + Total ESG")
    else: print(" -> Skipping VIF (Total ESG): No valid predictors.")
else: print("  -> Skipping VIF (Total ESG): Primary ESG var missing.")
if component_esg_vars:
    predictors_components_vif = available_factors + component_esg_vars; predictors_components_vif = [p for p in predictors_components_vif if p in final_panel_data.columns]
    if predictors_components_vif: vif_results_components, _ = calculate_vif(final_panel_data.reset_index(), predictors_components_vif, "Factors + ESG Components")
    else: print(" -> Skipping VIF (Components): No valid predictors.")
else: print("  -> Skipping VIF (Components): ESG component vars missing.")
if primary_esg_var: print(f"\n  -> Primary ESG var for models: '{primary_esg_var}'")
if component_esg_vars: print(f"  -> Component ESG vars: {component_esg_vars}")
if not primary_esg_var and not component_esg_vars: print("\n!!! WARNING: No lagged ESG vars available. !!!")

# ==============================================================================
# --- Step 6: Panel Regression Analysis ---
# ==============================================================================
print("\n--- 6. Panel Regression Analysis (Main Run - Imputed Data if Enabled) ---")
regression_results = {}; model_summaries = {}; sensitivity_regression_results = {}; model_formulas_used = {}
sensitivity_summaries = {}; sensitivity_formulas_used = {} # Separate dicts for sensitivity

formula_pooled_interaction = None; formula_fe_re_simple = None
if primary_esg_var and available_factors:
    base_factors_str = ' + '.join(available_factors); esg_term = primary_esg_var
    # *** Use standardized variable name ***
    formula_fe_re_simple = f"ExcessReturn ~ 1 + {base_factors_str} + {esg_term}"; print(f"\n  -> Formula for FE & RE models: {formula_fe_re_simple}")
    if lagged_category_col and lagged_category_col in final_panel_data.columns and final_panel_data[lagged_category_col].nunique() > 1:
        preferred_reference_category = 'Medium'; print(f"  -> Attempting to set '{preferred_reference_category}' as reference category for Pooled OLS.")
        available_cats = [c for c in final_panel_data[lagged_category_col].unique() if isinstance(c, str)]; final_reference_category = None
        if preferred_reference_category in available_cats: final_reference_category = preferred_reference_category; print(f"     -> Found exact match: Using '{final_reference_category}'.")
        else:
            ref_cat_lower = preferred_reference_category.lower(); matching_cats = [c for c in available_cats if c.lower() == ref_cat_lower]
            if matching_cats: final_reference_category = matching_cats[0]; print(f"     -> Found case-insensitive match: Using '{final_reference_category}'.")
            elif available_cats:
                 most_frequent_cat = final_panel_data[lagged_category_col].mode()
                 if not most_frequent_cat.empty: final_reference_category = most_frequent_cat[0]; print(f"     -> '{preferred_reference_category}' not found. Using most frequent category '{final_reference_category}' as fallback reference.")
                 else: print(f"     -> Warning: Could not determine most frequent category. Cannot create Pooled OLS interaction formula.")
            else: print(f"     -> Warning: No valid string categories found. Cannot create Pooled OLS interaction formula.")
        if final_reference_category: interaction_term = f"{esg_term} * C({lagged_category_col}, Treatment(reference='{final_reference_category}'))"; formula_pooled_interaction = f"ExcessReturn ~ 1 + {base_factors_str} + {interaction_term}"; print(f"  -> Formula for Pooled OLS (Interaction): {formula_pooled_interaction}")
        else: print(f"  -> Warn: Could not determine reference category. Using simple formula for Pooled OLS."); formula_pooled_interaction = formula_fe_re_simple
    else:
        if not lagged_category_col or lagged_category_col not in final_panel_data.columns: reason = "missing"
        else: reason = "has <= 1 unique value"
        print(f"  -> Warn: Category column ('{lagged_category_col}') {reason}. Using simple formula for Pooled OLS."); formula_pooled_interaction = formula_fe_re_simple
else: missing_info = [];
if not primary_esg_var: missing_info.append("primary ESG variable")
if not available_factors: missing_info.append("factor variables"); print(f"\n!!! CRITICAL WARNING: Cannot construct formulas (missing {', '.join(missing_info)}). Regression cannot proceed. !!!")

def run_panel_model(formula, model_type, model_key, data,
                    cov_config={'cov_type':'clustered', 'cluster_entity':True, 'cluster_time': False},
                    results_dict=None, summary_dict=None, formula_dict=None):
    """Fits panel model, handles errors, stores results/summaries."""
    print(f"\n  --- Fitting {model_key} ({model_type}) ---")
    if results_dict is None: results_dict = {}
    if summary_dict is None: summary_dict = {}
    if formula_dict is None: formula_dict = {}
    results = None; error_msg = None; formula_status = "Attempted"
    if not formula: error_msg = "Skipped: No formula."; formula_status = "Skipped - No Formula"
    elif data is None or data.empty: error_msg = "Skipped: Empty data."; formula_status = "Skipped - Empty Data"
    if error_msg: print(f"    -> {error_msg}"); summary_dict[model_key] = error_msg; results_dict[model_key] = None; formula_dict[model_key] = formula_status; return
    try:
        print(f"    Using Formula: {formula}")
        dep, exog_formula = formula.split('~', 1); dep = dep.strip(); exog_formula = exog_formula.strip()
        final_cov_config = cov_config.copy(); model = None; summary_obj = None
        if model_type == 'Pooled':
            pooled_cov_config = {'cov_type': 'robust'}; print("     (Note: Using robust covariance for Pooled OLS)")
            model = PanelOLS.from_formula(formula, data=data); final_cov_config = pooled_cov_config; formula_status = "Pooled Spec (Robust SE)"
        elif model_type == 'RE':
            model = RandomEffects.from_formula(formula, data=data)
            if 'cluster' not in cov_config.get('cov_type','robust').lower(): final_cov_config = {'cov_type': 'robust'}
            else: print("     (Note: Applying requested clustering to RE model)")
            formula_status = "RE Spec"
        elif model_type == 'FE_Entity':
            model = PanelOLS.from_formula(f"{dep} ~ {exog_formula} + EntityEffects", data=data, drop_absorbed=True); final_cov_config['cluster_time'] = False; formula_status = "FE Entity Spec"
        elif model_type == 'FE_TwoWay':
            model = PanelOLS.from_formula(f"{dep} ~ {exog_formula} + EntityEffects + TimeEffects", data=data, drop_absorbed=True); final_cov_config['cluster_time'] = True; formula_status = "FE TwoWay Spec"
        else: raise ValueError(f"Invalid model_type: {model_type}")
        results = model.fit(**final_cov_config); print(f"    -> Fit OK.")
        try:
             summary_obj = results.summary; print(f"    -> Summary generation OK.")
             summary_dict[model_key] = summary_obj
             if model_type.startswith('FE'):
                 summary_str = str(summary_obj); # Define summary_str here
                 if 'Absorbed' in summary_str or 'dropped' in summary_str.lower(): print("    -> Warning: Summary indicates potential variable absorption/dropping.")
        except (np.linalg.LinAlgError, ValueError) as summary_err: error_msg = f"Error: Summary failed - {type(summary_err).__name__}: {summary_err} (Singular matrix likely)."; print(f"    -> {error_msg}"); summary_dict[model_key] = error_msg; results_dict[model_key] = results; formula_dict[model_key] = formula_status + " (Summary Failed)"; return
        except Exception as gen_summary_err: error_msg = f"Error: Unexpected summary error - {type(gen_summary_err).__name__}: {gen_summary_err}"; print(f"    -> {error_msg}"); summary_dict[model_key] = error_msg; results_dict[model_key] = results; formula_dict[model_key] = formula_status + " (Summary Failed - Unknown)"; return
    except (ValueError, np.linalg.LinAlgError, PerfectSeparationError, ZeroDivisionError) as fit_err: error_msg = f"Error: Fit failed - {type(fit_err).__name__}: {fit_err}"; print(f"    -> {error_msg}")
    except Exception as e: error_msg = f"Error: Unexpected fit error - {type(e).__name__}: {e}"; print(f"    -> {error_msg}"); traceback.print_exc(limit=1)
    if results is not None and error_msg is None: results_dict[model_key] = results; formula_dict[model_key] = formula_status + " (Success)"
    elif results is not None and error_msg is not None: pass
    else: results_dict[model_key] = None; summary_dict[model_key] = error_msg; formula_dict[model_key] = formula_status + " (Fit Failed)"

# --- Run Main Models ---
if formula_pooled_interaction and formula_fe_re_simple:
    run_panel_model(formula_pooled_interaction, 'Pooled', 'Pooled_Interaction', final_panel_data, results_dict=regression_results, summary_dict=model_summaries, formula_dict=model_formulas_used)
    run_panel_model(formula_fe_re_simple, 'RE', 'RE_Simple', final_panel_data, results_dict=regression_results, summary_dict=model_summaries, formula_dict=model_formulas_used)
    run_panel_model(formula_fe_re_simple, 'FE_Entity', 'FE_Entity_Simple', final_panel_data, results_dict=regression_results, summary_dict=model_summaries, formula_dict=model_formulas_used)
    run_panel_model(formula_fe_re_simple, 'FE_TwoWay', 'FE_TwoWay_Simple', final_panel_data, results_dict=regression_results, summary_dict=model_summaries, formula_dict=model_formulas_used)
else: print("\n!!! Skipping main panel estimations: Missing required formulas. !!!")

# --- Run Sensitivity Analysis (No Imputation) ---
if RUN_WITHOUT_IMPUTATION_SENSITIVITY:
    print("\n--- 6b. Panel Regression Analysis (Sensitivity Run - NO IMPUTATION) ---")
    if panel_data_no_imputation is None or panel_data_no_imputation.empty:
        print("    -> Skipping Sensitivity: Non-imputed dataset empty.")
        sensitivity_regression_results['Pooled_Interaction_Sens'] = None; sensitivity_summaries['Pooled_Interaction_Sens'] = "Skipped - Empty Data"; sensitivity_formulas_used['Pooled_Interaction_Sens'] = "Skipped - Empty Data"
        sensitivity_regression_results['FE_Entity_Simple_Sens'] = None; sensitivity_summaries['FE_Entity_Simple_Sens'] = "Skipped - Empty Data"; sensitivity_formulas_used['FE_Entity_Simple_Sens'] = "Skipped - Empty Data"
    elif formula_pooled_interaction and formula_fe_re_simple:
        panel_data_no_imputation_indexed = None
        if isinstance(panel_data_no_imputation.index, pd.MultiIndex): panel_data_no_imputation_indexed = panel_data_no_imputation.copy()
        elif {'Ticker', 'Date'}.issubset(panel_data_no_imputation.columns):
             panel_data_no_imputation_idx_temp = panel_data_no_imputation.copy(); panel_data_no_imputation_idx_temp['Date'] = pd.to_datetime(panel_data_no_imputation_idx_temp['Date'])
             panel_data_no_imputation_indexed = panel_data_no_imputation_idx_temp.set_index(['Ticker', 'Date']).sort_index()
        if panel_data_no_imputation_indexed is not None:
            run_panel_model(formula_pooled_interaction, 'Pooled', 'Pooled_Interaction_Sens', panel_data_no_imputation_indexed, results_dict=sensitivity_regression_results, summary_dict=sensitivity_summaries, formula_dict=sensitivity_formulas_used)
            run_panel_model(formula_fe_re_simple, 'FE_Entity', 'FE_Entity_Simple_Sens', panel_data_no_imputation_indexed, results_dict=sensitivity_regression_results, summary_dict=sensitivity_summaries, formula_dict=sensitivity_formulas_used)
        else: print("    -> Error: Cannot set index for sensitivity data. Skipping."); sensitivity_summaries['Pooled_Interaction_Sens'] = "Skipped - Index Error"; sensitivity_summaries['FE_Entity_Simple_Sens'] = "Skipped - Index Error"; sensitivity_formulas_used['Pooled_Interaction_Sens'] = "Skipped - Index Error"; sensitivity_formulas_used['FE_Entity_Simple_Sens'] = "Skipped - Index Error"
    else: print("    -> Skipping Sensitivity: Missing required formulas."); sensitivity_summaries['Pooled_Interaction_Sens'] = "Skipped - No Formula"; sensitivity_summaries['FE_Entity_Simple_Sens'] = "Skipped - No Formula"; sensitivity_formulas_used['Pooled_Interaction_Sens'] = "Skipped - No Formula"; sensitivity_formulas_used['FE_Entity_Simple_Sens'] = "Skipped - No Formula"
else: print("\n--- Sensitivity Analysis Skipped (Configured Off). ---")

# ==============================================================================
# --- Step 7: Specification Tests & Interpretation ---
# ==============================================================================
print("\n--- 7. Specification Tests & Interpretation (using Main Run results) ---")
spec_test_results_list = []; preferred_model_key = None
fe_model = regression_results.get('FE_Entity_Simple'); re_model = regression_results.get('RE_Simple')
pooled_model = regression_results.get('Pooled_Interaction'); fe_tw_model = regression_results.get('FE_TwoWay_Simple')
is_fe_valid = isinstance(fe_model, PanelEffectsResults); is_re_valid = isinstance(re_model, RandomEffectsResults)
is_pooled_valid = isinstance(pooled_model, PanelEffectsResults); is_fe_tw_valid = isinstance(fe_tw_model, PanelEffectsResults)

try:
    print("\n    Comparing FE (Simple) vs RE (Simple) - Hausman Test:")
    if is_fe_valid and is_re_valid:
        try:
            common_params = list(set(fe_model.params.index) & set(re_model.params.index))
            if not common_params: print("      -> Skipping Hausman: No common parameters."); spec_test_results_list.append({'Test': 'Hausman (FE vs RE - Simple)', 'Details': 'No common parameters', 'P-value': '-', 'Conclusion': 'Cannot Run'})
            else:
                print("      -> Performing Hausman test via model comparison..."); comparison_fe_re = model_compare({"FE_Simple": fe_model, "RE_Simple": re_model})
                print(comparison_fe_re); hausman_pval_str = "See Table"; pval_num = np.nan
                try:
                    summary_str = str(comparison_fe_re); match = re.search(r"Hausman\s+([\d\.]+)", summary_str);
                    if match: pval_num = float(match.group(1)); hausman_pval_str = f"{pval_num:.4f}"
                except Exception as parse_err: print(f"       -> Warning: Could not parse Hausman p-value: {parse_err}")
                conclusion_hausman_test = 'Check Table';
                if not pd.isna(pval_num): conclusion_hausman_test = 'Prefer FE if Hausman p < 0.05'
                spec_test_results_list.append({'Test': 'Hausman (FE vs RE - Simple)', 'Details': 'Comparison table printed', 'P-value': hausman_pval_str, 'Conclusion': conclusion_hausman_test})
        except Exception as comp_e: print(f"      -> Error running Hausman comparison: {comp_e}"); spec_test_results_list.append({'Test': 'Hausman (FE vs RE - Simple)', 'Details': f"Error: {comp_e}", 'P-value': '-', 'Conclusion': 'Error'})
    else: details = "RE invalid" if is_fe_valid else "FE invalid" if is_re_valid else "Both invalid"; print(f"\n    Skipping Hausman test: {details}."); spec_test_results_list.append({'Test': 'Hausman (FE vs RE - Simple)', 'Details': details, 'P-value': '-', 'Conclusion': 'Cannot Run'})

    print("\n    F-test for Poolability (Entity Effects):")
    if is_fe_valid:
        try:
            if hasattr(fe_model, 'f_pooled'):
                f_pool = fe_model.f_pooled; stat_val = f_pool.stat; pval_val = f_pool.pval; df_num = getattr(f_pool, 'df_num', '?'); df_denom = getattr(f_pool, 'df_denom', '?')
                print(f"      F={stat_val:.4f}, P-value={pval_val:.4f} (df_num={df_num}, df_denom={df_denom})"); conclusion = 'Reject Pooling (Use FE)' if pval_val < 0.05 else 'Cannot Reject Pooling (Pooled OK)'
                spec_test_results_list.append({'Test': 'F-test (Poolability - Entity)', 'Details': f'F({df_num},{df_denom})={stat_val:.4f}', 'P-value': f'{pval_val:.4f}', 'Conclusion': conclusion})
            else: print("      -> Poolability F-stat (f_pooled) not available."); spec_test_results_list.append({'Test': 'F-test (Poolability - Entity)', 'Details': 'f_pooled unavailable', 'P-value': '-', 'Conclusion': 'Cannot Run'})
        except Exception as ftest_e: print(f"      -> Error accessing Poolability F-test: {ftest_e}"); spec_test_results_list.append({'Test': 'F-test (Poolability - Entity)', 'Details': f"Error: {ftest_e}", 'P-value': '-', 'Conclusion': 'Error'})
    else: print("      -> Skipping Poolability F-test: FE (Simple) model invalid."); spec_test_results_list.append({'Test': 'F-test (Poolability - Entity)', 'Details': 'FE Invalid', 'P-value': '-', 'Conclusion': 'Cannot Run'})

    print("\n    F-test for Time Effects:");
    if is_fe_tw_valid:
         try:
            if hasattr(fe_tw_model, 'f_test_time'):
                f_time = fe_tw_model.f_test_time; stat_val = f_time.stat; pval_val = f_time.pval; df_num = getattr(f_time, 'df_num', '?'); df_denom = getattr(f_time, 'df_denom', '?')
                print(f"      F={stat_val:.4f}, P-value={pval_val:.4f} (df_num={df_num}, df_denom={df_denom})"); conclusion = 'Time Effects Significant (Use Two-Way FE)' if pval_val < 0.05 else 'Time Effects Not Significant (Entity FE OK)'
                spec_test_results_list.append({'Test': 'F-test (Time Effects)', 'Details': f'F({df_num},{df_denom})={stat_val:.4f}', 'P-value': f'{pval_val:.4f}', 'Conclusion': conclusion})
            else: print("      -> Time Effects F-stat (f_test_time) not available."); spec_test_results_list.append({'Test': 'F-test (Time Effects)', 'Details': 'f_test_time unavailable', 'P-value': '-', 'Conclusion': 'Cannot Run'})
         except Exception as ftest_e: print(f"      -> Error accessing Time Effects F-test: {ftest_e}"); spec_test_results_list.append({'Test': 'F-test (Time Effects)', 'Details': f"Error: {ftest_e}", 'P-value': '-', 'Conclusion': 'Error'})
    else: print("      -> Skipping Time Effects F-test: Two-Way FE (Simple) model invalid."); spec_test_results_list.append({'Test': 'F-test (Time Effects)', 'Details': 'Two-Way FE Invalid', 'P-value': '-', 'Conclusion': 'Cannot Run'})
    spec_test_df = pd.DataFrame(spec_test_results_list)
except Exception as e: print(f"\nError during spec tests: {e}"); traceback.print_exc(); spec_test_df = pd.DataFrame()

print("\n--- Preferred Model Selection Logic (Main Run) ---")
conclusion_pool = 'Cannot Run'; conclusion_time = 'Cannot Run'; conclusion_hausman = 'Cannot Run'
if not spec_test_df.empty:
    pool_row = spec_test_df[spec_test_df['Test'] == 'F-test (Poolability - Entity)']; conclusion_pool = pool_row['Conclusion'].iloc[0] if not pool_row.empty else 'Cannot Run'
    time_row = spec_test_df[spec_test_df['Test'] == 'F-test (Time Effects)']; conclusion_time = time_row['Conclusion'].iloc[0] if not time_row.empty else 'Cannot Run'
    hausman_row = spec_test_df[spec_test_df['Test'] == 'Hausman (FE vs RE - Simple)']; conclusion_hausman = hausman_row['Conclusion'].iloc[0] if not hausman_row.empty else 'Cannot Run'
print(f"  - Poolability Test Result: '{conclusion_pool}'"); print(f"  - Hausman Test (Simple Models) Result: '{conclusion_hausman}'"); print(f"  - Time Effects Test Result: '{conclusion_time}'")

preferred_model_key = None # Reset
if 'Time Effects Significant' in conclusion_time and is_fe_tw_valid:
    print("  - Logic: Time effects significant & Two-Way FE valid."); preferred_model_key = 'FE_TwoWay_Simple'; print("  -> Tentative Preference: FE Two-Way (Simple).")
elif 'Reject Pooling' in conclusion_pool and is_fe_valid:
    print("  - Logic: Pooling rejected & Entity FE valid.")
    if 'Prefer FE' in conclusion_hausman: print("  - Logic: Hausman also prefers FE."); preferred_model_key = 'FE_Entity_Simple'; print("  -> Tentative Preference: FE Entity (Simple).")
    else: print(f"  - Logic: Hausman ({conclusion_hausman}), but Poolability rejects Pooled. Prioritizing FE Entity."); preferred_model_key = 'FE_Entity_Simple'; print("  -> Tentative Preference: FE Entity (Simple).")
elif is_re_valid and 'Prefer FE' not in conclusion_hausman:
     print("  - Logic: FE not selected/valid, RE valid, Hausman doesn't strongly prefer FE."); preferred_model_key = 'RE_Simple'; print("  -> Tentative Preference: RE (Simple).")
elif 'Cannot Reject Pooling' in conclusion_pool and is_pooled_valid:
     print("  - Logic: FE/RE models not selected/valid, Pooling allowed, Pooled OLS valid."); preferred_model_key = 'Pooled_Interaction'; print("  -> Tentative Preference: Pooled OLS (Interaction).")
if preferred_model_key is None:
    print("\n  - No model selected by primary logic. Applying fallback...")
    fallback_order = ['FE_TwoWay_Simple', 'FE_Entity_Simple', 'RE_Simple', 'Pooled_Interaction']
    for model_key_fb in fallback_order:
        if model_key_fb in regression_results and isinstance(regression_results.get(model_key_fb), (PanelEffectsResults, RandomEffectsResults)):
            summary_val = model_summaries.get(model_key_fb)
            if not isinstance(summary_val, str): preferred_model_key = model_key_fb; print(f"  -> Fallback Selection: '{preferred_model_key}'."); break
            else: print(f"      -> Skipping fallback {model_key_fb} (summary failed).")
if preferred_model_key and (preferred_model_key not in regression_results or not isinstance(regression_results.get(preferred_model_key), (PanelEffectsResults, RandomEffectsResults))): print(f"  -> ERROR: Preferred model '{preferred_model_key}' not valid. Resetting."); preferred_model_key = None
if preferred_model_key and preferred_model_key in model_summaries and isinstance(model_summaries[preferred_model_key], str): print(f"  -> ERROR: Preferred model '{preferred_model_key}' failed summary. Resetting."); preferred_model_key = None
if preferred_model_key is None: print("\n  -> !!! CRITICAL: No valid model results available after fallback. !!!")
print(f"\n---> Final Preferred Model Selected (Main Run): {preferred_model_key if preferred_model_key else 'None (Review Logs)'} <---")

print(f"\n--- Interpretation (Based on '{preferred_model_key if preferred_model_key else 'Available Models'}') ---")
interpretation_provided = False
def get_coeff_info(results, var_name):
    if results and hasattr(results, 'params') and var_name in results.params.index:
        coeff = results.params.get(var_name); pval = results.pvalues.get(var_name); stderr = results.std_errors.get(var_name)
        sig_marker = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else '.' if pval < 0.1 else ''
        return f"Coeff={coeff:.4f}, SE={stderr:.4f}, Pval={pval:.4f} {sig_marker}"
    return f"Variable '{var_name}' not found or results invalid."

if preferred_model_key and preferred_model_key in regression_results and isinstance(regression_results[preferred_model_key], (PanelEffectsResults, RandomEffectsResults)):
    preferred_model_results = regression_results[preferred_model_key]; formula_used_key = model_formulas_used.get(preferred_model_key, "Unknown")
    print(f"    Interpreting Preferred Model: '{preferred_model_key}' (Formula Type: {formula_used_key})")
    if primary_esg_var:
        print(f"\n    Interpretation for '{primary_esg_var}':")
        if preferred_model_key == 'Pooled_Interaction' and any(':' in p for p in preferred_model_results.params.index):
             ref_cat_used = 'Unknown'; formula_string_to_parse = formula_pooled_interaction
             if formula_string_to_parse:
                  try: match = re.search(r"Treatment\(reference='([^']+)'\)", formula_string_to_parse); ref_cat_used = match.group(1) if match else 'Unknown'
                  except Exception: pass
             print(f"      Model includes interactions (Ref Cat: '{ref_cat_used}')"); print(f"      - Baseline Effect ({ref_cat_used}): {get_coeff_info(preferred_model_results, primary_esg_var)}"); print("      - (Interactions: See Pooled summary)")
        elif preferred_model_key in ['FE_Entity_Simple', 'FE_TwoWay_Simple', 'RE_Simple']:
             effects_description = "Unknown"
             if 'FE_Entity' in preferred_model_key: effects_description = "Entity Fixed"
             elif 'FE_TwoWay' in preferred_model_key: effects_description = "Entity & Time Fixed"
             elif 'RE' in preferred_model_key: effects_description = "Random Entity"
             print(f"      Model estimates avg effect controlling for {effects_description} effects."); print(f"      - Average Effect: {get_coeff_info(preferred_model_results, primary_esg_var)}")
        else: print(f"      - Unknown structure. Basic Effect: {get_coeff_info(preferred_model_results, primary_esg_var)}")
    else: print("    -> Primary ESG variable not specified/found.")
    print("\n    Interpretation Notes:"); print(f"      - Results based on '{preferred_model_key}'. Check Pval.");
    if IMPUTE_DATA and initial_missing_stats: print("      - !!! CAVEAT: Uses imputed data. High initial missingness. Compare w/ Sensitivity. !!!")
    print("      - Consider economic significance."); interpretation_provided = True
elif RUN_WITHOUT_IMPUTATION_SENSITIVITY:
    print("\n    -> Main run failed/not selected. Checking Sensitivity Run...")
    sens_key_to_interpret = None; sens_results = None; sens_pref_order = ['FE_Entity_Simple_Sens', 'Pooled_Interaction_Sens']
    for key in sens_pref_order:
        result_obj = sensitivity_regression_results.get(key) # Check result object
        summary_obj_or_err = sensitivity_summaries.get(key) # Check summary object/error
        if isinstance(result_obj, (PanelEffectsResults, RandomEffectsResults)) and not isinstance(summary_obj_or_err, str):
            sens_key_to_interpret = key; sens_results = result_obj; break
    if sens_key_to_interpret and sens_results:
        print(f"    -> Interpreting Sensitivity Model: '{sens_key_to_interpret}' (NO IMPUTATION) as fallback."); print("       !!! CAVEATS: Smaller subset of data. !!!")
        if primary_esg_var: print(f"       - ESG Effect ('{primary_esg_var}'): {get_coeff_info(sens_results, primary_esg_var)}")
        else: print("       - Primary ESG variable not found."); interpretation_provided = True
    else: print("    -> Sensitivity models also failed or were skipped.")
if not interpretation_provided: print("\n    -> No valid model results found to provide interpretation.")

# ==============================================================================
# --- Step 9: Print Consolidated Results to Console ---
# ==============================================================================
print("\n\n==============================================================================")
print(f"--- 9. Consolidated Analysis Results ({SCRIPT_VERSION}) ---")
print("==============================================================================")

print(f"\n--- Panel Model Summaries (Main Run - Imputed: {IMPUTE_DATA}) ---")
if not model_summaries: print("No models run/attempted.")
else:
    model_order = ['Pooled_Interaction', 'RE_Simple', 'FE_Entity_Simple', 'FE_TwoWay_Simple']
    for name in model_order:
        if name in model_summaries:
            result_or_error_summary = model_summaries[name]
            formula_used_key = model_formulas_used.get(name, "Unknown")
            print(f"\n--- Model: {name} ---"); print(f"    Formula Type Used: {formula_used_key}")
            if isinstance(result_or_error_summary, str): print(f"    -> Model Failed/Summary Error: {result_or_error_summary}")
            elif hasattr(result_or_error_summary, 'tables'):
                try:
                    if hasattr(result_or_error_summary, 'tables') and isinstance(result_or_error_summary.tables, list):
                         for table in result_or_error_summary.tables: print(table.as_text())
                    else: print(str(result_or_error_summary))
                except Exception as e: print(f"    -> Error formatting/printing summary tables for '{name}': {e}");
                try: print(str(result_or_error_summary))
                except: print("    -> Could not even convert summary object to string.")
            else: print(f"    -> Unknown result status stored (Type: {type(result_or_error_summary)}): {result_or_error_summary}")

if RUN_WITHOUT_IMPUTATION_SENSITIVITY:
    print("\n\n--- Panel Model Summaries (Sensitivity Run - NO IMPUTATION) ---")
    if not sensitivity_summaries: print("No sensitivity models run/results available.")
    else:
        sens_model_order = ['Pooled_Interaction_Sens', 'FE_Entity_Simple_Sens']
        for name in sens_model_order:
             if name in sensitivity_summaries:
                result_or_error_summary = sensitivity_summaries[name]
                formula_used_key = sensitivity_formulas_used.get(name, "Unknown")
                print(f"\n--- Model: {name} ---"); print(f"    Formula Type Used: {formula_used_key}")
                if isinstance(result_or_error_summary, str): print(f"    -> Model Failed/Summary Error: {result_or_error_summary}")
                elif hasattr(result_or_error_summary, 'tables'):
                     try:
                         if hasattr(result_or_error_summary, 'tables') and isinstance(result_or_error_summary.tables, list):
                              for table in result_or_error_summary.tables: print(table.as_text())
                         else: print(str(result_or_error_summary))
                     except Exception as e: print(f"    -> Error formatting/printing summary tables for '{name}': {e}");
                     try: print(str(result_or_error_summary))
                     except: print("    -> Could not even convert summary object to string.")
                else: print(f"    -> Unknown result status stored for {name} (Type: {type(result_or_error_summary)}): {result_or_error_summary}")

print(f"\n\n--- Preferred Model Selection & Comparison (Main Run) ---")
print(f"  -> Preferred model selected: {preferred_model_key if preferred_model_key else 'None (Review Logs)'}"); print(f"  -> Review specification tests and theory.")
if preferred_model_key and RUN_WITHOUT_IMPUTATION_SENSITIVITY:
    sens_key_map = {'Pooled_Interaction': 'Pooled_Interaction_Sens', 'FE_Entity_Simple': 'FE_Entity_Simple_Sens', 'FE_TwoWay_Simple': None, 'RE_Simple': None}
    sens_key = sens_key_map.get(preferred_model_key)
    if sens_key and sens_key in sensitivity_summaries:
        sens_result_or_error = sensitivity_summaries[sens_key]
        if not isinstance(sens_result_or_error, str): print(f"  -> COMPARISON: Sensitivity '{sens_key}' ran successfully. Compare results.")
        else: print(f"  -> COMPARISON NOTE: Sensitivity counterpart '{sens_key}' failed/skipped ({sens_result_or_error}).")
    elif sens_key: print(f"  -> COMPARISON NOTE: Sensitivity counterpart '{sens_key}' not found in sensitivity results.")
    else: print(f"  -> COMPARISON NOTE: No sensitivity counterpart defined for '{preferred_model_key}'.")

print("\n--- Interpretation Summary (Refer to Step 7) ---")
if interpretation_provided: print("  -> Interpretation provided in Step 7.")
else: print("  -> No interpretation provided (model failures).")

print("\n--- Specification Tests Summary (Main Run) ---")
if 'spec_test_df' in locals() and isinstance(spec_test_df, pd.DataFrame) and not spec_test_df.empty:
    try: print(spec_test_df.to_string(index=False, justify='left', max_colwidth=60))
    except Exception as print_err: print(f"  -> Print error: {print_err}"); print(spec_test_df)
else: print("  -> Specification tests unavailable.")

print("\n--- VIF Results (Main Run Data) ---"); print("  (VIF > 10 indicates potential issues)")
if vif_results_total is not None and isinstance(vif_results_total, pd.DataFrame): print("\n  VIF (Factors + Total ESG):"); print(vif_results_total.sort_values('VIF', ascending=False).to_string(index=False))
else: print("\n  VIF check (Total ESG) N/A or failed.")
if vif_results_components is not None and isinstance(vif_results_components, pd.DataFrame): print("\n  VIF (Factors + ESG Components):"); print(vif_results_components.sort_values('VIF', ascending=False).to_string(index=False))
else: print("\n  VIF check (Components) N/A or failed.")

print("\n\n--- Overall Reliability Assessment & Disclaimers ---")
print("  - Data Quality: Verify sources (Steps 2 & 3 logs).")
print(f"  - Imputation ({'MICE' if IMPUTE_DATA else 'Disabled'}): Main run {'used' if IMPUTE_DATA else 'did not use'} imputation.")
if IMPUTE_DATA and initial_missing_stats: high_missing_cols = [col for col, pct in initial_missing_stats.items() if pct > 25];
if IMPUTE_DATA and initial_missing_stats and high_missing_cols: print(f"    -> !!! CONCERN: High initial missingness (>25%): {high_missing_cols}.")
if IMPUTE_DATA: print("    -> Compare Main Run vs. Sensitivity Run (if successful).")
print("  - Model Specification & Validity:")
main_models_failed_count = sum(1 for res in model_summaries.values() if isinstance(res, str))
if main_models_failed_count == len(model_summaries): print("    -> !!! CRITICAL: All main run models failed. Review Step 6 logs. !!!")
elif main_models_failed_count > 0: print(f"    -> Warning: {main_models_failed_count} main run model(s) failed/summary error. Review logs.")
# Check specific model failures
if 'Pooled_Interaction' in model_summaries and isinstance(model_summaries['Pooled_Interaction'], str): print("    -> Warning: Pooled OLS model failed or had summary error.")
elif 'Pooled_Interaction' in regression_results and isinstance(regression_results['Pooled_Interaction'], PanelEffectsResults):
    try:
        pooled_f_robust = regression_results['Pooled_Interaction'].f_statistic_robust.stat
        if not np.isfinite(pooled_f_robust) or pooled_f_robust < 0 :
             print("    -> Warning: Pooled OLS robust F-stat appears invalid. Check SE calculation.")
    except Exception:
        print("    -> Warning: Could not access Pooled OLS robust F-statistic.")
        pass
if 'RE_Simple' in model_summaries and isinstance(model_summaries['RE_Simple'], str): print("    -> Warning: RE model failed. Hausman test invalid.")
fe_models_to_check = ['FE_Entity_Simple', 'FE_TwoWay_Simple']
for fe_key in fe_models_to_check:
     if fe_key in model_summaries:
          summary_val = model_summaries[fe_key]
          if isinstance(summary_val, str) and ('Singular' in summary_val or 'Error' in summary_val): print(f"    -> Warning: {fe_key} failed summary ({summary_val[:60]}...).")
          elif not isinstance(summary_val, str) and hasattr(summary_val, 'summary'):
               summary_str = str(summary_val);
               if 'Absorbed' in summary_str or 'dropped' in summary_str.lower(): print(f"    -> Warning: {fe_key} summary indicates absorbed/dropped vars.")

print("    -> Note: FE/RE models used simplified spec (no category interactions). Pooled OLS used interactions.")
print("    -> Review model selection (Step 7) and theoretical fit.")
print("  - Data Characteristics: Check VIF results and Step 4 category warnings.")
print("\n  --- Conclusion ---"); print("  -> Treat findings with caution. Prioritize successful Sensitivity Run if Main had issues/imputation.")

print("\n==============================================================================")
print("--- End of Consolidated Results ---")
print("==============================================================================")

print(f"\n--- Script Finished ({SCRIPT_VERSION}) ---")

Attempting analysis for tickers: ['UPS', 'FDX', 'XPO', 'JYD', 'AMKBY', 'CNI', 'CP', 'UNP', 'CSX', 'ODFL', 'JBHT', 'ZIM']
--- Global Transportation ESG Impact Analysis Script Started (Panel Only v14.1 - Transportation & FF DataReader) ---
Analysis Period: 2015-01-01 to 2023-12-31
ESG Lag: 1 months
Imputation Enabled (Main Run - MICE): True
Run Sensitivity without Imputation: True
Factors Path: Data fetched from K. French Library
ESG Data Path: /content/historic_esg_scores_global_transportation.csv (Source/Quality Not Verified by Script)

--- 1. Downloading and Preparing Stock Returns ---
  -> Stock price data processed for 12 tickers: ['AMKBY', 'CNI', 'CP', 'CSX', 'FDX', 'JBHT', 'JYD', 'ODFL', 'UNP', 'UPS', 'XPO', 'ZIM']
  -> Stock monthly returns prepared: 2014-10-31 to 2024-12-31

--- 2. Loading and Preparing Fama-French Factors ---
  -> Downloading Fama-French factors using pandas-datareader...
  -> Factors downloaded and merged successfully.
  -> Raw factors shape: (132, 7)
  -> No 

### Good Transport firms Panel Regression

In [5]:
pip install pandas yfinance statsmodels fancyimpute

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.5 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=c408b69adef19bccd4367f520c0db36bcbfca67918696d2ec24a930f7f472dd6
  Stored in directory: /root/.cache/pip/wheels/1a/f3/a1/f7f10b5ae2c2459398762a3fcf4ac18c325311c7e3163d5a15
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11331 sha256=fe292638285209604799af734c7d19c51c3fb512ac008a30d0b1c9e2b8f2f79d
  Stored in directory: /root/.cache/pip/wheels/ea/e8/e0/79872972161e54486517ae507f94b2c7cea27fb7ef793bd415
Successfully built fancyimpute knnimpute


In [15]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.linear_model import OLS # Keep OLS for single entity fallback
from fancyimpute import IterativeImputer # MICE imputation
import io
import yesg # Import yesg

# Import PanelOLS correctly from linearmodels
try:
    from linearmodels.panel import PanelOLS
    panelols_available = True
    print("linearmodels PanelOLS imported successfully.")
except ImportError:
    panelols_available = False
    print("Warning: linearmodels not found. PanelOLS will not be available. Only Pooled OLS with robust errors will be used.")
    print("Please install linearmodels: pip install linearmodels")


# --- 1. Define Inputs ---
firms = {
    "Low ESG Risk": ["DPSGY", "AMKBY", "KHNGY", "DSDVY", "CNI", "CP"],
    "Medium ESG Risk": ["UPS", "FDX", "UNP", "CSX", "XPO", "ODFL", "JBHT", "ZIM"],
    "High ESG Risk": ["JYD"]
}
all_tickers_initial = [ticker for group in firms.values() for ticker in group]

# Define Date Ranges
# Full period for FF6 Only model
start_date_full = "2014-09-01"
end_date_full = "2025-03-01"

# Period post ESG methodology change (Nov 2019 change, use data from Dec 2019 onwards)
start_date_post_change = "2019-12-01"
end_date_post_change = "2025-03-01" # Same end date as full period

# Date range needed for downloading initial stock prices (starts one month before the first return needed)
start_date_prices_download = "2014-08-01"
# End date for prices to calculate returns up to end_date_full
end_date_prices_download = "2025-03-31"

# Define ESG score columns to use
esg_cols_to_use = ["Total-Score", "E-Score", "S-Score", "G-Score"]
ff_factors = ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'WML']

# --- 2. Load Fama-French Data ---
# Using the provided string data
ff_data = """Date	Mkt_RF	SMB	HML	RMW	CMA	RF	WML
9/1/14	-3.09	-2.77	-0.83	0.18	-0.23	0	1.08
10/1/14	0.33	-0.58	-2.81	0.84	-0.23	0	-0.14
11/1/14	1.65	-2.44	-2	0.76	-0.27	0	0.98
12/1/14	-1.44	1.78	0.04	-0.31	0.13	0	1.12
1/1/15	-1.74	-0.05	-2.49	2.09	-0.69	0	3.9
2/1/15	5.91	-0.51	-0.3	-0.99	-0.88	0	-2.43
3/1/15	-1.17	1.44	-0.8	0.05	-0.4	0	1.75
4/1/15	2.71	0.9	2.27	-0.7	-0.48	0	-3.3
5/1/15	0.5	1.05	-1.12	-1.06	-0.41	0	4.29
6/1/15	-2.09	2.06	-0.66	0	-0.27	0	1.99
7/1/15	1.13	-3.29	-3.25	1.76	-0.72	0	3.53
8/1/15	-6.18	1.37	1.27	0.51	0.84	0	-0.68
9/1/15	-3.91	-0.1	-1.02	1.86	0.53	0	3.48
10/1/15	7.32	-2.36	0.34	0.82	-0.49	0	-2.61
11/1/15	-0.3	1.46	-1.99	0.55	-1.39	0	2.07
12/1/15	-1.73	0.84	-1.7	0.76	-0.43	0.01	3.39
1/1/16	-6.32	-1.75	0.76	2.73	2.75	0.01	0.58
2/1/16	-0.5	0.93	-0.21	1.17	1.83	0.02	-2.71
3/1/16	6.92	1.34	0.42	0.66	-0.67	0.02	-2.74
4/1/16	1.91	1.25	2.98	-2.77	0.77	0.01	-3.26
5/1/16	0.45	-0.23	-2.11	0.64	-1.25	0.01	2.93
6/1/16	-1.67	-0.41	-0.73	1.59	2.11	0.02	6.09
7/1/16	4.48	1.38	0.46	0.02	-1.4	0.02	-1.71
8/1/16	0.29	0.56	2.19	-1.38	-0.44	0.02	-3.85
9/1/16	0.9	2.1	-1.23	-0.52	0.13	0.02	1.63
10/1/16	-1.88	-1.09	4.09	-1.07	1.27	0.02	-1.14
11/1/16	1.39	0.61	4.39	-2.02	1.77	0.01	-4.14
12/1/16	2.25	-0.14	2.76	-0.11	0.79	0.03	-1.49
1/1/17	2.72	0.4	-0.53	-0.16	-0.77	0.04	1.76
2/1/17	2.28	-0.71	-1.74	1.14	-0.99	0.04	-1.88
3/1/17	1.48	0.18	-1.81	1.01	-0.54	0.03	0.12
4/1/17	1.86	0.33	-1.39	1.03	-1.01	0.05	0.06
5/1/17	2.2	-0.61	-2.59	2.09	-1.03	0.06	1.11
6/1/17	0.6	1.64	1.74	-1.9	0.05	0.06	0.5
7/1/17	2.51	0.02	1.02	-0.76	0.34	0.07	1.77
8/1/17	0.13	-0.15	-1.34	1.21	-1.3	0.09	1.95
9/1/17	2.3	1.35	1.64	-1.17	1.09	0.09	0.01
10/1/17	1.8	-0.85	-0.9	0.99	-1.92	0.09	3.44
11/1/17	1.93	-0.67	-0.13	1.43	-0.21	0.08	0.46
12/1/17	1.38	0.83	0.11	-0.19	0.81	0.09	-1.07
1/1/18	5.15	-1.21	-1.43	-0.17	-0.54	0.11	3.27
2/1/18	-4	0.77	-1.79	1.1	-1.77	0.11	2.43
3/1/18	-1.76	1.2	-0.42	0.4	-0.55	0.12	-1.11
4/1/18	0.93	-0.4	1.76	-1.53	1.02	0.14	-0.49
5/1/18	0.49	1.51	-4.52	1.21	-2.58	0.14	3.11
6/1/18	-0.49	-0.8	-1.39	0.45	-0.08	0.14	-1.38
7/1/18	2.53	-2.48	1.19	-0.1	0.25	0.16	-1.63
8/1/18	0.87	-0.71	-4.19	1.2	-2.61	0.16	3.46
9/1/18	0.32	-1.43	0.3	0.16	0.57	0.15	0.59
10/1/18	-8.19	-2.28	2.44	0.28	2.31	0.19	-2.46
11/1/18	0.88	-0.62	-1.22	0.46	0.08	0.18	-1.58
12/1/18	-7.67	-0.96	0.56	-0.22	0.13	0.19	1.46
1/1/19	7.47	-0.14	-0.71	0.15	-1.19	0.21	-3.85
2/1/19	2.94	0.1	-2.09	0.21	-0.93	0.18	0.36
3/1/19	0.76	-2.08	-3.28	1.32	-1.49	0.19	2.89
4/1/19	3.44	-1.18	-0.26	0.8	-1.09	0.21	-2.81
5/1/19	-5.99	0.54	-1.7	0.45	0.8	0.21	6.68
6/1/19	6.13	-1.99	-0.17	0.43	-0.02	0.18	-1.07
7/1/19	-0.07	-1.13	-0.63	0.64	-0.33	0.19	1.01
8/1/19	-2.55	-1.64	-2.7	1.19	-0.18	0.16	4.32
9/1/19	1.9	0.19	4.15	0.41	2.17	0.18	-4.43
10/1/19	2.56	0.81	-0.53	0.96	-0.72	0.15	-1.23
11/1/19	2.74	0.04	-2.22	-0.09	-1.2	0.12	-1.02
12/1/19	3.05	1.34	0.62	-0.32	0.48	0.14	0.27
1/1/20	-1.2	-2.77	-4.33	0.07	-1.97	0.13	4.15
2/1/20	-8.54	-1.51	-1.07	-1.13	-1.73	0.12	0.19
3/1/20	-13.77	-4.44	-9.3	1.47	-0.88	0.12	6.23
4/1/20	10.68	1.76	-4.62	2.66	-3	0	0.3
5/1/20	5.3	2.02	-5.1	1.92	-3.18	0.01	1.57
6/1/20	2.76	-0.03	-1.73	-0.09	-0.67	0.01	2.21
7/1/20	4.34	-1.76	-3.64	1.33	-1.21	0.01	6.45
8/1/20	6.84	0.98	-3.88	2.27	-1.52	0.01	0.71
9/1/20	-2.99	1.98	-2.46	0.74	-1.19	0.01	2.4
10/1/20	-2.6	1.43	2.04	-1.04	-0.54	0.01	-1.69
11/1/20	13.34	1.29	4.34	-2.35	0.79	0.01	-10.92
12/1/20	4.82	3.16	-1.2	-0.64	0.14	0.01	-0.09
1/1/21	-0.54	2.93	1.01	-2.43	1.31	0	2.64
2/1/21	2.88	1.55	7.46	-1.92	0.61	0	-6.52
3/1/21	3.12	-1.32	5.58	1.98	3.12	0	-4.83
4/1/21	4.49	-1.22	-1.97	1.39	-2.3	0	2.05
5/1/21	1.65	0.24	4.14	0.2	2.29	0	0.69
6/1/21	1.01	-1.01	-5.23	0.7	-2.28	0	0.17
7/1/21	1.04	-2.16	-1.67	4.2	-0.22	0	-0.84
8/1/21	2.42	-0.13	-1.71	0.08	-2.07	0	1.3
9/1/21	-3.81	1.05	3.86	-2.21	1.88	0	0.79
10/1/21	5.04	-3.31	-1.48	1.51	-1.32	0	3.85
11/1/21	-2.64	-2.02	-2.21	4.59	-0.36	0	0.83
12/1/21	3.63	-0.91	3.9	2.18	4.98	0.01	-0.73
1/1/22	-5.58	-2.57	11.96	-2.91	8.09	0	-3.38
2/1/22	-2.25	2.11	2.89	-1.35	2.27	0	0.6
3/1/22	2.16	-1.89	-1.1	0.36	0.79	0	3.59
4/1/22	-8.17	0.48	4.95	0.94	5.97	0	2.32
5/1/22	0.28	-0.5	6.14	-1.04	4.04	0.03	0.12
6/1/22	-8.69	-0.02	-1.41	1.75	-0.64	0.06	1.27
7/1/22	7.79	-0.41	-5.48	2.37	-5.36	0.08	-3.59
8/1/22	-4.22	0.55	2.49	-2.57	1.34	0.19	3.33
9/1/22	-9.46	-1.49	1.89	-1.21	1.17	0.19	3.36
10/1/22	6.79	-1.35	4.41	0.36	3.27	0.23	2.63
11/1/22	7.25	-0.05	-0.28	2.18	0.98	0.29	-1.86
12/1/22	-4.33	2.42	2.51	-0.47	3.25	0.33	3.8
1/1/23	6.92	0.68	-1.89	-1.02	-3.11	0.35	-9
2/1/23	-2.67	0.05	0.69	-0.1	0.48	0.34	0.5
3/1/23	2.27	-3.68	-6.92	4.27	-1.12	0.36	-1.79
4/1/23	1.22	-1.52	1	0.56	1.87	0.35	0.8
5/1/23	-1.61	-1.31	-4.76	0.27	-3.99	0.36	0.32
6/1/23	5.6	-1.49	1.04	0.35	-0.52	0.4	-0.14
7/1/23	2.96	1.2	3.43	-1.05	0.08	0.45	-2.46
8/1/23	-3.06	-1.63	-0.36	2.49	0.55	0.45	2.45
9/1/23	-4.79	-0.76	3.3	0.04	1.08	0.43	-0.55
10/1/23	-3.76	-2.62	0.36	1.41	1.52	0.47	1.28
11/1/23	8.63	-0.88	-1.68	-0.77	-2.72	0.44	2.04
12/1/23	4.97	3.59	1.11	-1.4	-1.34	0.43	-3.67
1/1/24	0.09	-3.49	-0.03	1.2	0.02	0.47	4.05
2/1/24	3.75	-2.59	-1.35	0.14	-2.56	0.42	5.02
3/1/24	2.87	-0.43	4.06	-0.16	1.17	0.43	0.34
4/1/24	-4.19	-0.6	1.33	-0.08	0.31	0.47	-1.63
5/1/24	4.05	0.16	-0.95	1.03	-1.63	0.44	1.89
6/1/24	0.83	-3.27	-4.46	1.12	-1.33	0.41	1.55
7/1/24	1.81	4.21	3.36	-2.55	2.02	0.45	-2.83
8/1/24	1.83	-2.23	-1.93	1.25	-0.09	0.48	1.35
9/1/24	1.51	-0.07	-0.72	-0.46	0.32	0.4	-0.64
10/1/24	-2.47	-2.01	1.29	-1.44	0.91	0.39	2.82
11/1/24	4.08	-0.94	0.61	-2.47	-0.54	0.4	2.4
12/1/24	-3.15	-1.21	-2.2	2.07	0.05	0.37	-0.56
1/1/25	3.22	-1.4	0.82	-1.52	-2.7	0.37	0.66
2/1/25	-1.22	-1.38	3.75	0.79	2.6	0.33	-1.59
3/1/25	-4.35	2.43	4.4	0.39	0.35	0.34	-1.57"""

ff_df = pd.read_csv(io.StringIO(ff_data), sep='\t')
ff_df['Date'] = pd.to_datetime(ff_df['Date'], format='%m/%d/%y') + pd.offsets.MonthEnd(0)
# Convert percentages to decimals
ff_df[ff_factors] = ff_df[ff_factors] / 100
ff_df['RF'] = ff_df['RF'] / 100 # Risk-Free Rate

# Filter FF data to the full desired range
ff_df_full = ff_df[(ff_df['Date'] >= pd.to_datetime(start_date_full) + pd.offsets.MonthEnd(0)) &
                   (ff_df['Date'] <= pd.to_datetime(end_date_full) + pd.offsets.MonthEnd(0))].copy()


# --- 3. Load ESG Data using yesg ---
print("\nDownloading historic ESG data using yesg...")
all_esg_data_list = []
failed_esg_tickers = []

for ticker in all_tickers_initial:
    print(f"Attempting to download ESG data for {ticker}...")
    try:
        esg_ticker_df = yesg.get_historic_esg(ticker)
        if esg_ticker_df is not None and not esg_ticker_df.empty:
            esg_ticker_df = esg_ticker_df.reset_index()
            esg_ticker_df.columns = ['Date', 'Total-Score', 'E-Score', 'S-Score', 'G-Score']
            esg_ticker_df['Ticker'] = ticker
            # Ensure Date is end of month for merging
            esg_ticker_df['Date'] = pd.to_datetime(esg_ticker_df['Date']) + pd.offsets.MonthEnd(0)
            all_esg_data_list.append(esg_ticker_df)
            print(f"Successfully downloaded ESG data for {ticker}.")
        else:
            print(f"No historic ESG data found for {ticker} using yesg.")
            failed_esg_tickers.append(ticker)
    except Exception as e:
        print(f"Error downloading ESG data for {ticker} using yesg: {e}")
        failed_esg_tickers.append(ticker)

if all_esg_data_list:
    esg_df_raw = pd.concat(all_esg_data_list, ignore_index=True)
    # Filter ESG data to the relevant period *for potential merging* (full period initially)
    esg_df_raw = esg_df_raw[(esg_df_raw['Date'] >= pd.to_datetime(start_date_full) + pd.offsets.MonthEnd(0)) &
                            (esg_df_raw['Date'] <= pd.to_datetime(end_date_full) + pd.offsets.MonthEnd(0))].copy()
else:
    esg_df_raw = pd.DataFrame(columns=['Date', 'Total-Score', 'E-Score', 'S-Score', 'G-Score', 'Ticker'])
    print("No ESG data was successfully downloaded using yesg for any ticker in the specified period.")


# --- 4. Download Stock Data ---
print("\nDownloading stock data...")
stock_data = yf.download(all_tickers_initial, start=start_date_prices_download, end=end_date_prices_download, auto_adjust=False, progress=False)

# Keep only Adjusted Close and resample to monthly (end of month)
stock_adj_close = stock_data['Adj Close'].resample('M').last()

# Check for tickers that failed to download stock data
downloaded_tickers = stock_adj_close.columns.tolist()
failed_stock_tickers = [t for t in all_tickers_initial if t not in downloaded_tickers]
if failed_stock_tickers:
    print(f"Warning: Failed to download stock data for tickers: {failed_stock_tickers}")
    # Remove failed stock tickers from analysis groups and ESG data
    for group, tickers in firms.items():
        firms[group] = [t for t in tickers if t not in failed_stock_tickers]
    all_tickers_analysis = [ticker for group in firms.values() for ticker in group]
    stock_adj_close = stock_adj_close[all_tickers_analysis] # Filter stock data
    if not esg_df_raw.empty:
         esg_df_raw = esg_df_raw[esg_df_raw['Ticker'].isin(all_tickers_analysis)].copy()
    print(f"Analysis proceeding with tickers: {all_tickers_analysis}")


# --- 5. Calculate Returns (using the full period) ---
# Calculate monthly simple returns
stock_returns_full = stock_adj_close.pct_change()

# Reshape stock returns to long format (panel data)
stock_returns_long_full = stock_returns_full.stack().reset_index()
stock_returns_long_full.columns = ['Date', 'Ticker', 'Return']

# Ensure returns and FF dates align precisely for the full period
stock_returns_long_full = stock_returns_long_full[stock_returns_long_full['Date'].isin(ff_df_full['Date'])].copy()
ff_df_aligned = ff_df_full[ff_df_full['Date'].isin(stock_returns_long_full['Date'])].copy() # Ensure FF matches return dates

# Merge returns with Fama-French data for the full period
full_df = pd.merge(stock_returns_long_full, ff_df_aligned, on='Date', how='left')

# Calculate Excess Return (Return - RF)
full_df['Excess_Return'] = full_df['Return'] - full_df['RF']

# Drop the individual 'Return' and 'RF' columns
full_df = full_df.drop(columns=['Return', 'RF'], errors='ignore')

# Merge the combined FF+Returns data with ESG data (full period initially)
# Use left merge to keep all stock return/FF observations, adding ESG where available
full_panel_df_raw_esg = pd.merge(full_df, esg_df_raw, on=['Date', 'Ticker'], how='left')

# Set MultiIndex for panel data
full_panel_df_raw_esg = full_panel_df_raw_esg.set_index(['Ticker', 'Date'])

# Ensure relevant columns are numeric
# Only include ESG columns if they were actually downloaded/exist in esg_df_raw
available_esg_cols = [col for col in esg_cols_to_use if col in full_panel_df_raw_esg.columns]
numeric_cols = ['Excess_Return'] + ff_factors + available_esg_cols
for col in numeric_cols:
    full_panel_df_raw_esg[col] = pd.to_numeric(full_panel_df_raw_esg[col], errors='coerce')


# --- 6. Handle Missing ESG Data (MICE) ---
# Note: MICE should be applied BEFORE splitting by date, to use all available data for imputation.
# Identify columns to impute (ESG scores). Only impute if the column exists and has NaNs.
cols_to_impute = [col for col in available_esg_cols if full_panel_df_raw_esg[col].isnull().any()]

if cols_to_impute:
    print(f"\nApplying MICE imputation to: {cols_to_impute}")
    df_for_imputation = full_panel_df_raw_esg.copy()
    # Use all currently existing numeric columns for imputation predictors
    numerical_cols_for_mice = df_for_imputation.select_dtypes(include=['float64', 'int64']).columns.tolist()

    # Check if there's enough non-missing data across relevant columns to run MICE
    # Need at least 2 non-NaN rows across the columns being used for imputation predictors
    if df_for_imputation[numerical_cols_for_mice].dropna(how='all').shape[0] < 2:
         print("Warning: Not enough non-missing data across numerical columns to perform MICE. Skipping imputation.")
         full_panel_df_imputed = full_panel_df_raw_esg # Keep original with NaNs
    else:
        try:
            imputer = IterativeImputer(max_iter=10, random_state=0)
            imputed_data = imputer.fit_transform(df_for_imputation[numerical_cols_for_mice])
            imputed_df = pd.DataFrame(imputed_data, columns=numerical_cols_for_mice, index=full_panel_df_raw_esg.index)

            # Copy imputed columns back to the main dataframe structure
            full_panel_df_imputed = full_panel_df_raw_esg.copy()
            for col in numerical_cols_for_mice:
                 full_panel_df_imputed[col] = imputed_df[col]

            print("MICE imputation complete.")
        except Exception as e:
            print(f"Error during MICE imputation: {e}")
            print("Skipping MICE imputation. Analysis will use data with NaNs (which will be dropped per model).")
            full_panel_df_imputed = full_panel_df_raw_esg # Keep original with NaNs

else:
     print("\nNo ESG columns found with missing values to impute after data retrieval.")
     full_panel_df_imputed = full_panel_df_raw_esg # No imputation needed


# --- 7. Prepare dataframes for different analysis periods ---
# Add ESG Group to the DataFrame (before splitting)
esg_group_map = {}
for group, tickers in firms.items():
    for ticker in tickers:
        esg_group_map[ticker] = group

full_panel_df_imputed['ESG_Group'] = full_panel_df_imputed.index.get_level_values('Ticker').map(esg_group_map)

# Drop rows where Excess_Return is NaN (cannot perform regression)
initial_rows = len(full_panel_df_imputed)
full_panel_df_imputed = full_panel_df_imputed.dropna(subset=['Excess_Return'])
if len(full_panel_df_imputed) < initial_rows:
    print(f"Dropped {initial_rows - len(full_panel_df_imputed)} rows due to missing Excess_Return.")

# --- FIX: Sort the index before slicing ---
full_panel_df_imputed = full_panel_df_imputed.sort_index()
# --- End FIX ---

# Filter data for the post-change period for ESG analysis
post_change_panel_df = full_panel_df_imputed.loc[(slice(None), slice(pd.to_datetime(start_date_post_change) + pd.offsets.MonthEnd(0), pd.to_datetime(end_date_post_change) + pd.offsets.MonthEnd(0))), :].copy()

print(f"\nAnalysis period for FF6 Only: {start_date_full} to {end_date_full}")
print(f"Analysis period for FF6 + ESG: {start_date_post_change} to {end_date_post_change}")


# --- 8. Analysis per Group and per Model Specification ---

# Define the model specifications to test
model_specs = {
    'FF6 Only': {'esg_predictors': [], 'period_df': full_panel_df_imputed},
    'FF6 + Total-Score': {'esg_predictors': ['Total-Score'], 'period_df': post_change_panel_df},
    'FF6 + E-Score': {'esg_predictors': ['E-Score'], 'period_df': post_change_panel_df},
    'FF6 + S-Score': {'esg_predictors': ['S-Score'], 'period_df': post_change_panel_df},
    'FF6 + G-Score': {'esg_predictors': ['G-Score'], 'period_df': post_change_panel_df}
}

# Identify which ESG columns are actually present in the imputed data
available_esg_cols_imputed = [col for col in esg_cols_to_use if col in full_panel_df_imputed.columns]

for group_name, group_tickers in firms.items():
    print(f"\n--- Analyzing: {group_name} ---")

    # Determine which models to run for this group based on data availability
    model_specs_group_filtered = model_specs.copy()

    # Check for ESG data availability for the ESG models in this group *in the post-change period*
    for model_name in list(model_specs_group_filtered.keys()): # Iterate over a copy
        if model_name != 'FF6 Only':
            esg_col = model_specs_group_filtered[model_name]['esg_predictors'][0]

            # Check if the ESG column exists at all in the imputed data
            if esg_col not in available_esg_cols_imputed:
                 print(f"Note: Skipping '{model_name}' for '{group_name}' group as ESG column '{esg_col}' is not available in the data.")
                 del model_specs_group_filtered[model_name]
                 continue

            # Filter the post_change_panel_df to the current group
            group_post_change_df = post_change_panel_df[post_change_panel_df.index.get_level_values('Ticker').isin(group_tickers)].copy()

            # Check if the ESG column has any non-NaN data in this specific group/period subset
            if group_post_change_df[esg_col].isnull().all():
                 print(f"Note: Skipping '{model_name}' for '{group_name}' group due to lack of non-missing ESG data in the post-change period.")
                 del model_specs_group_filtered[model_name]
            elif group_name == "High ESG Risk": # Explicitly skip ESG models for High group due to lack of original data/imputation issues
                 print(f"Note: Skipping '{model_name}' for 'High ESG Risk' group due to data limitations (only JYD, no original ESG data via yesg).")
                 del model_specs_group_filtered[model_name]


    # Iterate through applicable model specifications for the current group
    for model_name, spec in model_specs_group_filtered.items():
        print(f"\n--- Model: {model_name} ---")

        esg_predictors = spec['esg_predictors']
        period_df = spec['period_df']

        # Filter data for the current group AND the specific period
        # The period_df is already filtered by date range, now filter by ticker group
        model_df = period_df[period_df.index.get_level_values('Ticker').isin(group_tickers)].copy()

        predictors = ff_factors + esg_predictors

        # Drop rows with any NaN in the selected predictors or dependent variable for THIS specific model
        model_df = model_df.dropna(subset=['Excess_Return'] + predictors).copy()

        if model_df.empty:
            print(f"Not enough data left for '{model_name}' analysis in this group/period after dropping NaNs.")
            continue

        # --- VIF Calculation (on pooled data for the specific model's predictors) ---
        print("\nPerforming VIF Calculation:")
        X_vif = model_df[predictors]
        # Add a constant for VIF calculation
        X_vif = sm.add_constant(X_vif, has_constant='add')

        # Ensure no NaNs left in X_vif before VIF (should be handled by dropna above)
        if X_vif.isnull().sum().sum() > 0:
             print("Warning: NaNs found in VIF predictors. Skipping VIF.")
        else:
            vif_data = pd.DataFrame()
            vif_data['feature'] = X_vif.columns
            try:
                # Check for sufficient data points for VIF calculation (must be > number of predictors)
                if X_vif.shape[0] > X_vif.shape[1]:
                    vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
                    print(vif_data.round(2))
                    print("Thresholds: Typically VIF > 5 or 10 indicates high multicollinearity.")
                else:
                    print("Not enough observations to compute VIFs.")

            except Exception as e:
                print(f"Could not compute VIFs: {e}")
                print("This might be due to perfect multicollinearity or insufficient data points.")


        # --- Panel Data Regression ---
        print(f"\nPerforming Regression ({'Fixed Effects PanelOLS with Robust Standard Errors' if panelols_available else 'Pooled OLS with Robust Standard Errors'}):")
        y = model_df['Excess_Return']
        X_reg = model_df[predictors] # Use predictors *without* constant for PanelOLS

        num_entities = len(X_reg.index.get_level_values('Ticker').unique())
        num_dates = len(X_reg.index.get_level_values('Date').unique())
        num_obs = len(X_reg)
        num_model_params = X_reg.shape[1] # Number of predictor variables


        # Check if PanelOLS is applicable (needs >1 entity and >1 date, and enough observations)
        # linearmodels requires the index to be sorted
        model_df_sorted = model_df.sort_index()
        y_sorted = model_df_sorted['Excess_Return']
        X_reg_sorted = model_df_sorted[predictors]

        if panelols_available and num_entities > 1 and num_dates > 1:
            try:
                 # Heuristic check for sufficient observations for FE identification
                 # A stricter check might be needed in research, but this avoids common errors
                 if num_obs > num_model_params + num_entities:
                    model = PanelOLS(y_sorted, X_reg_sorted, entity_effects=True)
                    # Use robust standard errors
                    results = model.fit(cov_type='robust')
                    print(results)
                 else:
                     print(f"Warning: Not enough observations ({num_obs}) relative to number of entities ({num_entities}) and predictors ({num_model_params}) for Fixed Effects ({num_entities + num_model_params} minimum recommended). Running OLS instead.")
                     # Fallback to OLS with robust standard errors
                     X_reg_ols = sm.add_constant(X_reg_sorted, has_constant='add')
                     try:
                         model_ols = OLS(y_sorted, X_reg_ols)
                         results_ols = model_ols.fit(cov_type='HC1') # HC1 is a common robust estimator for OLS
                         print(results_ols.summary())
                     except Exception as e_ols:
                         print(f"Error running OLS fallback for {group_name} - {model_name}: {e_ols}")

            except Exception as e:
                 print(f"Error running PanelOLS for {group_name} - {model_name}: {e}")
                 print("Attempting PooledOLS with robust standard errors instead.")
                 # Fallback to PooledOLS with robust standard errors if FixedEffects fails or conditions not met strictly
                 X_reg_pooled = sm.add_constant(X_reg_sorted, has_constant='add')
                 try:
                     model_pooled = OLS(y_sorted, X_reg_pooled)
                     results_pooled = model_pooled.fit(cov_type='HC1') # HC1 is a common robust estimator for OLS
                     print(results_pooled.summary())
                 except Exception as e_pooled:
                     print(f"Error running PooledOLS fallback for {group_name} - {model_name}: {e_pooled}")

        else:
            # Run OLS with robust standard errors if PanelOLS conditions are not met
            if not panelols_available:
                 print("linearmodels not installed. Running PooledOLS with robust errors.")
            elif num_entities <= 1:
                 print("Warning: Only one entity (firm) left in this subset. Panel regression is not meaningful. Running OLS with robust errors instead.")
            elif num_dates <= 1:
                 print("Warning: Only one time period left in this subset. Panel regression is not meaningful. Running OLS with robust errors instead.")

            X_reg_ols = sm.add_constant(X_reg_sorted, has_constant='add')
            try:
                model_ols = OLS(y_sorted, X_reg_ols)
                results_ols = model_ols.fit(cov_type='HC1') # HC1 is a common robust estimator for OLS
                print(results_ols.summary())
            except Exception as e_ols:
                print(f"Error running OLS for {group_name} - {model_name}: {e_ols}")


print("\n--- Analysis Complete ---")
print("\nNote on Reliability:")
print("- Stock data download failed for DPSGY.")
print("- ESG data was retrieved using the 'yesg' library. Download failed or no data found for DPSGY, AMKBY, KHNGY, DSDVY, JYD.")
print("- The ESG rating methodology and scale changed in November 2019 (before: high score = low risk; after: high score = high risk).")
print("- FF6-only models are run on the full period (Sept 2014 - Mar 2025).")
print(f"- FF6 + ESG models are run only on the post-change period ({start_date_post_change} - {end_date_post_change}) to maintain a consistent interpretation of ESG scores.")
print("- MICE imputation was applied to fill remaining gaps in the ESG data (in the full dataset) using other numerical columns as predictors.")
print("- Imputation for firms/periods with very limited original historical ESG data (e.g., for the earlier periods of ZIM, XPO, ODFL, and entirely for the downloaded tickers with failed yesg download) is speculative and reduces the reliability of ESG coefficient estimates for groups containing these firms.")
print("- The High ESG Risk group (JYD) had no original ESG data from 'yesg' for the entire period, so ESG models are skipped for this group.")
print("- VIF analysis helps assess multicollinearity *within the predictors of each specific model*.")
print("- Panel Regression (Fixed Effects) with robust standard errors is used where appropriate, controlling for unobserved firm-specific effects. Fallback to OLS/PooledOLS with robust errors occurs if FE conditions aren't met or only one entity remains.")
print("\nConclusion regarding research reliability:")
print("The findings regarding the Fama-French factors are reasonably reliable, especially for the groups where PanelOLS was successfully applied. However, the ability to draw strong, reliable conclusions about the impact of ESG scores is severely limited by:")
print("  1. The necessity to restrict ESG analysis to the shorter post-change period.")
print("  2. The substantial reliance on MICE imputation for firms/periods with sparse or no original historical ESG data from the 'yesg' source.")
print("Relying solely on these ESG results for definitive research conclusions or generalizations about ESG's impact across the full period or different firms is not recommended without more complete original historical ESG data from a reliable source.")

linearmodels PanelOLS imported successfully.

Attempting to download ESG data for DPSGY...
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
No historic ESG data found for DPSGY using yesg.
Attempting to download ESG data for AMKBY...
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
No historic ESG data found for AMKBY using yesg.
Attempting to download ESG data for KHNGY...
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
No historic ESG data found for KHNGY using yesg.
Attempting to download ESG data for DSDVY...
An error has occurred. The ticker symbol might be wrong or you might need to wait to continue.
No historic ESG data found for DSDVY using yesg.
Attempting to download ESG data for CNI...
Successfully downloaded ESG data for CNI.
Attempting to download ESG data for CP...
Successfully downloaded ESG data for CP.
Attempting to download ESG dat

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['DPSGY']: YFTzMissingError('possibly delisted; no timezone found')



Applying MICE imputation to: ['Total-Score', 'E-Score', 'S-Score', 'G-Score']
MICE imputation complete.

Analysis period for FF6 Only: 2014-09-01 to 2025-03-01
Analysis period for FF6 + ESG: 2019-12-01 to 2025-03-01

--- Analyzing: Low ESG Risk ---

--- Model: FF6 Only ---

Performing VIF Calculation:
  feature    VIF
0   const 1.1500
1  Mkt_RF 1.4100
2     SMB 1.2300
3     HML 5.2200
4     RMW 1.8200
5     CMA 3.7600
6     WML 1.6000
Thresholds: Typically VIF > 5 or 10 indicates high multicollinearity.

Performing Regression (Fixed Effects PanelOLS with Robust Standard Errors):
                          PanelOLS Estimation Summary                           
Dep. Variable:          Excess_Return   R-squared:                        0.3836
Estimator:                   PanelOLS   R-squared (Between):              0.8134
No. Observations:                 593   R-squared (Within):               0.3836
Date:                Tue, Apr 29 2025   R-squared (Overall):              0.3884
Time:   

### 2 Electronics Panel Regression
#### The script will:
Download stock and ESG data for the tickers it can find on Yahoo Finance within the specified date range (Nov 2019 to Mar 2025).
Load and process the Fama-French data for the same period.
Merge the data, creating a panel structure.
Perform MICE imputation if there are missing numerical values.
Split the data based on the initial, static ESG risk grouping.
For each group (Low, Medium, High), it will:
Calculate and print VIFs for the independent variables.
Run and print the summary of a Pooled OLS regression.
Run and print the results of Fixed Effects and Random Effects panel regressions.


In [1]:
!pip install pandas yfinance yesg statsmodels linearmodels scikit-learn openpyxl # openpyxl for excel reading if needed, added as safeguard
!pip install yesg
!pip install linearmodels
!pip install scikit-learn
!pip install openpyxl
!pip install statsmodels


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.9 MB/s eta 0:00:00
  Created wheel for yesg: filename=yesg-2.1.1-py3-none-any.whl size=6105 sha256=746ef1b3040c4153184935636cf3986c3a659230d3c91d235d3653ea2cdc6b5a
  Stored in directory: /root/.cache/pip/wheels/78/8d/48/f5e8ff0315a46301e15c68371e297b460b33e1c846117725bc
Successfully built yesg


In [3]:
import yfinance as yf
import datetime as dt
import pandas as pd

# Try a standard ticker
ticker = "AAPL"
# Try a recent, short date range
start = dt.date.today() - dt.timedelta(days=30) # Last 30 days
end = dt.date.today()
print(f"Attempting to download {ticker} from {start} to {end}")

try:
    data = yf.download(ticker, start=start, end=end)

    if data.empty:
        print("\nTest download failed: Data is empty.")
    elif 'Adj Close' not in data.columns:
        print("\nTest download failed: 'Adj Close' column not found.")
        print("Available columns:", data.columns.tolist()) # See what columns *are* there
    else:
        print("\nTest download successful. Head of data:")
        print(data.head())
        print("\nColumns:", data.columns.tolist()) # Confirm Adj Close is there

except Exception as e:
    print(f"\nTest download failed with an error: {e}")

[*********************100%***********************]  1 of 1 completed

Attempting to download AAPL from 2025-03-30 to 2025-04-29

Test download failed: 'Adj Close' column not found.
Available columns: [('Close', 'AAPL'), ('High', 'AAPL'), ('Low', 'AAPL'), ('Open', 'AAPL'), ('Volume', 'AAPL')]


In [4]:
import pandas as pd
import yfinance as yf
import yesg
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import PanelOLS
import numpy as np
import warnings
from sklearn.experimental import enable_iterative_imputer # Enable IterativeImputer
from sklearn.impute import IterativeImputer
from io import StringIO # To read FF data from string
import time # To add a small delay if needed
import datetime as dt # Import datetime

# Suppress specific warnings for cleaner output, be cautious in production code
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# 1. Identify Target Companies and Tickers
company_ticker_map = {
    "Apple Inc.": "AAPL",
    "Samsung Electro-Mechanics": "009150.KS", # South Korea
    "Sony Group Corp.": "SONY", # US Listing (ADR)
    "LG Display Co., Ltd.": "034220.KS", # South Korea
    "Foxconn Industrial Internet Co., Ltd.": "601138.SS", # Shanghai
    "Siemens Energy": "ENR.DE", # Frankfurt (FWB) - using .DE suffix
    "Philips": "PHG", # US Listing (ADR)
    "GoerTek Inc.": "002241.SZ", # Shenzhen
    "STMicroelectronics": "STM", # US Listing (ADR)
    "Elite Material Co., Ltd.": None, # Need to find ticker, assuming not readily available on Yahoo
    "Garmin Ltd.": "GRMN", # US Listing
    "Hamamatsu Photonics KK": None, # Need to find ticker, assuming not readily available on Yahoo
    "Amazon": "AMZN", # US Listing (for comparison)
    "OFILM Group Co., Ltd.": None, # Need to find ticker, assuming not readily available on Yahoo
    "TCL Technology Group Corp.": "000100.SZ", # Shenzhen - Added based on search, might not be the right entity or ticker
    "Shenzhen Huaqiang Industry Co., Ltd.": None # Need to find ticker, assuming not readily available on Yahoo
}

tickers_to_analyze = {k: v for k, v in company_ticker_map.items() if v is not None}
initial_tickers_list = list(tickers_to_analyze.values())
print(f"Attempting to download data for tickers: {initial_tickers_list}")

# 2. Define ESG Risk Groups (Static based on the prompt)
ticker_esg_group_map = {
    "AAPL": "Low",
    "009150.KS": "Low",
    "SONY": "Low",
    "034220.KS": "Low",
    "601138.SS": "Low",
    "ENR.DE": "Low",
    "PHG": "Low",
    "002241.SZ": "Medium",
    "STM": "Medium",
    "GRMN": "Medium",
    "AMZN": "High",
    "000100.SZ": "High",
}

# Filter the group map to only include tickers we are attempting to analyze
ticker_esg_group_map_working = ticker_esg_group_map.copy() # Create a working copy

# Define the analysis date range
start_date = '2019-11-01'
end_date = '2025-04-01' # Align with FF data end + 1 day

print(f"\nAnalysis period: {start_date} to {end_date}")

# Helper function to get stock price data robustly
def get_stock_prices(ticker, start, end):
    """Downloads stock data and attempts to get 'Adj Close', handling MultiIndex."""
    try:
        # Try with auto_adjust=True explicitly
        # Use timezone=None to avoid potential issues later
        data = yf.download(ticker, start=str(pd.to_datetime(start).date()), end=str(pd.to_datetime(end).date()), progress=False, auto_adjust=True, ignore_tz=True)

        if not data.empty and 'Close' in data.columns:
             # If auto_adjust=True worked, 'Close' should be the adjusted close
             # Rename 'Close' to 'Adj Close' for consistency
             data = data.rename(columns={'Close': 'Adj Close'})

        # If auto_adjust=True didn't work as expected or returned a MultiIndex
        if 'Adj Close' not in data.columns:
             # Try downloading without auto_adjust and check for MultiIndex
             data = yf.download(ticker, start=str(pd.to_datetime(start).date()), end=str(pd.to_datetime(end).date()), progress=False, auto_adjust=False, ignore_tz=True)

             if not data.empty:
                 # Check for MultiIndex columns like ('AAPL', 'Adj Close') or ('AAPL', 'Close')
                 if isinstance(data.columns, pd.MultiIndex):
                     # Flatten the MultiIndex columns to single level, e.g., 'AAPL_Adj Close'
                     # Or try to directly access levels if we know the ticker
                     try:
                         # Try accessing using the ticker directly if it's the first level
                         if ticker in data.columns.get_level_values(0):
                            # Prioritize ('Ticker', 'Adj Close')
                            if (ticker, 'Adj Close') in data.columns:
                                data['Adj Close'] = data[(ticker, 'Adj Close')]
                                print(f"Accessed MultiIndex ('{ticker}', 'Adj Close') for {ticker}")
                            # Fallback to ('Ticker', 'Close') if Adj Close is missing in MultiIndex
                            elif (ticker, 'Close') in data.columns:
                                data['Adj Close'] = data[(ticker, 'Close')]
                                print(f"Accessed MultiIndex ('{ticker}', 'Close') for {ticker}, using as Adj Close")
                            else:
                                print(f"Could not find ('{ticker}', 'Adj Close') or ('{ticker}', 'Close') in MultiIndex for {ticker}. Columns: {data.columns.tolist()}")
                                return pd.DataFrame() # Indicate failure

                         else:
                            # Fallback to trying to find 'Adj Close' or 'Close' in the second level
                            if 'Adj Close' in data.columns.get_level_values(1):
                                # Find the column tuple where the second level is 'Adj Close'
                                adj_close_col = [col for col in data.columns if col[1] == 'Adj Close']
                                if adj_close_col:
                                     data['Adj Close'] = data[adj_close_col[0]]
                                     print(f"Accessed MultiIndex with 'Adj Close' at level 1 for {ticker}")
                                elif 'Close' in data.columns.get_level_values(1):
                                    # Find the column tuple where the second level is 'Close'
                                    close_col = [col for col in data.columns if col[1] == 'Close']
                                    if close_col:
                                         data['Adj Close'] = data[close_col[0]]
                                         print(f"Accessed MultiIndex with 'Close' at level 1 for {ticker}, using as Adj Close")
                                    else:
                                        print(f"Could not find 'Adj Close' or 'Close' in MultiIndex level 1 for {ticker}. Columns: {data.columns.tolist()}")
                                        return pd.DataFrame() # Indicate failure
                                else:
                                    print(f"Could not find 'Adj Close' or 'Close' in MultiIndex level 1 for {ticker}. Columns: {data.columns.tolist()}")
                                    return pd.DataFrame() # Indicate failure
                            else:
                                print(f"Could not find 'Adj Close' or 'Close' in MultiIndex level 1 for {ticker}. Columns: {data.columns.tolist()}")
                                return pd.DataFrame() # Indicate failure


                     except Exception as e_multiindex:
                         print(f"Error processing MultiIndex columns for {ticker}: {e_multiindex}. Columns: {data.columns.tolist()}")
                         return pd.DataFrame() # Indicate failure

                 # Check for standard column names if not MultiIndex
                 elif 'Adj Close' not in data.columns and 'Close' in data.columns:
                     data['Adj Close'] = data['Close'] # Use 'Close' if 'Adj Close' is missing
                     print(f"Using 'Close' column as 'Adj Close' for {ticker}")
                 elif 'Adj Close' not in data.columns:
                     print(f"'Adj Close' and 'Close' columns not found for {ticker}. Available columns: {data.columns.tolist()}")
                     return pd.DataFrame() # Indicate failure
                 # If 'Adj Close' was found directly, the column exists, continue.

        # Ensure index is timezone naive
        if not data.empty and data.index.tz is not None:
            data.index = data.index.tz_convert(None)


        # If data is empty or 'Adj Close' is still missing after checks
        if data.empty or 'Adj Close' not in data.columns:
             print(f"Failed to get usable 'Adj Close' data for {ticker}.")
             return pd.DataFrame() # Indicate failure


        # Return only the 'Adj Close' column
        return data[['Adj Close']]

    except Exception as e:
        print(f"Error during yfinance download or processing for {ticker}: {e}")
        return pd.DataFrame() # Indicate failure


# --- Quick Test Download ---
# Add a small buffer for the quick test end date
quick_test_end_date = pd.to_datetime(start_date) + pd.Timedelta(days=30) # Test for one month starting from start_date

print("\nPerforming quick test download for first ticker...")
test_ticker = initial_tickers_list[0] if initial_tickers_list else None
if test_ticker:
    test_data = get_stock_prices(test_ticker, start_date, quick_test_end_date)
    if test_data.empty:
         print(f"Quick test download failed for {test_ticker}. yfinance might be having issues or data is unavailable.")
         print("Please try troubleshooting steps (check internet, retry later, verify ticker on Yahoo Finance website).")
         # If test fails for the first ticker, remove it from consideration
         if test_ticker in ticker_esg_group_map_working:
             del ticker_esg_group_map_working[test_ticker]
    else:
         print(f"Quick test download successful for {test_ticker}. Found {test_data.shape[0]} data points.")
    print("-" * 30) # Separator
else:
    print("No tickers provided to attempt download.")
    exit()


# 3. Data Acquisition

# --- Stock Data ---
print("\nDownloading stock data for analysis period...")
stock_data = {}
successful_stock_tickers = []

tickers_to_process_stock = list(ticker_esg_group_map_working.keys()) # Use working map after quick test

for ticker in tickers_to_process_stock:
    print(f"Processing stock data for {ticker}...")
    daily_data = get_stock_prices(ticker, start_date, end_date)

    if not daily_data.empty:
        # Resample to monthly, taking the last trading day's adjusted close
        # Ensure we have enough data points before resampling
        if daily_data.shape[0] < 2:
             print(f"Warning: Insufficient daily data points ({daily_data.shape[0]}) for {ticker} to calculate monthly returns. Excluding from analysis.")
             if ticker in ticker_esg_group_map_working:
                del ticker_esg_group_map_working[ticker]
             continue

        monthly_close = daily_data['Adj Close'].resample('M').last()
        # Calculate monthly returns
        monthly_returns = monthly_close.pct_change()

        # --- FIX: Renaming Series Name ---
        # The error "TypeError: 'str' object is not callable" occurs because
        # series.rename('Name') is meant to rename the Series *name*, not its index.
        # However, in some Pandas versions or contexts, passing a string
        # might incorrectly trigger index value renaming using the string
        # as a mapper, leading to the error.
        # The correct and unambiguous way to rename the Series is:
        monthly_returns.name = 'Return'
        # Or using rename method explicitly for name: monthly_returns = monthly_returns.rename(None, name='Return')
        # --- END FIX ---

        # Shift dates to the first of the month for consistency
        # Ensure index is timezone naive before converting to period/timestamp
        if monthly_returns.index.tz is not None:
             monthly_returns.index = monthly_returns.index.tz_convert(None)
        monthly_returns.index = monthly_returns.index.to_period('M').to_timestamp('D')


        # Store the monthly returns if calculation was successful and there's data
        if not monthly_returns.dropna().empty and monthly_returns.shape[0] > 1: # Need at least 2 monthly points for a return
            stock_data[ticker] = monthly_returns # Name is already set to 'Return'
            successful_stock_tickers.append(ticker)
            print(f"Successfully processed monthly stock returns for {ticker}")
        else:
             print(f"Warning: Processed monthly stock returns for {ticker} are all NaN or insufficient data points ({monthly_returns.shape[0]}). Excluding from analysis.")
             if ticker in ticker_esg_group_map_working:
                del ticker_esg_group_map_working[ticker]
    else:
        print(f"Could not retrieve usable daily stock data for {ticker}. Excluding from analysis.")
        if ticker in ticker_esg_group_map_working:
            del ticker_esg_group_map_working[ticker]


# Check if any stock data was successfully downloaded for the remaining tickers
if not stock_data:
    print("\nFATAL ERROR: No stock data was successfully downloaded for any ticker after trying. Cannot proceed with analysis.")
    exit()


# Combine successful stock returns into a single DataFrame
# The 'Return' column name is now correctly set on the Series before concat
stock_returns_df = pd.concat(stock_data, names=['Ticker', 'Date']).reset_index()
# Filter to only include tickers that were successfully processed for stock data
stock_returns_df = stock_returns_df[stock_returns_df['Ticker'].isin(stock_data.keys())].copy()


# --- ESG Data ---
print("\nDownloading ESG data...")
esg_data = {}
successful_esg_tickers = []

tickers_to_process_esg = list(stock_data.keys()) # Use keys from successfully downloaded stock data

for ticker in tickers_to_process_esg:
     if ticker not in ticker_esg_group_map_working:
         continue

     print(f"Processing ESG data for {ticker}...")
     try:
        esg_hist = yesg.get_historic_esg(ticker)
        # Optional: Add a small delay
        # time.sleep(0.5)

        if esg_hist is None or esg_hist.empty:
            print(f"Warning: No ESG data found for {ticker} from yesg. Excluding from analysis.")
            if ticker in ticker_esg_group_map_working:
                del ticker_esg_group_map_working[ticker]
            continue

        # Filter to the analysis period (post-Nov 2019 for risk rating scale)
        esg_hist.index = pd.to_datetime(esg_hist.index)
        # Ensure index is timezone naive BEFORE filtering and resampling
        if esg_hist.index.tz is not None:
             esg_hist.index = esg_hist.index.tz_convert(None)

        esg_filtered = esg_hist.loc[(esg_hist.index >= start_date) & (esg_hist.index <= end_date)].copy() # Use .copy()

        # Ensure index is on the first day of the month
        esg_filtered.index = esg_filtered.index.to_period('M').to_timestamp('D')

        # Keep only the Total-Score
        if 'Total-Score' in esg_filtered.columns:
            # Store ESG data if there's at least some non-NaN data in the score
            if not esg_filtered['Total-Score'].dropna().empty:
                esg_data[ticker] = esg_filtered['Total-Score'].rename('Total_ESG_Risk_Score')
                successful_esg_tickers.append(ticker)
                print(f"Successfully processed ESG data for {ticker}")
            else:
                print(f"Warning: Processed ESG total scores for {ticker} are all NaN in the period {start_date} to {end_date}. Excluding from analysis.")
                if ticker in ticker_esg_group_map_working:
                    del ticker_esg_group_map_working[ticker]
        else:
             print(f"Warning: 'Total-Score' column not found in ESG data for {ticker}. Excluding from analysis.")
             if ticker in ticker_esg_group_map_working:
                del ticker_esg_group_map_working[ticker]

     except Exception as e:
        print(f"Error downloading or processing ESG data for {ticker}: {e}. Excluding from analysis.")
        if ticker in ticker_esg_group_map_working:
             del ticker_esg_group_map_working[ticker]

# Check if any ESG data was successfully downloaded for the remaining tickers
if not esg_data and successful_stock_tickers:
     print("\nFATAL ERROR: No ESG data was successfully downloaded for any ticker that had stock data. Cannot proceed with analysis.")
     # exit() # Let the merge fail naturally

# Update the group map one final time based on successful stock AND ESG download
final_tickers_for_analysis = list(esg_data.keys())
ticker_esg_group_map_final = {k: v for k, v in ticker_esg_group_map_working.items() if k in final_tickers_for_analysis}

if not ticker_esg_group_map_final:
     print("\nFATAL ERROR: No tickers remain for analysis after attempting stock and ESG data downloads. Cannot proceed.")
     exit()

# Combine ESG scores into a single DataFrame
esg_scores_df = pd.concat(esg_data, names=['Ticker', 'Date']).reset_index()


# --- Fama-French Data ---
print("\nLoading Fama-French data...")
try:
    # Use the provided string data directly
    ff_data_string = """Date	Mkt_RF	SMB	HML	RMW	CMA	RF	WML
9/1/14	-3.09	-2.77	-0.83	0.18	-0.23	0	1.08
10/1/14	0.33	-0.58	-2.81	0.84	-0.23	0	-0.14
11/1/14	1.65	-2.44	-2	0.76	-0.27	0	0.98
12/1/14	-1.44	1.78	0.04	-0.31	0.13	0	1.12
1/1/15	-1.74	-0.05	-2.49	2.09	-0.69	0	3.9
2/1/15	5.91	-0.51	-0.3	-0.99	-0.88	0	-2.43
3/1/15	-1.17	1.44	-0.8	0.05	-0.4	0	1.75
4/1/15	2.71	0.9	2.27	-0.7	-0.48	0	-3.3
5/1/15	0.5	1.05	-1.12	-1.06	-0.41	0	4.29
6/1/15	-2.09	2.06	-0.66	0	-0.27	0	1.99
7/1/15	1.13	-3.29	-3.25	1.76	-0.72	0	3.53
8/1/15	-6.18	1.37	1.27	0.51	0.84	0	-0.68
9/1/15	-3.91	-0.1	-1.02	1.86	0.53	0	3.48
10/1/15	7.32	-2.36	0.34	0.82	-0.49	0	-2.61
11/1/15	-0.3	1.46	-1.99	0.55	-1.39	0	2.07
12/1/15	-1.73	0.84	-1.7	0.76	-0.43	0.01	3.39
1/1/16	-6.32	-1.75	0.76	2.73	2.75	0.01	0.58
2/1/16	-0.5	0.93	-0.21	1.17	1.83	0.02	-2.71
3/1/16	6.92	1.34	0.42	0.66	-0.67	0.02	-2.74
4/1/16	1.91	1.25	2.98	-2.77	0.77	0.01	-3.26
5/1/16	0.45	-0.23	-2.11	0.64	-1.25	0.01	2.93
6/1/16	-1.67	-0.41	-0.73	1.59	2.11	0.02	6.09
7/1/16	4.48	1.38	0.46	0.02	-1.4	0.02	-1.71
8/1/16	0.29	0.56	2.19	-1.38	-0.44	0.02	-3.85
9/1/16	0.9	2.1	-1.23	-0.52	0.13	0.02	1.63
10/1/16	-1.88	-1.09	4.09	-1.07	1.27	0.02	-1.14
11/1/16	1.39	0.61	4.39	-2.02	1.77	0.01	-4.14
12/1/16	2.25	-0.14	2.76	-0.11	0.79	0.03	-1.49
1/1/17	2.72	0.4	-0.53	-0.16	-0.77	0.04	1.76
2/1/17	2.28	-0.71	-1.74	1.14	-0.99	0.04	-1.88
3/1/17	1.48	0.18	-1.81	1.01	-0.54	0.03	0.12
4/1/17	1.86	0.33	-1.39	1.03	-1.01	0.05	0.06
5/1/17	2.2	-0.61	-2.59	2.09	-1.03	0.06	1.11
6/1/17	0.6	1.64	1.74	-1.9	0.05	0.06	0.5
7/1/17	2.51	0.02	1.02	-0.76	0.34	0.07	1.77
8/1/17	0.13	-0.15	-1.34	1.21	-1.3	0.09	1.95
9/1/17	2.3	1.35	1.64	-1.17	1.09	0.09	0.01
10/1/17	1.8	-0.85	-0.9	0.99	-1.92	0.09	3.44
11/1/17	1.93	-0.67	-0.13	1.43	-0.21	0.08	0.46
12/1/17	1.38	0.83	0.11	-0.19	0.81	0.09	-1.07
1/1/18	5.15	-1.21	-1.43	-0.17	-0.54	0.11	3.27
2/1/18	-4.00	0.77	-1.79	1.10	-1.77	0.11	2.43
3/1/18	-1.76	1.20	-0.42	0.40	-0.55	0.12	-1.11
4/1/18	0.93	-0.40	1.76	-1.53	1.02	0.14	-0.49
5/1/18	0.49	1.51	-4.52	1.21	-2.58	0.14	3.11
6/1/18	-0.49	-0.80	-1.39	0.45	-0.08	0.14	-1.38
7/1/18	2.53	-2.48	1.19	-0.10	0.25	0.16	-1.63
8/1/18	0.87	-0.71	-4.19	1.20	-2.61	0.16	3.46
9/1/18	0.32	-1.43	0.30	0.16	0.57	0.15	0.59
10/1/18	-8.19	-2.28	2.44	0.28	2.31	0.19	-2.46
11/1/18	0.88	-0.62	-1.22	0.46	0.08	0.18	-1.58
12/1/18	-7.67	-0.96	0.56	-0.22	0.13	0.19	1.46
1/1/19	7.47	-0.14	-0.71	0.15	-1.19	0.21	-3.85
2/1/19	2.94	0.10	-2.09	0.21	-0.93	0.18	0.36
3/1/19	0.76	-2.08	-3.28	1.32	-1.49	0.19	2.89
4/1/19	3.44	-1.18	-0.26	0.80	-1.09	0.21	-2.81
5/1/19	-5.99	0.54	-1.70	0.45	0.80	0.21	6.68
6/1/19	6.13	-1.99	-0.17	0.43	-0.02	0.18	-1.07
7/1/19	-0.07	-1.13	-0.63	0.64	-0.33	0.19	1.01
8/1/19	-2.55	-1.64	-2.70	1.19	-0.18	0.16	4.32
9/1/19	1.90	0.19	4.15	0.41	2.17	0.18	-4.43
10/1/19	2.56	0.81	-0.53	0.96	-0.72	0.15	-1.23
11/1/19	2.74	0.04	-2.22	-0.09	-1.20	0.12	-1.02
12/1/19	3.05	1.34	0.62	-0.32	0.48	0.14	0.27
1/1/20	-1.20	-2.77	-4.33	0.07	-1.97	0.13	4.15
2/1/20	-8.54	-1.51	-1.07	-1.13	-1.73	0.12	0.19
3/1/20	-13.77	-4.44	-9.30	1.47	-0.88	0.12	6.23
4/1/20	10.68	1.76	-4.62	2.66	-3.00	0.00	0.30
5/1/20	5.30	2.02	-5.10	1.92	-3.18	0.01	1.57
6/1/20	2.76	-0.03	-1.73	-0.09	-0.67	0.01	2.21
7/1/20	4.34	-1.76	-3.64	1.33	-1.21	0.01	6.45
8/1/20	6.84	0.98	-3.88	2.27	-1.52	0.01	0.71
9/1/20	-2.99	1.98	-2.46	0.74	-1.19	0.01	2.40
10/1/20	-2.60	1.43	2.04	-1.04	-0.54	0.01	-1.69
11/1/20	13.34	1.29	4.34	-2.35	0.79	0.01	-10.92
12/1/20	4.82	3.16	-1.20	-0.64	0.14	0.01	-0.09
1/1/21	-0.54	2.93	1.01	-2.43	1.31	0.00	2.64
2/1/21	2.88	1.55	7.46	-1.92	0.61	0.00	-6.52
3/1/21	3.12	-1.32	5.58	1.98	3.12	0.00	-4.83
4/1/21	4.49	-1.22	-1.97	1.39	-2.30	0.00	2.05
5/1/21	1.65	0.24	4.14	0.20	2.29	0.00	0.69
6/1/21	1.01	-1.01	-5.23	0.70	-2.28	0.00	0.17
7/1/21	1.04	-2.16	-1.67	4.20	-0.22	0.00	-0.84
8/1/21	2.42	-0.13	-1.71	0.08	-2.07	0.00	1.30
9/1/21	-3.81	1.05	3.86	-2.21	1.88	0.00	0.79
10/1/21	5.04	-3.31	-1.48	1.51	-1.32	0.00	3.85
11/1/21	-2.64	-2.02	-2.21	4.59	-0.36	0.00	0.83
12/1/21	3.63	-0.91	3.90	2.18	4.98	0.01	-0.73
1/1/22	-5.58	-2.57	11.96	-2.91	8.09	0.00	-3.38
2/1/22	-2.25	2.11	2.89	-1.35	2.27	0.00	0.60
3/1/22	2.16	-1.89	-1.10	0.36	0.79	0.00	3.59
4/1/22	-8.17	0.48	4.95	0.94	5.97	0.00	2.32
5/1/22	0.28	-0.50	6.14	-1.04	4.04	0.03	0.12
6/1/22	-8.69	-0.02	-1.41	1.75	-0.64	0.06	1.27
7/1/22	7.79	-0.41	-5.48	2.37	-5.36	0.08	-3.59
8/1/22	-4.22	0.55	2.49	-2.57	1.34	0.19	3.33
9/1/22	-9.46	-1.49	1.89	-1.21	1.17	0.19	3.36
10/1/22	6.79	-1.35	4.41	0.36	3.27	0.23	2.63
11/1/22	7.25	-0.05	-0.28	2.18	0.98	0.29	-1.86
12/1/22	-4.33	2.42	2.51	-0.47	3.25	0.33	3.80
1/1/23	6.92	0.68	-1.89	-1.02	-3.11	0.35	-9.00
2/1/23	-2.67	0.05	0.69	-0.10	0.48	0.34	0.50
3/1/23	2.27	-3.68	-6.92	4.27	-1.12	0.36	-1.79
4/1/23	1.22	-1.52	1.00	0.56	1.87	0.35	0.80
5/1/23	-1.61	-1.31	-4.76	0.27	-3.99	0.36	0.32
6/1/23	5.60	-1.49	1.04	0.35	-0.52	0.40	-0.14
7/1/23	2.96	1.20	3.43	-1.05	0.08	0.45	-2.46
8/1/23	-3.06	-1.63	-0.36	2.49	0.55	0.45	2.45
9/1/23	-4.79	-0.76	3.30	0.04	1.08	0.43	-0.55
10/1/23	-3.76	-2.62	0.36	1.41	1.52	0.47	1.28
11/1/23	8.63	-0.88	-1.68	-0.77	-2.72	0.44	2.04
12/1/23	4.97	3.59	1.11	-1.40	-1.34	0.43	-3.67
1/1/24	0.09	-3.49	-0.03	1.20	0.02	0.47	4.05
2/1/24	3.75	-2.59	-1.35	0.14	-2.56	0.42	5.02
3/1/24	2.87	-0.43	4.06	-0.16	1.17	0.43	0.34
4/1/24	-4.19	-0.60	1.33	-0.08	0.31	0.47	-1.63
5/1/24	4.05	0.16	-0.95	1.03	-1.63	0.44	1.89
6/1/24	0.83	-3.27	-4.46	1.12	-1.33	0.41	1.55
7/1/24	1.81	4.21	3.36	-2.55	2.02	0.45	-2.83
8/1/24	1.83	-2.23	-1.93	1.25	-0.09	0.48	1.35
9/1/24	1.51	-0.07	-0.72	-0.46	0.32	0.40	-0.64
10/1/24	-2.47	-2.01	1.29	-1.44	0.91	0.39	2.82
11/1/24	4.08	-0.94	0.61	-2.47	-0.54	0.40	2.40
12/1/24	-3.15	-1.21	-2.20	2.07	0.05	0.37	-0.56
1/1/25	3.22	-1.40	0.82	-1.52	-2.70	0.37	0.66
2/1/25	-1.22	-1.38	3.75	0.79	2.60	0.33	-1.59
3/1/25	-4.35	2.43	4.40	0.39	0.35	0.34	-1.57"""
    ff_df = pd.read_csv(StringIO(ff_data_string), sep='\t')

    # Convert 'Date' to datetime, handle potential 2-digit year ambiguity
    # Use infer_datetime_format=True for robustness if format isn't fixed
    ff_df['Date'] = pd.to_datetime(ff_df['Date'], infer_datetime_format=True)

    ff_df = ff_df.set_index('Date')
    # Ensure index is timezone naive
    if ff_df.index.tz is not None:
        ff_df.index = ff_df.index.tz_convert(None)

    ff_df = ff_df.loc[(ff_df.index >= start_date) & (ff_df.index <= end_date)]
    ff_df.columns = ff_df.columns.str.replace('[ .]', '_', regex=True)
    ff_factors = ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'WML']
    # Ensure columns exist before dividing
    existing_ff_factors = [f for f in ff_factors if f in ff_df.columns]
    if existing_ff_factors:
        ff_df[existing_ff_factors] = ff_df[existing_ff_factors] / 100
    else:
        print("Warning: No Fama-French factor columns found in loaded data.")

    ff_df = ff_df.reset_index()
    print("Successfully loaded and processed Fama-French data.")
except Exception as e:
    print(f"FATAL ERROR: Error loading or processing Fama-French data: {e}. Cannot proceed.")
    exit()


# 4. Data Preprocessing and Merging

# Use a left merge from stock_returns_df_filtered to ensure we only keep rows for tickers
# and dates where we successfully got stock data and the ticker is in the final map.
stock_returns_df_filtered = stock_returns_df[stock_returns_df['Ticker'].isin(ticker_esg_group_map_final.keys())].copy()
esg_scores_df_filtered = esg_scores_df[esg_scores_df['Ticker'].isin(ticker_esg_group_map_final.keys())].copy()

# Ensure Date columns are datetime and timezone-naive before merging
stock_returns_df_filtered['Date'] = pd.to_datetime(stock_returns_df_filtered['Date']).dt.tz_localize(None)
esg_scores_df_filtered['Date'] = pd.to_datetime(esg_scores_df_filtered['Date']).dt.tz_localize(None)
ff_df['Date'] = pd.to_datetime(ff_df['Date']).dt.tz_localize(None)


merged_df = pd.merge(stock_returns_df_filtered, esg_scores_df_filtered, on=['Ticker', 'Date'], how='left')

# Merge with Fama-French factors (broadcast on Date)
merged_df = pd.merge(merged_df, ff_df, on='Date', how='left')

# Calculate Excess Return
# Ensure 'RF' column exists after merging
if 'RF' in merged_df.columns:
    merged_df['Excess_Return'] = merged_df['Return'] - merged_df['RF']
    # Drop the original 'Return' and 'RF' columns
    merged_df = merged_df.drop(columns=['Return', 'RF'])
else:
    print("Warning: 'RF' column not found after merging. Cannot calculate Excess_Return.")
    # Cannot proceed with model if Excess_Return is the dependent variable
    exit()


# Add ESG Risk Group column
merged_df['ESG_Risk_Group'] = merged_df['Ticker'].map(ticker_esg_group_map_final)

# Set MultiIndex (Ticker, Date) for panel analysis
# Ensure 'Date' is still datetime and timezone-naive
merged_df['Date'] = pd.to_datetime(merged_df['Date']).dt.tz_localize(None)
merged_df = merged_df.set_index(['Ticker', 'Date']).sort_index()

# Final check if any data remains after merging and filtering
if merged_df.empty:
    print("\nFATAL ERROR: No combined data available for analysis after merging and filtering successful downloads. Cannot proceed.")
    exit()

print("\nMerged DataFrame head:")
print(merged_df.head())
print("\nMerged DataFrame info:")
merged_df.info()
print("\nMissing values before MICE:")
print(merged_df.isnull().sum())


# 5. MICE Imputation

print("\nPerforming MICE imputation...")

numerical_cols_to_impute = ['Excess_Return', 'Total_ESG_Risk_Score', 'Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'WML']
# Filter to columns that actually exist in merged_df
numerical_cols_to_impute = [col for col in numerical_cols_to_impute if col in merged_df.columns]

if merged_df[numerical_cols_to_impute].isnull().sum().sum() > 0:
    try:
        imputer = IterativeImputer(max_iter=10, random_state=0)

        index = merged_df.index
        columns_to_impute = merged_df[numerical_cols_to_impute].columns
        other_cols = merged_df.drop(columns=numerical_cols_to_impute).columns

        # Perform imputation on the numerical subset
        # IterativeImputer works on numpy arrays
        imputed_data = imputer.fit_transform(merged_df[numerical_cols_to_impute].values)

        imputed_df_numerical = pd.DataFrame(imputed_data, columns=columns_to_impute, index=index)

        # Recombine with non-imputed columns (like ESG_Risk_Group)
        df_imputed = imputed_df_numerical.join(merged_df[other_cols])

        print("MICE imputation complete.")
        print("\nMissing values after MICE:")
        print(df_imputed.isnull().sum())

    except Exception as e:
        print(f"Error during MICE imputation: {e}")
        print("Proceeding with analysis on data with original NaNs (might cause errors in regressions).")
        df_imputed = merged_df.copy()
else:
    print("No missing numerical values found in specified columns. Skipping MICE imputation.")
    df_imputed = merged_df.copy()

# Ensure ESG_Risk_Group is still category, especially if it was dropped and re-added
if 'ESG_Risk_Group' in df_imputed.columns:
    df_imputed['ESG_Risk_Group'] = df_imputed['ESG_Risk_Group'].astype('category')


# 6. Analysis by ESG Group

esg_groups = ['Low', 'Medium', 'High']
results = {}

# Check which groups actually have tickers with data in the final filtered data
# Use the index to get the available tickers from the imputed data
available_tickers_in_imputed_data = df_imputed.index.get_level_values('Ticker').unique()
available_groups = df_imputed['ESG_Risk_Group'].dropna().unique() # Get groups present in the imputed data

print(f"\nTickers with data after MICE: {available_tickers_in_imputed_data.tolist()}")
print(f"ESG Groups with data after MICE: {available_groups.tolist()}")


for group in esg_groups:
    # Check if the group exists in the data *after* all filtering and MICE
    if group not in available_groups:
        print(f"\n--- Skipping {group} ESG Risk Group (No data available after filtering/imputation) ---")
        continue

    print(f"\n--- Analyzing {group} ESG Risk Group ---")

    group_df = df_imputed[df_imputed['ESG_Risk_Group'] == group].copy()

    # Ensure the DataFrame is a MultiIndex DataFrame suitable for PanelOLS
    # This should already be the case from previous steps, but add a check
    if not isinstance(group_df.index, pd.MultiIndex) or 'Ticker' not in group_df.index.names or 'Date' not in group_df.index.names:
        print(f"Error: Could not retain MultiIndex for {group} group. Skipping analysis for this group.")
        continue

    # Define dependent and independent variables from the group_df
    dependent_var_col = 'Excess_Return'
    independent_vars_cols = ['Total_ESG_Risk_Score', 'Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'WML']
    # Filter independent_vars_cols to only include columns that exist in the group_df
    independent_vars_cols = [col for col in independent_vars_cols if col in group_df.columns]

    if dependent_var_col not in group_df.columns:
        print(f"Error: Dependent variable '{dependent_var_col}' not found in {group} group data. Skipping.")
        continue
    if not independent_vars_cols:
        print(f"Error: No independent variables found in {group} group data. Skipping.")
        continue


    # Prepare data for regression: select columns and handle NaNs again as MICE might not fill all (e.g., if a row was all NaN)
    # Or if the independent variables list changed.
    analysis_cols = [dependent_var_col] + independent_vars_cols
    analysis_data_for_reg = group_df[analysis_cols].dropna(how='any') # Drop rows with any NaN in regression columns

    if analysis_data_for_reg.empty:
         print(f"No complete data rows for regression analysis in {group} group after handling NaNs.")
         continue

    dependent_var_clean = analysis_data_for_reg[dependent_var_col]
    independent_vars_clean = analysis_data_for_reg[independent_vars_cols]

    # Ensure the cleaned data retains the MultiIndex for PanelOLS
    panel_data_clean = analysis_data_for_reg # It should already have the MultiIndex


    # 7. VIF Analysis
    print(f"\nCalculating VIF for {group} group:")
    # Check for sufficient data points relative to variables for VIF
    # Need at least k+1 data points for OLS, k predictors + constant. VIF needs slightly more stability.
    # A common heuristic is N > k+1, ideally N >> k.
    if independent_vars_clean.shape[0] < independent_vars_clean.shape[1] + 2:
         print(f"Insufficient data points ({independent_vars_clean.shape[0]}) relative to predictors ({independent_vars_clean.shape[1]}) to reliably calculate VIF for {group} group.")
    else:
        try:
            # Add a constant for VIF calculation
            independent_vars_vif = sm.add_constant(independent_vars_clean, has_constant='add')
            vif_data = pd.DataFrame()
            vif_data['Variable'] = independent_vars_vif.columns
            vif_data['VIF'] = [variance_inflation_factor(independent_vars_vif.values, i)
                               for i in range(independent_vars_vif.shape[1])]
            print(vif_data)
            results[f'{group}_VIF'] = vif_data
        except Exception as e:
            print(f"Could not calculate VIF for {group} group: {e}")


    # 8. Regression Analysis (Pooled OLS)
    print(f"\nPerforming Pooled OLS Regression for {group} group:")
    # Check for sufficient data points for OLS
    if dependent_var_clean.shape[0] <= independent_vars_clean.shape[1]: # Need at least n > k
        print(f"Insufficient data points ({dependent_var_clean.shape[0]}) for Pooled OLS regression for {group} group (Need > {independent_vars_clean.shape[1]} observations).")
    else:
        try:
            independent_vars_ols = sm.add_constant(independent_vars_clean, has_constant='add')
            model_ols = sm.OLS(dependent_var_clean, independent_vars_ols)
            results_ols = model_ols.fit()
            print(results_ols.summary())
            results[f'{group}_Pooled_OLS'] = results_ols
        except Exception as e:
            print(f"Could not perform Pooled OLS for {group} group: {e}")


    # 9. Panel Data Regression
    print(f"\nPerforming Panel Regression (Fixed Effects) for {group} group:")
    n_entities = panel_data_clean.index.get_level_values('Ticker').nunique()
    n_time_periods_overall = panel_data_clean.index.get_level_values('Date').nunique()
    n_observations = panel_data_clean.shape[0]
    n_predictors = independent_vars_clean.shape[1] # Number of independent variables

    # Check if there's at least one entity with more than one time period for PanelOLS
    entity_counts = panel_data_clean.index.get_level_values('Ticker').value_counts()
    has_multi_period_entity = (entity_counts > 1).any()


    # PanelOLS requires at least 2 entities *with data* or at least one entity with >1 time period *with data*
    # And generally N*T > k+1. The effective number of observations is n_observations.
    if n_entities < 1:
         print(f"Insufficient entities (< 1) with data for Panel Regressions for {group} group.")
    elif n_observations <= n_predictors: # Check degrees of freedom
         print(f"Insufficient total observations ({n_observations}) relative to predictors ({n_predictors}) for Panel Regressions for {group} group.")
    elif n_entities < 2 and not has_multi_period_entity:
         print(f"Insufficient entities (< 2) AND no single entity with multi-period data for basic PanelOLS structure for {group} group (Entities: {n_entities}, Has Multi-period Entity: {has_multi_period_entity}).")
         # If there is only one entity *but* it has multiple periods, PanelOLS still works.
         # If there are multiple entities *but* each has only one period, PanelOLS Fixed Effects won't work correctly (absorbs the single point). Random Effects might still be possible but unusual.
         # Let's allow it if there's at least one multi-period entity or at least 2 entities.
         if n_entities >= 2 or has_multi_period_entity:
              pass # Continue to attempt PanelOLS
         else:
              print(f"Skipping Panel regressions for {group} group due to insufficient panel structure.")
              continue # Skip panel regressions for this group
    else: # Sufficient structure to attempt PanelOLS
        try:
            # Fixed Effects model
            # PanelOLS adds the constant automatically when entity_effects=True
            model_fe = PanelOLS(dependent_var_clean, independent_vars_clean, entity_effects=True)
            results_fe = model_fe.fit()
            print("Fixed Effects Results:")
            print(results_fe)
            results[f'{group}_Fixed_Effects'] = results_fe
        except Exception as e:
            print(f"Could not perform Fixed Effects regression for {group} group: {e}")

        # Only attempt Random Effects if Fixed Effects was attempted (implies sufficient basic structure)
        # Random effects also requires at least 2 entities or one multi-period entity
        if n_entities >= 2 or has_multi_period_entity:
            print(f"\nPerforming Panel Regression (Random Effects) for {group} group:")
            try:
                # Random Effects model
                # PanelOLS adds the constant automatically when random_effects=True
                model_re = PanelOLS(dependent_var_clean, independent_vars_clean, random_effects=True)
                results_re = model_re.fit()
                print("Random Effects Results:")
                print(results_re)
                results[f'{group}_Random_Effects'] = results_re
            except Exception as e:
                print(f"Could not perform Random Effects regression for {group} group: {e}")
        else:
             print(f"\nSkipping Random Effects for {group} group (requires at least 2 entities or one multi-period entity with data).")


print("\n--- Analysis Complete ---")
print(f"\nAnalysis was attempted for groups: {list(esg_groups)}. Successful groups with data for analysis: {list(available_groups)}")
print("\nResults stored in 'results' dictionary.")

Attempting to download data for tickers: ['AAPL', '009150.KS', 'SONY', '034220.KS', '601138.SS', 'ENR.DE', 'PHG', '002241.SZ', 'STM', 'GRMN', 'AMZN', '000100.SZ']

Analysis period: 2019-11-01 to 2025-04-01

Performing quick test download for first ticker...
Quick test download successful for AAPL. Found 20 data points.
------------------------------

Processing stock data for AAPL...
Successfully processed monthly stock returns for AAPL
Processing stock data for 009150.KS...
Successfully processed monthly stock returns for 009150.KS
Processing stock data for SONY...
Successfully processed monthly stock returns for SONY
Processing stock data for 034220.KS...
Successfully processed monthly stock returns for 034220.KS
Processing stock data for 601138.SS...
Successfully processed monthly stock returns for 601138.SS
Processing stock data for ENR.DE...
Successfully processed monthly stock returns for ENR.DE
Processing stock data for PHG...
Successfully processed monthly stock returns for PHG

KeyError: 'Return'

### GEM dataset Imputation and calibration

In [2]:
import pandas as pd
import numpy as np

# --- Configuration ---
file_path = '/content/GEM2024_2014.csv'
imputation_method = 'median' # or 'mean' or 'mode'
output_file_path = '/content/GEM2024_2014_imputed.csv' # Define the output file path

# List of columns that should NOT be imputed (identifiers)
identifier_columns = ['country', 'year']

# --- Load the dataset ---
try:
    df = pd.read_csv(file_path, na_values='')

    print(f"Successfully loaded data from {file_path}")
    print("\nInitial data information:")
    df.info()

    # --- Check for missing values before imputation ---
    print("\nMissing values per column before imputation:")
    print(df.isnull().sum())

    # --- Identify columns for imputation ---
    numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
    cols_to_impute = [col for col in numerical_cols if col not in identifier_columns]

    print(f"\nColumns identified for {imputation_method} imputation: {cols_to_impute}")

    # --- Perform Imputation ---
    print(f"\nPerforming {imputation_method} imputation...")

    for col in cols_to_impute:
        if imputation_method == 'median':
            fill_value = df[col].median()
        elif imputation_method == 'mean':
            fill_value = df[col].mean()
        elif imputation_method == 'mode':
            mode_values = df[col].mode()
            if not mode_values.empty:
                 fill_value = mode_values[0]
            else:
                 print(f"Warning: No mode found for column {col}. Using median instead.")
                 fill_value = df[col].median() # Fallback if mode is problematic
        else:
            print(f"Error: Unknown imputation method '{imputation_method}'.")
            # Decide how to handle this error - maybe skip imputation for this column?
            # For simplicity, we'll just continue without filling this specific column
            continue # Skip to the next column

        # Fill missing values in the column
        if pd.notna(fill_value):
            df[col].fillna(fill_value, inplace=True)
            # print(f"Filled missing values in '{col}' with {fill_value:.2f}") # Optional: detailed log

    # --- Verify Imputation ---
    print("\nMissing values per column after imputation:")
    print(df.isnull().sum())

    # --- Display results ---
    print("\nData head after imputation:")
    print(df.head())

    print("\nData info after imputation:")
    df.info()

    # --- Save the imputed data to a new CSV file ---
    # These lines are now uncommented to save the file
    df.to_csv(output_file_path, index=False)
    print(f"\nImputed data saved to {output_file_path}")

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    print("Please make sure you have saved the data into a CSV file with this name.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded data from /content/GEM2024_2014.csv

Initial data information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582 entries, 0 to 581
Data columns (total 17 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   country                                         582 non-null    object 
 1   year                                            582 non-null    int64  
 2   Perceived_opportunities                         582 non-null    float64
 3   Perceived_capabilities                          582 non-null    float64
 4   Fear_of_failure_rate                            582 non-null    float64
 5   Entrepreneurial_intentions                      582 non-null    float64
 6   Total_early_stage_Entrepreneurial_Activity_TEA  582 non-null    float64
 7   Established_Business_Ownership                  582 non-null    float64
 8   Entrepreneurial_Employee_Activity    

<ipython-input-2-b6303277b039>:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(fill_value, inplace=True)


In [4]:
!pip install pandas openpyxl
!pip install statsmodels
!pip install linearmodels

#python calibrate_data.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.7 MB/s eta 0:00:00


### Calibrate GEM dataset

#### *Explanation:
Import Libraries: Imports pandas for data manipulation and numpy for detecting numeric columns.
#### Configuration: Sets the paths for input/output files and the percentiles for calibration (5, 50, 95).
#### calibrate_percentile Function: This function takes a single data value and the calculated 5th, 50th, and 95th percentiles for its variable. It implements the piecewise linear calibration logic:
#### Values at or below the 5th percentile get a score of 0.05.
#### Values at or above the 95th percentile get a score of 0.95.
#### Values between the 5th and 50th percentiles are linearly interpolated between 0.05 and 0.5.
#### Values between the 50th and 95th percentiles are linearly interpolated between 0.5 and 0.95.
#### Includes basic checks to prevent division by zero if percentiles happen to be equal (which can occur with variables having very limited unique values).
#### Load Data: Reads your specified CSV file into a pandas DataFrame. Includes error handling for FileNotFoundError.
#### Identify Columns: Separates the columns into identifiers, those to be excluded (like the constant one), and the remaining numeric columns that will be calibrated.
#### Perform Calibration:
#### Creates a new DataFrame calibrated_df starting with just the identifier columns (country, year).
#### Loops through each col in cols_to_calibrate.
#### Calculates the 5th, 50th, and 95th percentiles (p5, p50, p95) for the entire column using df[col].quantile().
#### Applies the calibrate_percentile function to each value in the current column df[col].
#### Stores the resulting fuzzy scores in a new column in calibrated_df, prefixed with f_.
#### Save Data:
#### Saves the calibrated_df to a new CSV file using to_csv(). index=False prevents writing the DataFrame index as a column.
#### Saves the calibrated_df to a new XLSX file using to_excel(). This requires the openpyxl library. Error handling is included to remind you to install it if needed.
#### To use this code:
#### Save the code: Copy the code block and save it as a Python file (e.g., calibrate_data.py).
#### Ensure the input file: Make sure your imputed data file (GEM2024_2014_imputed.csv) is in the location specified by input_csv_path. If you run this code in a Colab or Jupyter environment and uploaded the file, /content/GEM2024_2014_imputed.csv might be correct. If running locally, update the path.
#### Install pandas and openpyxl: If you don't have them installed, open your terminal or command prompt and run:
#### pip install pandas openpyxl
#### Run the script: Open your terminal or command prompt, navigate to the directory where you saved the Python file, and run:
#### python calibrate_data.py

In [1]:
import pandas as pd
import numpy as np

# --- Configuration ---
input_csv_path = '/content/GEM2024_2014_imputed.csv'
output_csv_path = 'calibrated_data.csv'
output_excel_path = 'calibrated_data.xlsx'
percentiles = [5, 50, 95] # Percentiles for calibration anchors

# Define the calibration function (piecewise linear based on percentiles)
def calibrate_percentile(value, p5, p50, p95):
    """Calibrates a single value based on 5th, 50th, and 95th percentiles."""
    # Handle edge cases where percentiles might be equal (e.g., low variation)
    if p5 == p95:
        # Variable is constant, return 0.5 (or could exclude, but calibrating to 0.5 is also an option)
        # Based on previous output, we will exclude this variable entirely later.
        return 0.5 # This case shouldn't be reached if constant variables are excluded from cols_to_calibrate

    if value <= p5:
        return 0.05
    elif value >= p95:
        return 0.95
    elif value < p50:
        # Linear interpolation between p5 (0.05) and p50 (0.5)
        if p50 == p5: # Handle division by zero if p50 equals p5
            return 0.05 # If value is less than p50 (which equals p5), it must be <= p5
        return 0.05 + (value - p5) / (p50 - p5) * (0.5 - 0.05)
    else: # value >= p50
        # Linear interpolation between p50 (0.5) and p95 (0.95)
        if p95 == p50: # Handle division by zero if p95 equals p50
             return 0.95 # If value is >= p50 (which equals p95), it must be >= p95
        return 0.5 + (value - p50) / (p95 - p50) * (0.95 - 0.5)

# --- Load Data ---
try:
    df = pd.read_csv(input_csv_path)
    print(f"Successfully loaded data from {input_csv_path}. Shape: {df.shape}")
    print("Original columns:", df.columns.tolist())
except FileNotFoundError:
    print(f"Error: File not found at {input_csv_path}")
    exit()
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")
    exit()

# --- Identify Columns to Calibrate ---
# Exclude non-numeric and obviously constant columns
identifier_cols = ['country', 'year']
constant_or_excluded_cols = ['Female_Male_Opportunity_Driven_TEA'] # Identified as constant

numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

# Variables to calibrate are numeric columns minus identifiers and excluded ones
cols_to_calibrate = [col for col in numeric_cols if col not in identifier_cols and col not in constant_or_excluded_cols]

print("\nColumns to calibrate:", cols_to_calibrate)
print("Columns to keep as is (identifiers):", identifier_cols)
print("Columns excluded:", constant_or_excluded_cols)


# --- Perform Calibration ---
calibrated_df = df[identifier_cols].copy() # Start with identifier columns

for col in cols_to_calibrate:
    if col in df.columns:
        # Calculate percentiles across the entire dataset for this variable
        p5 = df[col].quantile(percentiles[0] / 100)
        p50 = df[col].quantile(percentiles[1] / 100)
        p95 = df[col].quantile(percentiles[2] / 100)

        # Print percentiles for transparency (optional)
        # print(f"Calibrating '{col}': 5th={p5:.2f}, 50th={p50:.2f}, 95th={p95:.2f}")

        # Apply the calibration function to the column
        # Create a new column with 'f_' prefix for fuzzy scores
        calibrated_col_name = f'f_{col}'
        calibrated_df[calibrated_col_name] = df[col].apply(
            lambda x: calibrate_percentile(x, p5, p50, p95)
        )
    else:
        print(f"Warning: Column '{col}' not found in the dataset. Skipping calibration.")

# --- Save Calibrated Data ---

# 1) Save to CSV
try:
    calibrated_df.to_csv(output_csv_path, index=False)
    print(f"\nCalibrated data saved successfully to CSV: {output_csv_path}")
except Exception as e:
    print(f"Error saving to CSV: {e}")


# 2) Save to XLSX
try:
    # You might need to install openpyxl: pip install openpyxl
    calibrated_df.to_excel(output_excel_path, index=False)
    print(f"Calibrated data saved successfully to XLSX: {output_excel_path}")
except ImportError:
    print("\nError: 'openpyxl' library not found.")
    print("Please install it to save to Excel (.xlsx) format: pip install openpyxl")
except Exception as e:
    print(f"Error saving to XLSX: {e}")

print("\nCalibration process finished.")

Error: File not found at /content/GEM2024_2014_imputed.csv


NameError: name 'df' is not defined

In [2]:
!pip install pyfca
!pip install openpyxl
!pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

# --- Configuration ---
input_csv_file = '/content/PLS_SEM_SE_Pre.csv'
output_excel_file = 'PLS_SEM_Calibrated_Output.xlsx'

# List of columns to calibrate
columns_to_calibrate = [
    'Perceived_opportunities',
    'Perceived_capabilities',
    'Fear_failure_rate',
    'Entrepreneurial_intentions',
    'Entrepreneurial_TEA',
    'Established_Business_Ownership',
    'Entrepreneurial_Employee_Activity',
    'Motivational_Index',
    'Female_Male_TEA',
    'Female_Male_Opportunity_Driven_TEA',
    'High_Job_Creation_Expectation',
    'Innovation',
    'Business_Services_Sector',
    'High_Status_Successful_Entrepreneurs',
    'Entrepreneurship_Good_Career_Choice'
]

# --- Manual Calibration Function (Min=0, Mean=0.5, Max=1 Membership) ---
def calibrate_min_mean_max(series):
    """
    Calibrates a pandas Series to a fuzzy set using min, mean, and max
    as anchor points for 0, 0.5, and 1 membership respectively.
    Handles edge cases where min, mean, or max are identical.
    """
    # Handle potential NaN values
    series_clean = series.dropna()

    if series_clean.empty:
        return pd.Series(np.nan, index=series.index) # Return NaNs if no data

    min_val = series_clean.min()
    mean_val = series_clean.mean()
    max_val = series_clean.max()

    # Initialize calibrated series with NaNs to preserve original index and NaNs
    calibrated_series = pd.Series(np.nan, index=series.index)

    # Handle edge case where all values are the same
    if min_val == max_val:
        # Membership is 0.5 for all non-NaN values if min==mean==max
        calibrated_series[series.notna()] = 0.5
        return calibrated_series

    # Handle cases where min, mean, or max overlap
    # Use slightly adjusted points if mean coincides with min or max, but not all three
    c1, c2, c3 = min_val, mean_val, max_val

    # Apply calibration logic element-wise, handling NaNs implicitly
    # We'll use boolean indexing and vectorized operations for efficiency

    # Values less than or equal to c1 (min) get membership 0
    calibrated_series[series <= c1] = 0.0

    # Values greater than or equal to c3 (max) get membership 1
    calibrated_series[series >= c3] = 1.0

    # Values between c1 and c3 require interpolation
    mask_between = (series > c1) & (series < c3)

    # Linear interpolation between c1 (0 membership) and c2 (0.5 membership)
    if c1 < c2: # Avoid division by zero if c1 == c2
        mask_lower_half = (series > c1) & (series <= c2)
        calibrated_series[mask_lower_half] = 0.5 * (series[mask_lower_half] - c1) / (c2 - c1)
    elif c1 == c2 and c2 < c3: # Case like 0, 0, 100
        # If values are > c1 (which is also c2), they are in the upper half
        mask_between &= (series > c1) # All values > c1 go into upper half logic below

    # Linear interpolation between c2 (0.5 membership) and c3 (1 membership)
    if c2 < c3: # Avoid division by zero if c2 == c3
         mask_upper_half = (series > c2) & (series < c3) # Only strictly > c2
         calibrated_series[mask_upper_half] = 0.5 + 0.5 * (series[mask_upper_half] - c2) / (c3 - c2)
    elif c1 < c2 and c2 == c3: # Case like 0, 100, 100
        # If values are < c2 (which is also c3), they are in the lower half
        mask_between &= (series < c2) # All values < c2 go into lower half logic above


    # Handle the point exactly at c2 if it wasn't min or max
    if c1 < c2 < c3:
        calibrated_series[series == c2] = 0.5


    # Note: Cases exactly at c1 or c3 are handled by the >= c3 and <= c1 conditions.
    # If c1=c2<c3 or c1<c2=c3, the interpolation logic covers the transitions correctly
    # for values not equal to the problematic point. Values exactly at the problematic point
    # (like c1 if c1=c2) might need explicit handling if the previous logic missed them.
    # Let's ensure points at the boundaries are covered explicitly if needed, though
    # the >= and <= masks should handle c1 and c3 correctly.

    # The logic `series <= c1` and `series >= c3` correctly assigns 0 and 1
    # The intermediate values are handled by the linear interpolation.
    # The case c1=c2=c3 returns 0.5 for non-NaNs.
    # If c1=c2 < c3, values > c1 interpolate from 0 to 1 between c1 and c3. Our code uses 0 to 0.5 c1-c2 and 0.5 to 1 c2-c3. If c1=c2, (series-c1)/(c2-c1) is division by zero. This needs fixing.

    # Let's refine the interpolation logic for edge cases where endpoints coincide
    mask = series.notna() # Only process non-NaN values
    values = series[mask]
    calibrated_values = pd.Series(np.nan, index=values.index) # To store results

    if min_val == max_val:
        calibrated_values = 0.5
    elif min_val == mean_val: # Case like 0, 0, 100 -> linear from 0 (at min) to 1 (at max)
        calibrated_values[values <= min_val] = 0.0
        calibrated_values[values > min_val] = (values[values > min_val] - min_val) / (max_val - min_val)
    elif mean_val == max_val: # Case like 0, 100, 100 -> linear from 0 (at min) to 1 (at max)
        calibrated_values[values < max_val] = (values[values < max_val] - min_val) / (max_val - min_val)
        calibrated_values[values >= max_val] = 1.0
    else: # Standard case: min < mean < max
        calibrated_values[values <= min_val] = 0.0
        calibrated_values[(values > min_val) & (values < mean_val)] = 0.5 * (values[(values > min_val) & (values < mean_val)] - min_val) / (mean_val - min_val)
        calibrated_values[values == mean_val] = 0.5
        calibrated_values[(values > mean_val) & (values < max_val)] = 0.5 + 0.5 * (values[(values > mean_val) & (values < max_val)] - mean_val) / (max_val - mean_val)
        calibrated_values[values >= max_val] = 1.0

    calibrated_series[mask] = calibrated_values # Assign back to the original series structure
    return calibrated_series


# --- Load Data ---
try:
    # Use keep_default_na=True and na_values='' explicitly if needed, pandas usually handles common NaNs
    df = pd.read_csv(input_csv_file)
    print(f"Successfully loaded data from '{input_csv_file}'.")
except FileNotFoundError:
    print(f"Error: Input file '{input_csv_file}' not found.")
    exit()
except Exception as e:
    print(f"Error loading CSV: {e}")
    exit()


# --- Calibration Process ---
print("Starting calibration...")

for col_name in columns_to_calibrate:
    if col_name in df.columns:
        # Ensure the column is numeric. Attempt conversion if necessary.
        try:
            # Attempt to convert to numeric, coercing errors to NaN
            df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
        except Exception as e:
            print(f" - Warning: Could not convert column '{col_name}' to numeric: {e}. Skipping calibration.")
            df[f'{col_name}_calibrated'] = np.nan
            continue # Skip to the next column

        # Perform calibration using the custom function
        calibrated_data = calibrate_min_mean_max(df[col_name])

        # Store the calibrated data in a new column
        df[f'{col_name}_calibrated'] = calibrated_data

        # Provide feedback on calibration points used
        col_data_clean = df[col_name].dropna()
        if not col_data_clean.empty:
             min_val = col_data_clean.min()
             mean_val = col_data_clean.mean()
             max_val = col_data_clean.max()
             print(f" - Calibrated '{col_name}' (Min={min_val:.2f}, Mean={mean_val:.2f}, Max={max_val:.2f}).")
        else:
             print(f" - Calibrated '{col_name}' (No valid data for points).")


    else:
        print(f" - Warning: Column '{col_name}' not found in the dataset. Skipping.")
        df[f'{col_name}_calibrated'] = np.nan # Add NaN column if skipped


print("Calibration complete.")

# --- Save Calibrated Data to Excel ---
try:
    # Use ExcelWriter to specify the engine for potentially larger files
    with pd.ExcelWriter(output_excel_file, engine='openpyxl') as writer:
        df.to_excel(writer, index=False)
    print(f"Successfully saved calibrated data to '{output_excel_file}'.")
except Exception as e:
    print(f"Error saving to Excel: {e}")

Successfully loaded data from '/content/PLS_SEM_SE_Pre.csv'.
Starting calibration...
 - Calibrated 'Perceived_opportunities' (Min=7.27, Mean=48.64, Max=95.38).
 - Calibrated 'Perceived_capabilities' (Min=10.05, Mean=54.67, Max=92.63).
 - Calibrated 'Fear_failure_rate' (Min=7.14, Mean=39.55, Max=75.42).
 - Calibrated 'Entrepreneurial_intentions' (Min=2.12, Mean=22.81, Max=83.00).
 - Calibrated 'Entrepreneurial_TEA' (Min=1.56, Mean=13.12, Max=49.60).
 - Calibrated 'Established_Business_Ownership' (Min=1.25, Mean=7.86, Max=35.94).
 - Calibrated 'Entrepreneurial_Employee_Activity' (Min=0.00, Mean=2.52, Max=11.47).
 - Calibrated 'Motivational_Index' (Min=0.00, Mean=1.43, Max=19.50).
 - Calibrated 'Female_Male_TEA' (Min=0.25, Mean=0.72, Max=1.34).
 - Calibrated 'Female_Male_Opportunity_Driven_TEA' (Min=0.00, Mean=0.48, Max=1.18).
 - Calibrated 'High_Job_Creation_Expectation' (Min=0.50, Mean=22.40, Max=76.74).
 - Calibrated 'Innovation' (Min=0.00, Mean=13.34, Max=58.70).
 - Calibrated 'Busine

In [3]:
import pandas as pd
import numpy as np

# --- Configuration ---
input_csv_file = '/Users/henryefeonomakpo/Downloads/1_SE_Prep/PLS_SEM_Calibrated_Output.csv' # Use the CSV with the calibrated columns from your PLS-SEM runs
output_excel_file = 'FSQCA_Calibrated_Data.xlsx'

# List of columns to calibrate for fsQCA.
# Use the *original* raw data columns here, NOT the _calibrated columns from PLS-SEM.
# The calibration points (min, mean, max) will be calculated from these original columns.
columns_to_calibrate = [
    'Perceived_opportunities',
    'Perceived_capabilities',
    'Fear_failure_rate',
    'Entrepreneurial_intentions',
    'Entrepreneurial_TEA',
    'Established_Business_Ownership',
    'Entrepreneurial_Employee_Activity',
    'Motivational_Index',
    'Female_Male_TEA',
    'Female_Male_Opportunity_Driven_TEA',
    'High_Job_Creation_Expectation',
    'Innovation',
    'Business_Services_Sector',
    'High_Status_Successful_Entrepreneurs',
    'Entrepreneurship_Good_Career_Choice'
    # Include any other relevant numerical columns from your dataset
]

# --- Manual Calibration Function (Min=0, Mean=0.5, Max=1 Membership) ---
# This function correctly implements the 3-anchor calibration with interpolation
def calibrate_min_mean_max(series, col_name):
    """
    Calibrates a pandas Series to a fuzzy set using min, mean, and max
    as anchor points for 0, 0.5, and 1 membership respectively.
    Uses piecewise linear interpolation and handles edge cases.
    Prints calibration points used.
    """
    # Handle potential NaN values
    series_clean = series.dropna()

    if series_clean.empty:
        print(f"    No valid data found for calibration of '{col_name}'.")
        return pd.Series(np.nan, index=series.index) # Return NaNs if no data

    min_val = series_clean.min()
    mean_val = series_clean.mean()
    max_val = series_clean.max()

    # Print calibration points for the user
    print(f"    Calibrating '{col_name}' with points (0, 0.5, 1): ({min_val:.2f}, {mean_val:.2f}, {max_val:.2f})")

    # Initialize calibrated series with NaNs to preserve original index and NaNs
    calibrated_series = pd.Series(np.nan, index=series.index)
    mask = pd.notna(series) # Mask for non-NaN original values
    values = series[mask] # Non-NaN values

    # Handle edge case: all values identical
    if min_val == max_val:
        calibrated_values = np.full(values.shape, 0.5) # Membership is 0.5 for all non-NaNs
    # Handle edge case: min == mean but < max
    elif min_val == mean_val and mean_val < max_val:
        # Linear interpolation from 0 (at min/mean) to 1 (at max)
        calibrated_values = (values - min_val) / (max_val - min_val)
    # Handle edge case: mean == max but > min
    elif min_val < mean_val and mean_val == max_val:
        # Linear interpolation from 0 (at min) to 1 (at mean/max)
        calibrated_values = (values - min_val) / (mean_val - min_val)
    # Standard case: min < mean < max (piecewise linear)
    else:
        calibrated_values = np.empty(values.shape)
        # Lower half (0 to 0.5 between min and mean)
        mask_lower_half = values <= mean_val
        calibrated_values[mask_lower_half] = 0.5 * (values[mask_lower_half] - min_val) / (mean_val - min_val)
        # Upper half (0.5 to 1 between mean and max)
        mask_upper_half = values > mean_val
        calibrated_values[mask_upper_half] = 0.5 + 0.5 * (values[mask_upper_half] - mean_val) / (max_val - mean_val)

    # Ensure memberships are strictly between 0 and 1 (numerical precision)
    calibrated_values = np.clip(calibrated_values, 0.0, 1.0)

    # Assign calibrated values back, preserving original NaNs
    calibrated_series[mask] = calibrated_values

    return calibrated_series


# --- Load Data ---
try:
    # Use keep_default_na=True and na_values='' explicitly if needed, pandas usually handles common NaNs
    df = pd.read_csv(input_csv_file)
    print(f"Successfully loaded data from '{input_csv_file}'.")
except FileNotFoundError:
    print(f"Error: Input file '{input_csv_file}' not found.")
    exit()
except Exception as e:
    print(f"Error loading CSV: {e}")
    exit()


# --- Calibration Process ---
print("Starting calibration...")

# Create new columns for calibrated data
for col_name in columns_to_calibrate:
    if col_name in df.columns:
        # Ensure the column is numeric. Attempt conversion if necessary.
        try:
            # Attempt to convert to numeric, coercing errors to NaN
            df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
        except Exception as e:
            print(f" - Warning: Could not convert column '{col_name}' to numeric: {e}. Skipping calibration.")
            df[f'f_{col_name}'] = np.nan # Use 'f_' prefix for fuzzy set columns
            continue # Skip to the next column

        # Perform calibration using the custom function
        calibrated_data = calibrate_min_mean_max(df[col_name], col_name)

        # Store the calibrated data in a new column (use 'f_' prefix for fuzzy set)
        df[f'f_{col_name}'] = calibrated_data

    else:
        print(f" - Warning: Column '{col_name}' not found in the dataset. Skipping.")
        df[f'f_{col_name}'] = np.nan # Add NaN column if skipped


print("Calibration complete.")
print("Added fuzzy set columns (e.g., 'f_Perceived_opportunities', 'f_Entrepreneurial_intentions') to the DataFrame.")


# --- Save Calibrated Data to Excel ---
try:
    # Use ExcelWriter to specify the engine for potentially larger files
    with pd.ExcelWriter(output_excel_file, engine='openpyxl') as writer:
        df.to_excel(writer, index=False)
    print(f"Successfully saved calibrated data to '{output_excel_file}'.")
except Exception as e:
    print(f"Error saving to Excel: {e}")

Error: Input file '/Users/henryefeonomakpo/Downloads/1_SE_Prep/PLS_SEM_Calibrated_Output.csv' not found.
Starting calibration...
    Calibrating 'Perceived_opportunities' with points (0, 0.5, 1): (7.27, 48.64, 95.38)
    Calibrating 'Perceived_capabilities' with points (0, 0.5, 1): (10.05, 54.67, 92.63)
    Calibrating 'Fear_failure_rate' with points (0, 0.5, 1): (7.14, 39.55, 75.42)
    Calibrating 'Entrepreneurial_intentions' with points (0, 0.5, 1): (2.12, 22.81, 83.00)
    Calibrating 'Entrepreneurial_TEA' with points (0, 0.5, 1): (1.56, 13.12, 49.60)
    Calibrating 'Established_Business_Ownership' with points (0, 0.5, 1): (1.25, 7.86, 35.94)
    Calibrating 'Entrepreneurial_Employee_Activity' with points (0, 0.5, 1): (0.00, 2.52, 11.47)
    Calibrating 'Motivational_Index' with points (0, 0.5, 1): (0.00, 1.43, 19.50)
    Calibrating 'Female_Male_TEA' with points (0, 0.5, 1): (0.25, 0.72, 1.34)
    Calibrating 'Female_Male_Opportunity_Driven_TEA' with points (0, 0.5, 1): (0.00, 0.

### SE -ESG Panel Regression

In [2]:
# ==============================================================================
#                               Complete Data Analysis Script
#           Merging, Imputation, VIF, Panel Regression, ML Models, Visualization
# ==============================================================================

# --- 1. Setup and Imports ---

print("--- Starting Setup and Imports ---")

# Install necessary libraries
# Use %pip instead of !pip in notebook environments for better integration
try:
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    from linearmodels.panel import PanelOLS
    from sklearn.experimental import enable_iterative_imputer # Required for IterativeImputer
    from sklearn.impute import SimpleImputer, IterativeImputer
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    import xgboost as xgb
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    import matplotlib.pyplot as plt # Import matplotlib.pyplot as plt
    import seaborn as sns # Import seaborn as sns
    import warnings
    print("Required libraries already installed or successfully imported.")
except ImportError:
    print("Installing required libraries...")
    %pip install pandas openpyxl statsmodels linearmodels scikit-learn xgboost matplotlib seaborn
    # Re-import after installing
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    from linearmodels.panel import PanelOLS
    from sklearn.experimental import enable_iterative_imputer # Required for IterativeImputer
    from sklearn.impute import SimpleImputer, IterativeImputer
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    import xgboost as xgb
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    import matplotlib.pyplot as plt
    import seaborn as sns
    print("Libraries installed and imported.")


# Suppress warnings for cleaner output in the notebook (optional)
warnings.filterwarnings('ignore')

print("--- Setup and Imports Complete ---")

# --- 2. Load and Merge Data ---

print("\n--- Starting Data Loading and Merging ---")

# --- File Paths ---
# Adjust these paths if your files are in Google Drive or a different location
pls_sem_file = '/content/PLS_SEM_SE_Pre.csv'
esg_panel_file = '/content/ESG_panel_processed.csv'

# Initialize df variables to None
df_pls = None
df_esg = None

# --- Load Datasets ---
try:
    df_pls = pd.read_csv(pls_sem_file)
    print(f"Successfully loaded data from '{pls_sem_file}'.")
except FileNotFoundError:
    print(f"Error: Input file '{pls_sem_file}' not found. Make sure it's uploaded or the path is correct.")
    # No exit here, let's try loading the second file anyway, maybe only one is needed for some steps
except Exception as e:
    print(f"An error occurred during loading '{pls_sem_file}': {e}")


try:
    df_esg = pd.read_csv(esg_panel_file)
    print(f"Successfully loaded data from '{esg_panel_file}'.")
except FileNotFoundError:
    print(f"Error: Input file '{esg_panel_file}' not found. Make sure it's uploaded or the path is correct.")
    # No exit here
except Exception as e:
    print(f"An error occurred during loading '{esg_panel_file}': {e}")

# --- Check if both DataFrames were loaded successfully ---
if df_pls is None or df_esg is None:
    print("\n--- Data Loading Failed ---")
    print("Cannot proceed with merging and analysis as one or both input files failed to load.")
    # Skip subsequent steps by using a flag or exiting more cleanly
    data_loading_successful = False
else:
    data_loading_successful = True
    print("Both input files loaded successfully. Proceeding with merging.")

    # --- Clean Column Names (Remove leading/trailing whitespace) ---
    df_pls.columns = df_pls.columns.str.strip()
    df_esg.columns = df_esg.columns.str.strip()
    print("Cleaned column names.")

    # --- Merge Datasets ---
    # Assuming 'Country' and 'Year' are the common columns for merging
    # Use an inner merge to keep only country-year pairs present in both datasets
    identifier_cols = ['Country', 'Year']
    # Check if identifier columns exist in both dataframes before merging
    if all(col in df_pls.columns for col in identifier_cols) and all(col in df_esg.columns for col in identifier_cols):
        merged_df = pd.merge(df_pls, df_esg, on=identifier_cols, how='inner')
        print(f"Merged data shape: {merged_df.shape}")
        # print(f"Merged data columns: {merged_df.columns.tolist()}") # Optional: Print all columns

        # --- Identify Potential Predictors and Target ---
        # For simplicity, let's use Entrepreneurial_intentions as the target for now.
        # You can change this or loop through multiple targets if needed.
        target_variable = 'Entrepreneurial_intentions' # Example target - ENSURE THIS IS CORRECT

        all_cols = merged_df.columns.tolist()

        # Attempt to convert all potentially numeric columns (excluding identifiers and the target)
        # This helps identify non-numeric columns that might cause errors later
        potential_predictor_cols = [col for col in all_cols if col not in identifier_cols + [target_variable]]

        for col in potential_predictor_cols:
            merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

        # Define predictor columns based on those that are now numeric and were in the potential list
        predictor_cols = [col for col in potential_predictor_cols if pd.api.types.is_numeric_dtype(merged_df[col])]

        # Check if target variable exists and is numeric
        if target_variable not in merged_df.columns:
            print(f"Error: Target variable '{target_variable}' not found in the merged dataset.")
            data_loading_successful = False # Set flag to false if target is missing
        else:
            merged_df[target_variable] = pd.to_numeric(merged_df[target_variable], errors='coerce')
            if not pd.api.types.is_numeric_dtype(merged_df[target_variable]):
                 print(f"Error: Target variable '{target_variable}' is not numeric after conversion attempt.")
                 data_loading_successful = False # Set flag to false if target is not numeric


        if data_loading_successful:
            # Drop rows where the target variable is missing, as imputation is typically for predictors
            merged_df.dropna(subset=[target_variable], inplace=True)
            print(f"Shape after dropping rows with missing target: {merged_df.shape}")

            # Select only the relevant columns for the analysis
            # Ensure all predictor_cols are actually in the DataFrame after potential dropping
            analysis_df = merged_df[identifier_cols + [target_variable] + [col for col in predictor_cols if col in merged_df.columns]].copy()
            # Update predictor_cols list to only include those actually in the analysis_df
            predictor_cols = [col for col in analysis_df.columns if col not in identifier_cols + [target_variable]]

            print("\n--- Data Loading and Merging Complete ---")
            print("Analysis DataFrame Head:")
            print(analysis_df.head())
            print(f"\nTarget Variable: {target_variable}")
            print(f"Predictor Variables ({len(predictor_cols)}): {predictor_cols}")

        else:
            # If target variable was missing or not numeric, stop analysis
            print("\n--- Data Loading and Merging Failed due to target variable ---")
            print(f"Could not find or use the target variable: {target_variable}. Analysis stopped.")
            # analysis_df will not be created or used in subsequent steps

    else:
        print(f"Error: Identifier columns {identifier_cols} not found in one or both dataframes.")
        data_loading_successful = False


# --- 3. Imputation (Handling Missing Data) ---

# Proceed only if data loading was successful and there are columns to impute
if data_loading_successful and (identifier_cols + [target_variable] + predictor_cols):
    print("\n--- Starting Imputation ---")

    # Select columns to impute (all analysis columns except identifiers)
    cols_to_impute = [col for col in analysis_df.columns if col not in identifier_cols]

    # Check initial NaN counts
    initial_nan_counts = analysis_df[cols_to_impute].isnull().sum()
    initial_nan_counts = initial_nan_counts[initial_nan_counts > 0]

    if not initial_nan_counts.empty:
        print("NaN counts before imputation:")
        print(initial_nan_counts)

        # Initialize the imputer
        # max_iter controls the number of imputation rounds
        # random_state for reproducibility
        # IterativeImputer is more advanced, SimpleImputer('mean') is simpler
        imputer = IterativeImputer(max_iter=10, random_state=42) # Using a fixed random_state for reproducibility
        # imputer = SimpleImputer(strategy='mean')

        # Fit and transform the selected columns
        # Use .values to work with the numpy array expected by imputer
        try:
            imputed_data = imputer.fit_transform(analysis_df[cols_to_impute])

            # Put the imputed data back into the DataFrame
            analysis_df[cols_to_impute] = imputed_data
            print("Imputation complete.")

            # Check NaN counts after imputation
            final_nan_counts = analysis_df[cols_to_impute].isnull().sum()
            final_nan_counts = final_nan_counts[final_nan_counts > 0]
            if not final_nan_counts.empty:
                 print("Warning: NaNs still found after imputation:")
                 print(final_nan_counts)
                 # Decide whether to continue or stop if NaNs remain
                 # data_loading_successful = False # Uncomment to stop if NaNs remain
            else:
                print("No missing values found after imputation.")

        except Exception as e:
             print(f"Error during imputation: {e}")
             print("Imputation failed.")
             # If imputation fails, subsequent steps requiring clean data will likely fail
             data_loading_successful = False # Set flag to false


    else:
        print("No missing values found in columns to impute. Imputation skipped.")

    print("--- Imputation Complete ---")
    # print(analysis_df.head()) # Optional: Check head after imputation
else:
    print("\nSkipping Imputation as data loading failed or no columns to impute.")


# --- 4. VIF Analysis (Checking for Multicollinearity) ---

# Proceed only if data loading and (optional) imputation were successful and there are predictors
if data_loading_successful and predictor_cols:
    print("\n--- Starting VIF Analysis ---")

    # Select only the predictor columns for VIF
    X_vif = analysis_df[predictor_cols].copy()

    # Drop rows with NaN in X_vif - should be handled by imputation, but safety check
    # X_vif.dropna(inplace=True) # Should ideally not drop rows if imputation worked

    if X_vif.empty:
         print("Warning: Predictor data is empty after handling NaNs. Skipping VIF.")
    else:
        # Add a constant term for the intercept, required for VIF calculation
        # Use has_constant='add' to avoid adding it multiple times if the code is re-run
        X_vif = sm.add_constant(X_vif, has_constant='add')

        # Ensure no NaNs remain in X_vif values used for VIF calculation
        if np.isnan(X_vif.values).any():
            print("Warning: NaNs detected in VIF input data after imputation. VIF calculation may fail or be inaccurate.")
            print(X_vif.isnull().sum()[X_vif.isnull().sum() > 0]) # Show which columns have NaNs
            # Decide whether to continue or stop if NaNs remain
            # data_loading_successful = False # Uncomment to stop if NaNs remain
        else:
            try:
                # Calculate VIF for each predictor
                vif_data = pd.DataFrame()
                vif_data['Variable'] = X_vif.columns
                # Use the values attribute for the numpy array needed by variance_inflation_factor
                vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

                # Sort by VIF
                vif_data = vif_data.sort_values(by='VIF', ascending=False)

                print("VIF Results:")
                print(vif_data)

                # --- Interpretation ---
                print("\nVIF Interpretation:")
                # Exclude 'const' when interpreting VIFs for actual variables
                high_vif_threshold = 10 # Common strict threshold
                # high_vif_threshold = 5 # More lenient threshold

                high_vif_vars = vif_data[(vif_data['VIF'] > high_vif_threshold) & (vif_data['Variable'] != 'const')]

                if not high_vif_vars.empty:
                    print(f"Warning: The following variables have VIF > {high_vif_threshold}, indicating potential multicollinearity:")
                    print(high_vif_vars)
                    print("Consider removing one or more of these highly correlated predictors, combining them, or using dimension reduction techniques.")
                else:
                    print(f"No variables found with VIF > {high_vif_threshold}. Multicollinearity does not appear to be a major issue based on VIF.")

            except Exception as e:
                print(f"Error during VIF calculation: {e}")
                print("This might happen if there are still NaNs or perfect multicollinearity.")
                # Decide whether to continue or stop if VIF calculation failed
                # data_loading_successful = False # Uncomment to stop

else:
    print("\nSkipping VIF analysis as data loading/imputation failed or no predictor variables were identified.")

print("--- VIF Analysis Complete ---")


# --- 5. Panel Data Regression (PanelOLS) ---

# Proceed only if data loading/imputation was successful, there are predictors, and the target is available
if data_loading_successful and predictor_cols and target_variable in analysis_df.columns and not analysis_df[target_variable].isnull().any():
    print("\n--- Starting Panel Data Regression (PanelOLS) ---")

    # Create a MultiIndex using 'Country' and 'Year' for linearmodels
    # Use the imputed data DataFrame
    panel_df = analysis_df.set_index(identifier_cols)

    # Define the dependent (target) and independent (predictor) variables
    y_panel = panel_df[target_variable]
    X_panel = panel_df[predictor_cols].copy() # Use the same predictors as VIF

    # Add a constant for the intercept
    # Use has_constant='add' to avoid adding it multiple times if the code is re-run
    X_panel = sm.add_constant(X_panel, has_constant='add')

    # Ensure no NaNs remain in PanelOLS input - Should be handled by imputation
    if np.isnan(X_panel.values).any() or np.isnan(y_panel.values).any():
         print("Warning: NaNs detected in PanelOLS input data. This should not happen after successful imputation and dropping missing target rows.")
         print("Skipping PanelOLS.")
    else:
        try:
            # Fit the PanelOLS model
            # entity_effects=True: Fixed effects for Country
            # time_effects=False: No fixed effects for Year (change to True if needed)
            # Other options: `model='random'` for Random Effects (check linearmodels docs)
            # Note: entity_effects=True requires variations WITHIN entities over time for predictors
            # If running Random Effects, use model='random' instead of default 'within' (fixed effects)
            panel_model = PanelOLS(y_panel, X_panel, entity_effects=True, time_effects=False)
            # panel_model = PanelOLS(y_panel, X_panel, entity_effects=True, time_effects=True) # Fixed Entity and Time Effects
            # panel_model = PanelOLS(y_panel, X_panel, entity_effects=False, time_effects=False, model='random') # Random Effects

            panel_results = panel_model.fit()

            # Print the results summary
            print("\nPanelOLS Results Summary:")
            print(panel_results)

            # --- Interpretation ---
            print("\nPanelOLS Interpretation Notes:")
            print(f"- Dependent Variable: {target_variable}")
            print("- Coefficients: Show the estimated effect of each predictor on the target, controlling for entity fixed effects.")
            print("- P-values (P>|t|): Indicate statistical significance (typically < 0.05).")
            print("- R-squared (Within): Measures the proportion of within-entity variance explained.")
            print("- F-statistic (Within): Tests the overall significance of the predictors in explaining within-entity variation.")
            if panel_model.entity_effects:
                print("- Entity Effects: Account for unobserved, time-invariant country-specific factors.")
            if panel_model.time_effects:
                print("- Time Effects: Account for unobserved, entity-invariant year-specific factors.")
            # Reliability of coefficients depends on significance, but overall model validity relies on global fit (not directly assessed here for PanelOLS in the same way as PLS-SEM).

        except Exception as e:
            print(f"\nError fitting PanelOLS model: {e}")
            print("Check the model specification, data (e.g., insufficient within-entity variation for fixed effects), and error messages.")
            # Decide whether to continue or stop if PanelOLS failed
            # data_loading_successful = False # Uncomment to stop


    # Reset index for subsequent analyses if needed (ML models don't use MultiIndex)
    analysis_df = analysis_df.reset_index()

else:
     print("\nSkipping Panel Data Regression as data loading/imputation failed, predictors/target missing, or data contains NaNs.")


print("--- Panel Data Regression Complete ---")


# --- 6. Machine Learning Models (Random Forest and XGBoost) ---

# Proceed only if data loading/imputation was successful and there are predictors and the target is available
if data_loading_successful and predictor_cols and target_variable in analysis_df.columns and not analysis_df[target_variable].isnull().any():
    print("\n--- Starting Machine Learning Models ---")

    # Define features (X) and target (y) for ML models
    # Use the imputed data DataFrame
    X_ml = analysis_df[predictor_cols].select_dtypes(include=np.number).copy()
    y_ml = analysis_df[target_variable].select_dtypes(include=np.number).copy()

    # Ensure no NaNs remain in X or y - Should be handled by imputation
    if np.isnan(X_ml.values).any() or np.isnan(y_ml.values).any():
         print("Warning: NaNs detected in ML input data after imputation. This should not happen after successful imputation and dropping missing target rows.")
         print("Skipping ML models.")
         # Decide whether to continue or stop if NaNs remain
         # data_loading_successful = False # Uncomment to stop if NaNs remain
         y_test_df = None # Clear variables to skip visualization
         rf_feature_importance = None
         xgb_feature_importance = None

    else:
        # Split data into training and testing sets
        # test_size: proportion of the dataset to include in the test split
        # random_state: for reproducibility
        try:
            X_train, X_test, y_train, y_test = train_test_split(X_ml, y_ml, test_size=0.25, random_state=42) # 75% train, 25% test
            print(f"\nData split: Train ({X_train.shape[0]} samples), Test ({X_test.shape[0]} samples)")

            # --- Random Forest Regressor ---
            print("\nFitting Random Forest Regressor...")
            # Initialize and train the model
            rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1) # n_jobs=-1 uses all available cores
            rf_model.fit(X_train, y_train)

            # Make predictions on the test set
            y_pred_rf = rf_model.predict(X_test)

            # Evaluate the model
            rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False) # squared=False gives RMSE
            mae_rf = mean_absolute_error(y_test, y_pred_rf)
            r2_rf = r2_score(y_test, y_pred_rf)

            print(f"Random Forest - RMSE: {rmse_rf:.2f}")
            print(f"Random Forest - MAE: {mae_rf:.2f}")
            print(f"Random Forest - R-squared: {r2_rf:.2f}")

            # Get Feature Importance
            rf_feature_importance = pd.DataFrame({
                'Feature': X_ml.columns,
                'Importance': rf_model.feature_importances_
            }).sort_values(by='Importance', ascending=False)

            # --- XGBoost Regressor ---
            print("\nFitting XGBoost Regressor...")
            # Initialize and train the model
            # objective='reg:squarederror' for regression
            # n_estimators: number of boosting rounds
            # learning_rate: step size shrinkage
            # random_state: for reproducibility
            xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1)
            xgb_model.fit(X_train, y_train)

            # Make predictions on the test set
            y_pred_xgb = xgb_model.predict(X_test)

            # Evaluate the model
            rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False) # squared=False gives RMSE
            mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
            r2_xgb = r2_score(y_test, y_pred_xgb)

            print(f"XGBoost - RMSE: {rmse_xgb:.2f}")
            print(f"XGBoost - MAE: {mae_xgb:.2f}")
            print(f"XGBoost - R-squared: {r2_xgb:.2f}")

            # Get Feature Importance
            xgb_feature_importance = pd.DataFrame({
                'Feature': X_ml.columns,
                'Importance': xgb_model.feature_importances_
            }).sort_values(by='Importance', ascending=False)

            # --- Store predictions for visualization ---
            # Create a DataFrame for visualization data to link predictions back to original indices if needed
            # For simplicity here, we'll just use the test set indices
            y_test_df = y_test.to_frame(name='Actual')
            y_test_df['Predicted_RF'] = y_pred_rf
            y_test_df['Predicted_XGB'] = y_pred_xgb

        except Exception as e:
            print(f"Error during ML model fitting or splitting: {e}")
            print("Skipping ML results visualization.")
            # Clear variables to skip visualization step
            y_test_df = None
            rf_feature_importance = None
            xgb_feature_importance = None

else:
    print("\nSkipping Machine Learning models as data loading/imputation failed, predictors/target missing, or data contains NaNs.")
    # Clear variables to skip visualization step
    y_test_df = None
    rf_feature_importance = None
    xgb_feature_importance = None


print("--- Machine Learning Models Complete ---")


# --- 7. Visualization ---

# Proceed only if data loading/imputation was successful and analysis_df is not empty
if data_loading_successful and not analysis_df.empty:
    print("\n--- Generating visualizations ---")

    try:
        # Plot distributions of target and a few predictors
        # Select top N predictors by variance or mean/median for visualization diversity
        analysis_df_numeric = analysis_df[predictor_cols + [target_variable]].select_dtypes(include=np.number) # Include target here for variance check
        if not analysis_df_numeric.empty:
            # Calculate variance only if there's more than one row
            if analysis_df_numeric.shape[0] > 1:
                col_variances = analysis_df_numeric.var().sort_values(ascending=False)
                # Exclude target from top predictor selection if it was included in variance calculation
                top_predictor_cols_viz = col_variances[col_variances.index != target_variable].head(min(3, len(col_variances)-1)).index.tolist() # Top 3 or fewer predictors
            else:
                top_predictor_cols_viz = [] # Cannot calculate variance with 1 or fewer rows

            num_plots = 1 + len(top_predictor_cols_viz) # Target + selected predictors
            if num_plots > 0:
                plt.figure(figsize=(min(15, num_plots * 5), 5)) # Adjust figure width based on number of plots

                # Target distribution
                ax1 = plt.subplot(1, num_plots, 1)
                sns.histplot(analysis_df[target_variable].dropna(), kde=True, ax=ax1) # Plot non-NaN data for distribution
                ax1.set_title(f'Distribution of\n{target_variable}')

                # Top predictors distributions
                for i, col in enumerate(top_predictor_cols_viz):
                    ax = plt.subplot(1, num_plots, i+2)
                    sns.histplot(analysis_df[col].dropna(), kde=True, ax=ax) # Plot non-NaN data
                    ax.set_title(f'Distribution of\n{col}')

                plt.tight_layout()
                plt.show()
            else:
                print("Not enough numeric data for distribution plots.")

        else:
            print("No numeric data available for distribution plots.")


        # Plot Correlation Heatmap (select a subset of columns if too many)
        # Select top N predictors by correlation with the target for heatmap
        analysis_df_numeric_all = analysis_df[predictor_cols + [target_variable]].select_dtypes(include=np.number)
        if not analysis_df_numeric_all.empty and analysis_df_numeric_all.shape[0] > 1:
             # Calculate correlations, drop NaNs first for this calculation
             corr_matrix_calc = analysis_df_numeric_all.dropna().corr()
             # Handle case where correlation cannot be calculated (e.g., all values same)
             if not corr_matrix_calc.empty and target_variable in corr_matrix_calc.columns:
                correlation_with_target = corr_matrix_calc[target_variable].abs().sort_values(ascending=False)
                # Exclude target itself from top predictor selection
                top_corr_predictor_cols = correlation_with_target[correlation_with_target.index != target_variable].head(min(15, len(correlation_with_target)-1)).index.tolist() # Select top 15 or fewer

                if top_corr_predictor_cols: # Ensure list is not empty
                    plt.figure(figsize=(12, 10))
                    # Use the correlation matrix calculated from non-NaN data
                    sns.heatmap(corr_matrix_calc.loc[top_corr_predictor_cols + [target_variable], top_corr_predictor_cols + [target_variable]],
                                annot=False, cmap='coolwarm') # Set annot=True if you want to see values
                    plt.title('Correlation Heatmap (Top Predictors)')
                    plt.show()
                else:
                     print("Not enough varied numeric predictor data available for correlation heatmap.")
             else:
                 print("Correlation calculation failed for heatmap (check for constant columns).")
        else:
             print("No numeric data available for correlation heatmap.")


        # Plot Predictions vs Actuals for ML Models (only if models were fitted and results stored)
        if 'y_test_df' in locals() and y_test_df is not None and not y_test_df.empty:
            plt.figure(figsize=(12, 5))

            # Plot for Random Forest if prediction column exists
            if 'Predicted_RF' in y_test_df.columns:
                plt.subplot(1, 2, 1)
                plt.scatter(y_test_df['Actual'], y_test_df['Predicted_RF'], alpha=0.5)
                # Add a perfect prediction line
                min_val = y_test_df['Actual'].min()
                max_val = y_test_df['Actual'].max()
                plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2) # Diagonal line
                plt.xlabel("Actual")
                plt.ylabel("Predicted (Random Forest)")
                plt.title("Random Forest: Actual vs Predicted")
                plt.grid(True)
            else:
                plt.subplot(1, 2, 1).set_title('Random Forest Predictions N/A')


            # Plot for XGBoost if prediction column exists
            if 'Predicted_XGB' in y_test_df.columns:
                plt.subplot(1, 2, 2)
                plt.scatter(y_test_df['Actual'], y_test_df['Predicted_XGB'], alpha=0.5)
                # Add a perfect prediction line
                min_val = y_test_df['Actual'].min()
                max_val = y_test_df['Actual'].max()
                plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2) # Diagonal line
                plt.xlabel("Actual")
                plt.ylabel("Predicted (XGBoost)")
                plt.title("XGBoost: Actual vs Predicted")
                plt.grid(True)
            else:
                 plt.subplot(1, 2, 2).set_title('XGBoost Predictions N/A')


            plt.tight_layout()
            plt.show()
        else:
            print("Skipping Prediction vs Actuals plots (ML models not fitted or test data unavailable).")


        # Plot Feature Importance (only if models were fitted and importance stored)
        if 'rf_feature_importance' in locals() and rf_feature_importance is not None or ('xgb_feature_importance' in locals() and xgb_feature_importance is not None):
             # Check if either DataFrame is non-empty before plotting
             if (rf_feature_importance is not None and not rf_feature_importance.empty) or (xgb_feature_importance is not None and not xgb_feature_importance.empty):
                plt.figure(figsize=(12, 6))

                if rf_feature_importance is not None and not rf_feature_importance.empty:
                    plt.subplot(1, 2, 1)
                    sns.barplot(x='Importance', y='Feature', data=rf_feature_importance.head(10)) # Top 10 features
                    plt.title('Random Forest Top 10 Feature Importance')
                else:
                    plt.subplot(1, 2, 1).set_title('Random Forest Feature Importance N/A')


                if xgb_feature_importance is not None and not xgb_feature_importance.empty:
                    plt.subplot(1, 2, 2)
                    sns.barplot(x='Importance', y='Feature', data=xgb_feature_importance.head(10)) # Top 10 features
                    plt.title('XGBoost Top 10 Feature Importance')
                else:
                    plt.subplot(1, 2, 2).set_title('XGBoost Feature Importance N/A')


                plt.tight_layout()
                plt.show()
             else:
                print("Skipping Feature Importance plots (No feature importance data available).")

        else:
             print("Skipping Feature Importance plots (ML models not fitted).")


    except Exception as e:
        print(f"An error occurred during visualization: {e}")
        import traceback
        traceback.print_exc() # Print traceback for debugging visualization errors

else:
    print("\nSkipping visualizations as data loading/imputation failed or analysis data is empty.")


print("--- Analysis and Visualization Complete ---")

--- Starting Setup and Imports ---
Required libraries already installed or successfully imported.
--- Setup and Imports Complete ---

--- Starting Data Loading and Merging ---
Successfully loaded data from '/content/PLS_SEM_SE_Pre.csv'.
Successfully loaded data from '/content/ESG_panel_processed.csv'.
Both input files loaded successfully. Proceeding with merging.
Cleaned column names.
Merged data shape: (312, 51)
Shape after dropping rows with missing target: (312, 51)

--- Data Loading and Merging Complete ---
Analysis DataFrame Head:
   Country  Year  Entrepreneurial_intentions  Perceived_opportunities  \
0   Brazil  2023                       48.71                    65.37   
1   Canada  2023                       14.26                    62.57   
2    Chile  2023                       53.11                    59.42   
3    China  2023                        5.58                    69.21   
4  Croatia  2023                       21.64                    64.09   

   Perceived_capabi

AttributeError: 'Series' object has no attribute 'select_dtypes'

#### Merge dataset

In [1]:
import pandas as pd
import numpy as np
import re
import csv # Import csv for potential quoting options later if needed

print("Script started...")

# --- 1. Load Data ---
try:
    # --- TRY LOADING WB DATA WITH COMMA FIRST ---
    # The error suggests the WB file might not be tab-separated. Let's try comma.
    try:
        print("Attempting to load New_Entrepreneur_WB.csv with comma separator...")
        # Try comma separator
        df_wb = pd.read_csv('New_Entrepreneur_WB.csv', sep=',', engine='python')
        print("Successfully loaded New_Entrepreneur_WB.csv using comma separator.")
    except Exception as e_comma:
        print(f"Comma separator failed for WB data: {e_comma}")
        print("Attempting to load New_Entrepreneur_WB.csv with tab separator as fallback...")
        # Fallback to tab separator if comma fails
        df_wb = pd.read_csv('New_Entrepreneur_WB.csv', sep='\t', engine='python')
        print("Successfully loaded New_Entrepreneur_WB.csv using tab separator.")
    # --- END OF WB LOADING ATTEMPTS ---

    print(f"WB data shape: {df_wb.shape}")
    # print("WB data head:\n", df_wb.head()) # Uncomment for debugging if needed

    # Load PLS-SEM data (Assuming Comma-separated - this seems correct based on no error here)
    df_pls = pd.read_csv('PLS_SEM_SE_Pre.csv', sep=',', engine='python')
    print("Successfully loaded PLS_SEM_SE_Pre.csv")
    print(f"PLS data shape: {df_pls.shape}")
    # print("PLS data head:\n", df_pls.head()) # Uncomment for debugging if needed


except FileNotFoundError as e:
    print(f"Error: File not found: {e}")
    print("Please ensure 'New_Entrepreneur_WB.csv' and 'PLS_SEM_SE_Pre.csv' are in the same directory as the script.")
    exit()
except Exception as e:
    # This catches errors from the fallback tab attempt or PLS loading
    print(f"An critical error occurred during file loading: {e}")
    print("Please check the format (delimiters, quoting) of your CSV files, especially 'New_Entrepreneur_WB.csv'.")
    exit() # Exit if loading fails fundamentally

# --- 2. Initial Cleaning and Standardization ---

# Function to clean column names (lowercase, replace spaces/special chars with _)
def clean_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        # Remove leading/trailing whitespace
        col = col.strip()
        # Replace spaces and special characters (except _) with underscore
        col = re.sub(r'[^A-Za-z0-9_]+', '_', col)
        # Make lowercase
        col = col.lower()
        # Handle potential multiple underscores
        col = re.sub(r'_+', '_', col)
        # Remove leading/trailing underscores
        col = col.strip('_')
        new_cols.append(col)
    df.columns = new_cols
    return df

print("\nCleaning column names...")
# Now df_wb should be defined if loading succeeded with either separator
df_wb = clean_col_names(df_wb)
df_pls = clean_col_names(df_pls)
print("Column names cleaned.")
# print("Cleaned WB columns:", df_wb.columns.tolist())
# print("Cleaned PLS columns:", df_pls.columns.tolist())


# --- Cleaning World Bank Data (df_wb) ---
print("\nCleaning World Bank data...")

# Standardize Country names
country_replacements_wb = {
    r'\*|\xa0': '',  # Remove asterisks and non-breaking spaces (often appear from copy-paste)
    r'\s+$': '', # Remove trailing spaces
    r'^\s+': '', # Remove leading spaces
    'Antigua and Barbuda': 'Antigua and Barbuda', # Example: Ensure consistency if needed later
    'Korea, Rep.': 'South Korea',
    'China\*\*\*': 'China', # Handle specific cases if needed after general cleaning
    'Egypt, Arab Rep.': 'Egypt',
    'Hong Kong SAR, China': 'Hong Kong',
    'Iran, Islamic Rep.': 'Iran',
    'North Macedonia, Rep': 'North Macedonia',
    'Russian Federation': 'Russia',
    'Slovak Republic': 'Slovakia',
    'St.': 'St', # Abbreviate St. consistently if needed
    'Syrian Arab Republic': 'Syria',
    'Türkiye': 'Turkey',
    'United States': 'United States', # Ensure consistent naming
    'Puerto Rico, US': 'Puerto Rico',
    'Venezuela, RB': 'Venezuela',
    'Yemen, Rep.': 'Yemen',
    'Viet Nam': 'Vietnam',
    'Congo, Dem. Rep.': 'Congo, DR', # Use consistent abbreviations if desired
    'Congo, Rep.': 'Congo',
    'Central African Republic': 'Central African Republic', #Keep long names if consistent
    'Dominican Republic': 'Dominican Republic',
    'São Tomé and Príncipe': 'Sao Tome and Principe', # Remove accents for easier matching
    'Côte d’Ivoire': 'Cote d\'Ivoire',
    'Kyrgyz Republic': 'Kyrgyzstan' # Common name
}
# Ensure 'country' column exists before trying to replace
if 'country' in df_wb.columns:
    df_wb['country'] = df_wb['country'].astype(str).replace(country_replacements_wb, regex=True).str.strip()
    print("WB Country names standardized.")
else:
    print("Warning: 'country' column not found in df_wb. Cannot standardize country names.")


# Clean numeric columns (remove commas, convert to numeric)
numeric_cols_wb = ['adult_population', 'number_entrepreneur_llc']
for col in numeric_cols_wb:
    if col in df_wb.columns:
        if df_wb[col].dtype == 'object':
            print(f"Cleaning numeric column (WB): {col}")
            # Replace comma and any non-numeric character except '.' and '-' before coercion
            df_wb[col] = df_wb[col].astype(str).str.replace(',', '', regex=False)
            # Added cleaning for potential spaces within numbers
            df_wb[col] = df_wb[col].astype(str).str.replace(r'\s+', '', regex=True)
            df_wb[col] = pd.to_numeric(df_wb[col], errors='coerce')
        elif pd.api.types.is_numeric_dtype(df_wb[col]):
             print(f"Column {col} is already numeric.")
        else:
             print(f"Column {col} is not object or numeric, type: {df_wb[col].dtype}. Skipping numeric cleaning.")
    else:
        print(f"Warning: Expected numeric column '{col}' not found in WB data.")


# Convert 'new_business_density_rate' to numeric
if 'new_business_density_rate' in df_wb.columns:
    print("Converting 'new_business_density_rate' to numeric...")
    df_wb['new_business_density_rate'] = pd.to_numeric(df_wb['new_business_density_rate'], errors='coerce')
else:
     print("Warning: Column 'new_business_density_rate' not found in WB data.")

# Convert 'year' to numeric (integer)
if 'year' in df_wb.columns:
    print("Converting 'year' to integer...")
    df_wb['year'] = pd.to_numeric(df_wb['year'], errors='coerce').astype('Int64') # Use nullable integer
else:
    print("Warning: Column 'year' not found in WB data.")

print("WB data cleaning finished.")
# print("WB data info after cleaning:\n")
# df_wb.info()
# print("WB data head after cleaning:\n", df_wb.head())
# print("WB Missing values after cleaning:\n", df_wb.isnull().sum())


# --- Cleaning PLS-SEM Data (df_pls) ---
print("\nCleaning PLS-SEM data...")

# Standardize Country names (ensure they match WB format after cleaning)
country_replacements_pls = {
    r'\*|\xa0': '',  # Remove asterisks and non-breaking spaces
    r'\s+$': '', # Remove trailing spaces
    r'^\s+': '', # Remove leading spaces
    'South Korea': 'South Korea', # Already standardized above
    'United States': 'United States',
    'Puerto Rico': 'Puerto Rico',
     'Iran, Islamic Rep.': 'Iran',
     'Korea, Rep.': 'South Korea', # Just in case it appears here too
     'Viet Nam': 'Vietnam',
     'São Tomé and Príncipe': 'Sao Tome and Principe',
     'Côte d’Ivoire': 'Cote d\'Ivoire',
     'Kyrgyz Republic': 'Kyrgyzstan'
     # Add more replacements if needed based on inspection
}
# Ensure 'country' column exists
if 'country' in df_pls.columns:
    df_pls['country'] = df_pls['country'].astype(str).replace(country_replacements_pls, regex=True).str.strip()
    print("PLS Country names standardized.")
else:
    print("Warning: 'country' column not found in df_pls. Cannot standardize country names.")


# Convert 'year' to numeric (integer)
if 'year' in df_pls.columns:
    print("Converting 'year' to integer...")
    df_pls['year'] = pd.to_numeric(df_pls['year'], errors='coerce').astype('Int64')
else:
    print("Warning: Column 'year' not found in PLS data.")


# Ensure all other PLS columns are numeric (except country and year)
print("Converting PLS columns to numeric...")
pls_numeric_cols = [col for col in df_pls.columns if col not in ['country', 'year']]
for col in pls_numeric_cols:
     if col in df_pls.columns:
        if df_pls[col].dtype == 'object': # Only convert if it's currently an object type
             print(f"Converting PLS column to numeric: {col}")
             df_pls[col] = pd.to_numeric(df_pls[col], errors='coerce')
        elif pd.api.types.is_numeric_dtype(df_pls[col]):
            # Already numeric, do nothing
            pass
        else:
            print(f"PLS column {col} has non-numeric, non-object type: {df_pls[col].dtype}. Skipping conversion.")
     else:
          print(f"Warning: Expected numeric column '{col}' not found in PLS data during numeric conversion check.")


print("PLS data cleaning finished.")
# print("PLS data info after cleaning:\n")
# df_pls.info()
# print("PLS data head after cleaning:\n", df_pls.head())
# print("PLS Missing values after cleaning:\n", df_pls.isnull().sum())


# --- 3. Merge Data ---
print("\nMerging datasets...")
# Check if essential columns exist before merging
if 'country' not in df_wb.columns or 'year' not in df_wb.columns:
    print("Error: Cannot merge because 'country' or 'year' column is missing from WB data after cleaning.")
    exit()
if 'country' not in df_pls.columns or 'year' not in df_pls.columns:
    print("Error: Cannot merge because 'country' or 'year' column is missing from PLS data after cleaning.")
    exit()

# Use an outer merge to keep all rows initially, identify gaps
merged_df = pd.merge(df_wb, df_pls, on=['country', 'year'], how='outer', suffixes=('_wb', '_pls'))
print(f"Merged data shape: {merged_df.shape}")
# print("Merged data info:\n")
# merged_df.info()
# print("Merged data head:\n", merged_df.head())

# --- 4. Imputation ---
print("\nHandling missing values (Imputation)...")
print("Missing values BEFORE imputation:\n", merged_df.isnull().sum().sort_values(ascending=False))

# Identify all numeric columns in the merged dataframe
numeric_cols_merged = merged_df.select_dtypes(include=np.number).columns.tolist()
# Exclude 'year' if it's considered numeric but shouldn't be imputed
if 'year' in numeric_cols_merged:
    numeric_cols_merged.remove('year')

print(f"\nNumeric columns identified for potential imputation: {len(numeric_cols_merged)}")

# Simple Imputation Strategy: Fill NaN in numeric columns with the column's median
# Note: This is a basic strategy. More sophisticated methods (like group-wise median,
# KNNImputer, IterativeImputer) might be better depending on the analysis needs.
imputed_count = 0
for col in numeric_cols_merged:
    if merged_df[col].isnull().any():
        median_val = merged_df[col].median()
        # Handle case where median might be NaN (if all values were NaN)
        if pd.isna(median_val):
            median_val = 0 # Default to 0 if median cannot be calculated
            print(f"Warning: Could not calculate median for column '{col}'. Imputing with 0.")
        merged_df[col].fillna(median_val, inplace=True)
        # print(f"Imputed column '{col}' with median value: {median_val:.2f}") # Reduce verbosity
        imputed_count += 1

if imputed_count == 0:
    print("No missing numeric values found to impute.")
else:
     print(f"\nImputed {imputed_count} numeric columns with their respective medians (or 0 if median failed).")

print("\nMissing values AFTER imputation (numeric columns):\n", merged_df[numeric_cols_merged].isnull().sum())

# Handle potential remaining non-numeric NaNs (like 'country' if outer merge didn't match)
# For fsQCA/PLS-SEM, rows with missing Country/Year are usually unusable.
initial_rows = merged_df.shape[0]
merged_df.dropna(subset=['country', 'year'], inplace=True)
rows_dropped = initial_rows - merged_df.shape[0]
if rows_dropped > 0:
    print(f"\nDropped {rows_dropped} rows with missing 'country' or 'year' (likely due to unmatched merge).")

# --- 5. Final Preparation (Data Types) ---
# Ensure year is integer type after potential merge/imputation effects
if 'year' in merged_df.columns:
     merged_df['year'] = merged_df['year'].astype(int) # Convert back to standard int if no NaNs remain

# Ensure other numeric columns have appropriate float types
for col in numeric_cols_merged: # Re-iterate over originally identified numeric cols
    if col in merged_df.columns: # Check if column still exists
         if not pd.api.types.is_float_dtype(merged_df[col]) and not pd.api.types.is_integer_dtype(merged_df[col]):
              print(f"Converting column {col} to float.")
              # Try converting to float, coercing errors if any value became non-numeric somehow
              merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
              if merged_df[col].isnull().any():
                   print(f"Warning: Column {col} introduced NaNs during final float conversion. Re-imputing with 0.")
                   # Handle unexpected NaNs (e.g., fill with 0 or re-impute median)
                   merged_df[col].fillna(0, inplace=True) # Or use .median() again

print("\nFinal data types check:")
merged_df.info()

# Sort for better readability (optional)
merged_df.sort_values(by=['country', 'year'], inplace=True)

# --- 6. Save Prepared Data ---
output_base_filename = 'merged_entrepreneur_data_prepared'
excel_filename = f'{output_base_filename}.xlsx'
csv_filename = f'{output_base_filename}.csv'

print(f"\nSaving prepared data to {excel_filename} and {csv_filename}...")
try:
    merged_df.to_excel(excel_filename, index=False, engine='openpyxl')
    merged_df.to_csv(csv_filename, index=False)
    print("Files saved successfully.")
except Exception as e:
    print(f"An error occurred during file saving: {e}")

print("\nScript finished.")

Script started...
Attempting to load New_Entrepreneur_WB.csv with comma separator...
Successfully loaded New_Entrepreneur_WB.csv using comma separator.
WB data shape: (2365, 5)
Successfully loaded PLS_SEM_SE_Pre.csv
PLS data shape: (582, 17)

Cleaning column names...
Column names cleaned.

Cleaning World Bank data...
WB Country names standardized.
Cleaning numeric column (WB): adult_population
Cleaning numeric column (WB): number_entrepreneur_llc
Converting 'new_business_density_rate' to numeric...
Converting 'year' to integer...
WB data cleaning finished.

Cleaning PLS-SEM data...
PLS Country names standardized.
Converting 'year' to integer...
Converting PLS columns to numeric...
PLS data cleaning finished.

Merging datasets...
Merged data shape: (2551, 20)

Handling missing values (Imputation)...
Missing values BEFORE imputation:
 entrepreneurial_intentions              1969
fear_failure_rate                       1969
perceived_capabilities                  1969
perceived_opportunit

<ipython-input-1-b78bf17539d6>:255: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df[col].fillna(median_val, inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 2547 entries, 0 to 2546
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   country                               2547 non-null   object 
 1   year                                  2547 non-null   int64  
 2   adult_population                      2547 non-null   float64
 3   number_entrepreneur_llc               2547 non-null   float64
 4   new_business_density_rate             2547 non-null   float64
 5   perceived_opportunities               2547 non-null   float64
 6   perceived_capabilities                2547 non-null   float64
 7   fear_failure_rate                     2547 non-null   float64
 8   entrepreneurial_intentions            2547 non-null   float64
 9   entrepreneurial_tea                   2547 non-null   float64
 10  established_business_ownership        2547 non-null   float64
 11  entrepreneurial_employ

### Code to Prepare WB new Entrepreneurship LLC data and GEM for fsQCA

In [2]:
import pandas as pd
import numpy as np

print("Script started...")

# --- Configuration ---
input_filename = 'merged_entrepreneur_data_prepared.csv'
output_filename = 'calibrated_fsQCA_data.csv'

# !!! CRITICAL: DEFINE YOUR CALIBRATION PARAMETERS HERE !!!
# Structure: 'column_name_to_calibrate': (threshold_for_0, threshold_for_0.5, threshold_for_1)
# Replace 'column_name_...' with actual columns from your merged file.
# Replace the numeric tuples with YOUR anchors based on theory/evidence.
# --- These are PLACEHOLDERS - YOU MUST CHANGE THEM ---
calibration_params = {
    # Example WB Variables (replace with actual names if needed & decide thresholds)
    'adult_population': (100000, 10000000, 100000000), # Placeholder: e.g., <100k=0, 10M=0.5, >100M=1
    'number_entrepreneur_llc': (100, 5000, 50000),      # Placeholder: e.g., <100=0, 5k=0.5, >50k=1
    'new_business_density_rate': (0.1, 2.0, 10.0),     # Placeholder: e.g., <0.1=0, 2.0=0.5, >10=1

    # Example PLS Variables (Use the cleaned names from the previous step)
    # Add ALL conditions and the outcome variable you intend to use in fsQCA
    'perceived_opportunities': (20, 50, 80),          # Placeholder: e.g., <20%=0, 50%=0.5, >80%=1
    'perceived_capabilities': (30, 60, 85),           # Placeholder
    'fear_failure_rate': (20, 40, 60),             # Placeholder - Note: High score might mean LOW fear conceptually for fsQCA
    'entrepreneurial_intentions': (5, 20, 40),        # Placeholder
    'entrepreneurial_tea': (3, 10, 25),               # Placeholder (Total Early-stage Entrepreneurial Activity)
    'established_business_ownership': (2, 8, 15),   # Placeholder
    'innovation': (10, 30, 50),                    # Placeholder (Assuming this is % innovation)
    'high_status_successful_entrepreneurs': (50, 75, 90), # Placeholder
    'entrepreneurship_good_career_choice': (50, 70, 85) # Placeholder

    # --- ADD ALL OTHER VARIABLES (CONDITIONS & OUTCOME) YOU NEED FOR fsQCA HERE ---
    # 'your_outcome_variable': (threshold_0, threshold_0.5, threshold_1),
    # 'another_condition': (threshold_0, threshold_0.5, threshold_1),
}
# --- End Configuration ---


# --- Calibration Function (Direct Method) ---
def calibrate_direct(series, thresh_0, thresh_05, thresh_1, col_name=""):
    """
    Calibrates a pandas Series using the direct method with 3 anchors.

    Args:
        series (pd.Series): The data series to calibrate.
        thresh_0 (float): The threshold for full non-membership (score = 0).
        thresh_05 (float): The threshold for the crossover point (score = 0.5).
        thresh_1 (float): The threshold for full membership (score = 1).
        col_name (str): Optional name of the column for warning messages.

    Returns:
        pd.Series: The calibrated series with scores between 0 and 1.
    """
    # Input validation
    if not (thresh_0 < thresh_05 < thresh_1):
        print(f"Warning: Anchors for '{col_name}' are not strictly increasing ({thresh_0}, {thresh_05}, {thresh_1}). Calibration might be invalid.")
        # Decide how to handle: return NaNs, original data, or attempt anyway?
        # Returning NaNs might be safest to flag the issue.
        return pd.Series([np.nan] * len(series), index=series.index)

    # Ensure denominators are not zero
    denom_upper = thresh_1 - thresh_05
    denom_lower = thresh_05 - thresh_0
    if denom_upper == 0 or denom_lower == 0:
         print(f"Warning: Crossover anchor is identical to full membership or non-membership anchor for '{col_name}' ({thresh_0}, {thresh_05}, {thresh_1}). Cannot calibrate.")
         return pd.Series([np.nan] * len(series), index=series.index)


    # Define conditions for np.select
    conditions = [
        series >= thresh_1,                       # Fully in (or above)
        series <= thresh_0,                       # Fully out (or below)
        (series >= thresh_05) & (series < thresh_1), # Above crossover, below full membership
        (series > thresh_0) & (series < thresh_05)   # Below crossover, above full non-membership
        # Note: Values exactly equal to thresh_05 will be handled by the third condition.
    ]

    # Define corresponding choices (calibration formulas)
    choices = [
        1.0,
        0.0,
        0.5 + 0.5 * (series - thresh_05) / denom_upper,
        0.5 * (series - thresh_0) / denom_lower
    ]

    # Apply calibration using np.select
    # Default is np.nan for values exactly equal to thresh_05 if condition logic had gaps,
    # or for original NaNs, or unexpected values. Should handle original NaNs correctly.
    calibrated_series = np.select(conditions, choices, default=np.nan)

    # Convert the result back to a pandas Series with the original index
    return pd.Series(calibrated_series, index=series.index)

# --- Load Data ---
print(f"\nLoading data from {input_filename}...")
try:
    df = pd.read_csv(input_filename)
    print("Data loaded successfully.")
    print(f"Original shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: Input file not found at {input_filename}")
    exit()
except Exception as e:
    print(f"An error occurred loading {input_filename}: {e}")
    exit()

# --- Apply Calibration ---
print("\nStarting calibration process...")
calibrated_df = df.copy() # Work on a copy to keep original + calibrated

for col, anchors in calibration_params.items():
    if col in calibrated_df.columns:
        print(f"Calibrating column: '{col}' with anchors {anchors}")
        thresh_0, thresh_05, thresh_1 = anchors
        calibrated_col_name = f"{col}_cal" # Create a new name for the calibrated column

        # Perform calibration
        calibrated_series = calibrate_direct(calibrated_df[col], thresh_0, thresh_05, thresh_1, col_name=col)

        # Add the calibrated series to the dataframe
        calibrated_df[calibrated_col_name] = calibrated_series

        # Check for NaNs introduced during calibration (e.g., due to bad anchors)
        if calibrated_series.isnull().any():
             original_nan_count = calibrated_df[col].isnull().sum()
             new_nan_count = calibrated_series.isnull().sum()
             if new_nan_count > original_nan_count:
                  print(f"  -> Warning: NaNs were introduced or increased during calibration for '{col}'. Check anchors or original data.")

    else:
        print(f"Warning: Column '{col}' specified for calibration not found in the dataframe. Skipping.")

print("\nCalibration process finished.")

# Display info about the new dataframe
print("\nCalibrated dataframe info:")
calibrated_df.info()
# print("\nFirst 5 rows of calibrated data (selected columns):")
# cols_to_show = ['country', 'year'] + list(calibration_params.keys()) + [f"{c}_cal" for c in calibration_params.keys() if c in df.columns]
# print(calibrated_df[cols_to_show].head())

# --- Save Calibrated Data ---
print(f"\nSaving calibrated data to {output_filename}...")
try:
    calibrated_df.to_csv(output_filename, index=False)
    print("Calibrated file saved successfully.")
except Exception as e:
    print(f"An error occurred saving the calibrated file: {e}")

print("\nScript finished.")

Script started...

Loading data from merged_entrepreneur_data_prepared.csv...
Data loaded successfully.
Original shape: (2547, 20)

Starting calibration process...
Calibrating column: 'adult_population' with anchors (100000, 10000000, 100000000)
Calibrating column: 'number_entrepreneur_llc' with anchors (100, 5000, 50000)
Calibrating column: 'new_business_density_rate' with anchors (0.1, 2.0, 10.0)
Calibrating column: 'perceived_opportunities' with anchors (20, 50, 80)
Calibrating column: 'perceived_capabilities' with anchors (30, 60, 85)
Calibrating column: 'fear_failure_rate' with anchors (20, 40, 60)
Calibrating column: 'entrepreneurial_intentions' with anchors (5, 20, 40)
Calibrating column: 'entrepreneurial_tea' with anchors (3, 10, 25)
Calibrating column: 'established_business_ownership' with anchors (2, 8, 15)
Calibrating column: 'innovation' with anchors (10, 30, 50)
Calibrating column: 'high_status_successful_entrepreneurs' with anchors (50, 75, 90)
Calibrating column: 'entrep